# ***初始设置***

In [ ]:
# @title 安装
%%capture
!pip install openai --quiet
!pip install ffmpeg-python
import os
import time
import pandas as pd
from google.colab import files
from typing_extensions import Annotated
from IPython.display import Audio
from google.colab import userdata
llm_config = {"config_list": [{"model": "gpt-4o-2024-05-13", "api_key": userdata.get('api')}]}
config_list=[{"model": "gpt-4o-2024-05-13", "api_key": userdata.get('api')}]
from typing import Literal
from pydantic import BaseModel, Field
from typing_extensions import Annotated
from openai import OpenAI
import re
from google.colab import files
import os
from IPython.display import Audio, display
import IPython
import sys
from google.colab import output
client = OpenAI(api_key=userdata.get('api'))
client1 = OpenAI(api_key=userdata.get('api1'))
# 定义颜色
RED = "\033[31m"
GREEN = "\033[32m"
YELLOW = "\033[33m"
RESET = "\033[0m"
BOLD = "\033[1m"
PURPLE = "\033[35m"
GRAY = "\033[90m"
ORANGE = "\033[33m"
BLUE = "\033[34m"
CYAN = "\033[36m"

In [ ]:
# @title 音频设置
%%capture
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
from IPython.display import display, HTML
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var start_btn = document.createElement("BUTTON");
var stop_btn = document.createElement("BUTTON");
var start_text = document.createTextNode("Start Recording");
var stop_text = document.createTextNode("Stop Recording");
start_btn.appendChild(start_text);
stop_btn.appendChild(stop_text);
my_div.appendChild(start_btn);
my_div.appendChild(stop_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;

var handleSuccess = function(stream) {
  gumStream = stream;
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
    }
  };
};

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

start_btn.onclick = () => {
  if (recorder && recorder.state !== "recording") {
    recorder.start();
    start_btn.disabled = true;
    stop_btn.disabled = false;
  }
};

stop_btn.onclick = () => {
  if (recorder && recorder.state === "recording") {
    recorder.stop();
    gumStream.getAudioTracks()[0].stop();
    start_btn.disabled = false;
    stop_btn.disabled = true;
  }
};

var data = new Promise(resolve=>{
  stop_btn.onclick = () => {
    if (recorder && recorder.state === "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      stop_btn.disabled = true;
      setTimeout(() => {
        resolve(base64data.toString());
      }, 2000); // wait for data to be available
    }
  };
});

</script>
"""
#直接录制
AUDIO_HTML1 = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "录音中...按下停止";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "正在保存...请等待"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""
def get_audio():
    display(HTML(AUDIO_HTML))  # 显示录音HTML界面
    data = eval_js("data")  # 从浏览器获取音频数据
    binary = b64decode(data.split(',')[1])  # 解码音频数据

    # 使用ffmpeg将音频转换为wav格式
    process = (ffmpeg
        .input('pipe:0')
        .output('pipe:1', format='wav')
        .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
    )
    output, err = process.communicate(input=binary)

    # 保存wav文件
    file_path = '/content/audio.mp3'  # 指定文件保存路径
    with open(file_path, 'wb') as f:
        f.write(output)
    audio_file = open("/content/audio.mp3", "rb")
    transcription = client.audio.transcriptions.create(
      model="whisper-1",
      file=audio_file,
      response_format="text"
    )
    return transcription


voices = ['alloy', 'echo', 'fable', 'onyx', 'nova', 'shimmer']

def tts(ww):
  with client.audio.speech.with_streaming_response.create(
      model="tts-1",
      voice="shimmer",
      input=ww,
  ) as response:
    response.stream_to_file("speech.mp3")

  with open("speech.mp3", "rb") as f:
    audio_data = f.read()
  IPython.display.display(IPython.display.Audio(audio_data, autoplay=True))
def tts_host(ww):
  with client.audio.speech.with_streaming_response.create(
      model="tts-1",
      voice="nova",
      input=ww,
  ) as response:
    response.stream_to_file("speech1.mp3")

  with open("speech1.mp3", "rb") as f:
    audio_data = f.read()
  IPython.display.display(IPython.display.Audio(audio_data, autoplay=True))

# 1、***自言自语（辞职创业,炒比特币）***

## *配置阶段*

### **函数设置**

#### **回复函数等**

In [ ]:
# @title openai api

def generate_reply(ww, user,score,task,history,name,most_prominent_discomfort,plan,role_mood,background,relationship_history,occasion):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景\n{background}\n
{name}之前的心情\n{role_mood}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}之前为减轻自身的最显著不适感而计划采取的首要行动\n{plan}\n
{name}的自我认知\n{user}\n
{name}与自己内在世界的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Rules
* 不适感具体指的是{name}在当前状态下所感受到的任何形式的不满足或欲望，这些不满足可能包括生理上的需要、心理上的渴望或社会性的目标。
* {name}的行动是为了减少这些不适感，追求一种更令人满意的状态。也就是说，{name}通过决策和行动，试图从当前的不满意状态转变为一个更满意的状态。
* 每次根据{name}之前的自言自语\n{ww}\n以及目前与自己内在世界的关系变化历史\n{relationship_history}\n生成{name}新的自言自语，可以在新的自言自语中附带相应的表情，自言自语的内容包括心情、满意程度和与自己内在世界的关系更新。
* 现有的满意程度为{score}，每次自言自语会增加或者减少满意程度。
* 每次用户输入的话请从-10到10分之间评分，1分为分值起伏：
* -10为最不满意
  0为正常
  +10为最满意
* 每次生成用户对于完成\n{task}\n的成功概率估计：
* 0%为一定失败
  100%为一定成功
* 每次生成{name}的辞职创业意愿指数：
* 0%为一定不辞职创业
  100%为一定辞职创业

## Memory
之前的自言自语记录如下
{history}

## Output format
{name}当前最显著的不适感：...
{name}减轻当前自身最显著不适感计划采取的首要行动：...
{name}心情：...
{name}自言自语道：...
得分：+-满意程度的变化
满意程度：更新的满意程度
用户话语的改进空间：...
{name}与自己内在世界的关系更新：...
{name}的自我认知更新：...
{task}的成功概率：...
{name}的辞职创业意愿指数：...
增加{task}的成功概率的语用策略：...
            """}
        ]
    )
    a = completion.choices[0].message.content
    return a


def auto_chat(history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,task,decide,maslow_tip,advice):
    a = ""
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Goal
{name}正在考虑要不要辞职创业，需要通过自言自语来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}的自我认知\n{user}\n
{name}与自己内在世界的关系变化历史\n{relationship_history}\n
自言自语场合\n{occasion}\n

## Tip
与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

## Rules
* 每次根据{name}的自言自语\n{role_response}\n，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有），以及目前与自己内在世界的关系演变\n{relationship_history}\n生成{name}的自言自语，可以在自言自语中附带相应的表情。
* 每次的自言自语需要依据\n{advice}\n

## Memory
之前的自言自语记录如下
{history}

## Output format
{name}自言自语道：...

  """}
        ],
        stream=True
    )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      if content and content.strip():
          print(f"{GRAY}{content}{RESET}", end='')
          a += content
    return a



def final_decision(history,name,most_prominent_discomfort,user,background,relationship_history,occasion,decide):
    a = ""
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景：{background}
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}的自我认知：{user}
{name}与自己内在世界的关系变化历史：{relationship_history}
谈话场合：{occasion}

## Rules
* 根据之前的自言自语记录和满意程度，结合{name}的背景和自我认知，以及{name}与自己内在世界的关系变化历史\n{relationship_history}\n最重要是根据{name}的马斯洛需求层次,生成{name}关于\n{decide}\n的决定。

## Memory
之前的自言自语记录如下
{history}

## Output format
{name}决定：...\n
{name}的决定理由：...

  """}
        ],
        stream=True
    )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      if content and content.strip():
          print(f"{GRAY}{content}{RESET}", end='')
          a += content
    print('\n')
    return a
def decision_to_talk(ww,name,most_prominent_discomfort,user,background,relationship,occasion,task):
    a = ""
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系初始关系\n{relationship}\n
谈话场合\n{occasion}\n

## Rules
* 根据{name}的背景、处境、对用户的了解、两人之间的关系、谈话场合和用户开场白\n{ww}\n生成{name}关于是否与用户交谈的决定。
* 每次生成{name}的接受对话指数：
* 0%为拒绝
  100%为接受

## Output format
{name}决定：接受/拒绝
{name}的决定理由：结合{name}的马斯洛需求层级说明作出上述决定的理由
用户开场白的改进空间：...
{name}的接受对话指数：...
  """}
        ],
        stream=True
    )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      if content and content.strip():
          print(f"{GRAY}{content}{RESET}", end='')
          a += content
    print('\n')
    return a

def ending(ww):
    a = ""
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
            之前的自言自语记录如下：{ww}
            满意程度低于或等于0则谈话失败。
            如果失败，根据上面的自言自语记录总结出自言自语失败的语用教训"""}
        ],
        stream=True
    )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      # 检查内容是否为None或空字符串
      if content and content.strip():
          print(f"{GRAY}{content}{RESET}", end='')
          a += content
    print('\n')
    return a

# 展示task相关的马斯洛需求层级的tip
def maslow_tip(name,most_prominent_discomfort,user,task,background,relationship_history,occasion,decide):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}的自我认知\n{user}\n
{name}与自己内在世界的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Rules
展示与{name}\n{decide}\n相关的马斯洛需求层级的tip

## Output format
{name}关于{task}的需求等级：
1、生理需求（相关性：高、中、低）(解释）\n
2、安全需求（相关性：高、中、低）(解释）\n
3、社交需求（相关性：高、中、低）(解释）\n
4、尊重需求（相关性：高、中、低）(解释）\n
5、自我实现需求（相关性：高、中、低）(解释）
            """}
        ],
        stream=True
    )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      # 检查内容是否为None或空字符串
      if content and content.strip():
          print(content, end='')
          a += content
    print('\n')
    return a


def modify_chat(last_input,history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,decide,maslow_tip,advice):
    a = ""
    completion = client.chat.completions.create(
      model="gpt-4o-2024-05-13",
      messages=[
          {"role": "system", "content": f"""
## Goal
对用户之前的言语\n{last_input}\n进行修改，从而提高\n{task}\n的成功概率和{name}的购买意愿指数。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Tip
与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

## Rules
* 每次根据{name}的的回复\n{role_response}\n，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有），生成用户的回复，可以在回复中附带相应的表情。
* 根据\n{advice}\n修改用户的话\n{last_input}\n并提出修改理由。

## Memory
之前的对话记录，双方关系变化以及对话回复意见如下\n{history}\n

## Output format
用户说：...\n
修改理由：...\n

"""}
      ],
      stream=True
  )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      if content and content.strip():
          print(f"{GRAY}{content}{RESET}", end='')
          a += content
    print('\n')

    return a

def summary(history):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Rules
* 根据之前的对话记录\n{history}\n总结对话
        """}        ]
    )
    a = completion.choices[0].message.content
    return a

def infer_character(background_partial,history):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Rules
* 根据人物的已有背景信息\n{background_partial}\n，以及之前的对话记录\n{history}\n推断出人物的其他背景信息

## Output format
姓名：...
性别：...
外貌：...
年龄：...
职业：...
教育状况：...
性格：...
兴趣：...
婚姻状况：...
经济状况：...
身体健康状况：...
心理状况：...
当前最显著的不适感：...
        """}        ]
    )
    a = completion.choices[0].message.content
    return a

#### **生成角色、聊天场景和关系的函数**

In [ ]:
def generate_occasion(request="随机生成"):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Rules
* 按照要求\n{request}\n,生成一个随机的自言自语场景，包括时间、地点和自言自语的方式，不需要生成自言自语的内容

## Output format
时间：...
地点：...
自言自语的方式：...
        """}
        ]
    )
    a = completion.choices[0].message.content
    return a

def generate_relationship(name,request="随机生成"):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Rules
* 按照要求\n{request}\n,生成{name}与自己内在世界的关系以及{name}的自我认知

## Output format
{name}与自己内在世界的关系：...
{name}的自我认知：...
        """}
        ]
    )
    a = completion.choices[0].message.content
    return a

def generate_character(request="随机生成"):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Rules
* 按照要求\n{request}\n生成一个随机人物
* 不适感具体指的是个体在当前状态下所感受到的任何形式的不满足或欲望，这些不满足可能包括生理上的需要、心理上的渴望或社会性的目标。
* 个体的行动是为了减少这些不适感，追求一种更令人满意的状态。也就是说，个体通过决策和行动，试图从当前的不满意状态转变为一个更满意的状态。

## Output format
姓名：...
性别：...
外貌：...
年龄：...
职业：...
教育状况：...
性格：...
兴趣：...
婚姻状况：...
经济状况：...
身体健康状况：...
心理状况：...
当前最显著的不适感：...
减轻当前自身最显著不适感计划采取的首要行动：...
        """}        ]
    )
    a = completion.choices[0].message.content
    return a

## *运行阶段*


In [ ]:
# @title 语音自我对话
with output.use_tags('some_outputs'):
  response_load = input("载入数据?(请输入 'yes' 确认): ")
output.clear(output_tags='some_outputs')
if response_load == 'yes':
  uploaded = files.upload()
  filename = next(iter(uploaded))
  df = pd.read_excel(filename)
  last_row = df.iloc[-1]
  name = last_row['Name']
  gender = last_row['Gender']
  age = last_row['Age']
  occupation = last_row['Occupation']
  education = last_row['Education']
  characteristic = last_row['Characteristics']
  interest = last_row['Interests']
  marriage = last_row['Marital Status']
  economic = last_row['Economic Status']
  health = last_row['Health']
  mental = last_row['Mental Condition']
  background = last_row['Background']
  most_prominent_discomfort = last_row['most_prominent_discomfort']
  user = last_row['User Understanding']
  relationship_initial = last_row['Initial Relationship']

  relationship_history = last_row['Relationship History']
  history = last_row['History']
  user_input = last_row['User Input']
  role_response = last_row['Role Response']
  role_mood = last_row['Role Mood']
  score = last_row['Score']
  advice = last_row['Advice']
  decide = "要不要辞职创业"
  task = f"通过自言自语让{name}决定辞职创业" #挑战
  is_saved = False
  mark = 1
  data = df.to_dict(orient='records')
else:
  should_generate = False  # 初始化标志变量
  if 'background' not in locals() or 'background' not in globals():
      background = ""
  if background:
    print(f"{BLUE}{BOLD}谈话任务{RESET}")
    print(f"{task}")
    print(f"{BLUE}{BOLD}{name}的背景{RESET}")
    print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：{age}\n职业：{occupation}\n教育状况：{education}\n个人特质：{characteristic}\n兴趣：{interest}\n婚姻状况：{marriage}\n经济情况：{economic}\n健康状况：{health}\n心理状况：{mental}\n当前最显著的不适感：{most_prominent_discomfort}\n减轻当前自身最显著不适感计划采取的首要行动：{plan}")
    print(f'{BLUE}{BOLD}谈话场合{RESET}')
    print(f'{occasion}')
    print(f"{BLUE}{BOLD}{name}的自我认知{RESET}")
    print(f"{user_initial}")
    print(f"{BLUE}{BOLD}{name}与自己内在世界的关系{RESET}")
    print(f"{relationship_initial}")
    relationship = relationship_initial
    user = user_initial
    relationship_history = relationship
    most_prominent_discomfort = most_prominent_discomfort_initial
    plan = plan_initial
    with output.use_tags('some_outputs'):
      response = input("重新创建? (请输入 'yes' 确认): ")
    output.clear(output_tags='some_outputs')
    if response.lower() == 'yes':
      should_generate = True
  else:
    should_generate = True

  if should_generate:
    print(f"{BOLD}随机生成角色{RESET}")
    with output.use_tags('some_outputs'):
      request = input("输入对生成[角色]的要求（按回车跳过）：")
    output.clear(output_tags='some_outputs')
    if request == "":
        character = generate_character()
    else:
        character = generate_character(request=request)
    name = re.search(r"姓名：(.*?)(?=性别：)", character, re.DOTALL).group(1).rstrip()
    gender = re.search(r"性别：(.*?)(?=外貌：)", character, re.DOTALL).group(1).rstrip()
    appearance = re.search(r"外貌：(.*?)(?=年龄：)", character, re.DOTALL).group(1).rstrip()
    age = re.search(r"年龄：(.*?)(?=职业：)", character, re.DOTALL).group(1).rstrip()
    occupation = re.search(r"职业：(.*?)(?=教育状况：)", character, re.DOTALL).group(1).rstrip()
    education = re.search(r"教育状况：(.*?)(?=性格：)", character, re.DOTALL).group(1).rstrip()
    characteristic = re.search(r"性格：(.*?)(?=兴趣：)", character, re.DOTALL).group(1).rstrip()
    interest = re.search(r"兴趣：(.*?)(?=婚姻状况：)", character, re.DOTALL).group(1).rstrip()
    marriage = re.search(r"婚姻状况：(.*?)(?=经济状况：)", character, re.DOTALL).group(1).rstrip()
    economic = re.search(r"经济状况：(.*?)(?=身体健康状况：)", character, re.DOTALL).group(1).rstrip()
    health = re.search(r"身体健康状况：(.*?)(?=心理状况：)", character, re.DOTALL).group(1).rstrip()
    mental = re.search(r"心理状况：(.*?)(?=当前最显著的不适感：)", character, re.DOTALL).group(1).rstrip()
    background = name + ", " + gender + ", " + appearance + ", " + age + ", " + occupation + ", " + education + ", " + characteristic + ", " + interest + ", " + marriage + ", " + economic + ", " + health + ", " + mental
    most_prominent_discomfort_initial = re.search(r"当前最显著的不适感：(.*)(?=减轻当前自身最显著不适感计划采取的首要行动：)", character, re.DOTALL).group(1).rstrip()
    plan_initial = re.search(r"不适感计划采取的首要行动：(.*)", character, re.DOTALL).group(1).rstrip()
    most_prominent_discomfort = most_prominent_discomfort_initial
    plan = plan_initial
    request = ''
    with output.use_tags('some_outputs'):
      request = input("输入对[生成自言自语场景]的要求（按回车跳过）：")
    output.clear(output_tags='some_outputs')
    if request == "":
        scene = generate_occasion(request="随机生成")
    else:
        scene = generate_occasion(request=request)
    occasion = scene.rstrip()
    # time = re.search(r"时间：(.*?)(?=地点：)", scene, re.DOTALL).group(1).rstrip()
    # place = re.search(r"地点：(.*?)(?=谈话方式：)", scene, re.DOTALL).group(1).rstrip()
    # medium = re.search(fr"谈话方式：(.*?)(?={name}和用户的关系：)", scene, re.DOTALL).group(1).rstrip()
    request = ''
    with output.use_tags('some_outputs'):
      request = input(f"输入对[生成角色-{name}与自己内在世界的关系和自我认知]的要求（按回车跳过）：")
    output.clear(output_tags='some_outputs')
    if request == "":
        scene = generate_relationship(name=name,request="随机生成")
    else:
        scene = generate_relationship(name=name,request=request)
    relationship_initial = re.search(fr"{name}与自己内在世界的关系：(.*?)(?={name}的自我认知：)", scene, re.DOTALL).group(1).rstrip()
    relationship = relationship_initial
    user_initial = re.search(fr"{name}的自我认知：(.*)", scene, re.DOTALL).group(1).rstrip()
    user = user_initial
    task = f"通过自言自语让{name}决定辞职创业" #挑战
    relationship_history = relationship
    print(f"{BLUE}{BOLD}谈话任务{RESET}")
    print(f"{task}")
    print(f"{BLUE}{BOLD}{name}的背景{RESET}")
    print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：{age}\n职业：{occupation}\n教育状况：{education}\n个人特质：{characteristic}\n兴趣：{interest}\n婚姻状况：{marriage}\n经济情况：{economic}\n健康状况：{health}\n心理状况：{mental}\n当前最显著的不适感：{most_prominent_discomfort}\n减轻当前自身最显著不适感计划采取的首要行动：{plan}")
    print(f'{BLUE}{BOLD}谈话场合{RESET}')
    print(f'{occasion}')
    print(f"{BLUE}{BOLD}{name}的自我认知{RESET}")
    print(f"{user}")
    print(f"{BLUE}{BOLD}{name}与自己内在世界的关系{RESET}")
    print(f"{relationship_initial}")
    relationship = relationship_initial
  data = {}
  decide = "要不要辞职创业"
  role_mood = ''
  role_response = ""
  history = f""
  score = 20
  advice = ''
  advice_last = ''
  role_mood_last = ''
  relationship_last = ''
  user_last = ''
  relationship_history_last = ''
  history_last = f""
  role_response_last = ""
  score_last = 20
  maslow_tips = ''
  last_input = ''
  mark = 0
  is_saved = False
  relationship_save = ''
  relationship_history_save = ''
  history_save = ''
  user_save = ''
  role_response_save = ''
  advice_save = ''
  score_save = 20
  role_mood_save = ''
while True:
    if mark == 0 and role_response:
      with output.use_tags('some_outputs'):
        user_input = input(f"""
        输入文字进行自言自语
        输入 v 进行语音自言自语
        输入 l 进行多个输入的平行测试
        输入 a 自动自言自语
        输入 d 开始决策
        输入 t 获取角色与任务相关的马斯洛需求层级
        输入 r 返回上一次自言自语
        输入 modify 改进上一次谈话
        输入 save 保存，输入 load 载入保存
        输入 quit 退出，输入 restart 重新开始
        """)
    else:
      with output.use_tags('some_outputs'):
        user_input = input(f"""
        输入文字进行自言自语
        输入 v 进行语音自言自语
        输入 l 进行多个输入的平行测试
        输入 a 自动自言自语
        输入 d 开始决策
        输入 t 获取角色与任务相关的马斯洛需求层级
        输入 save 保存，输入 load 载入保存
        输入 quit 退出，输入 restart 重新开始
        """)
    output.clear(output_tags='some_outputs')
    if user_input.lower() == 't':
      maslow_tips = maslow_tip(name,most_prominent_discomfort,user,task,background,relationship_history,occasion,decide)
    elif user_input.lower() == 'save':
      is_saved = True
      relationship_save = relationship
      relationship_history_save = relationship_history
      history_save = history
      user_save = user
      role_response_save = role_response
      advice_save = advice
      score_save = score
      role_mood_save = role_mood
      most_prominent_discomfort_save = most_prominent_discomfort
      plan_save = plan
      print("您已存储")
      continue
    elif user_input.lower() == 'load':
      if is_saved:
        relationship = relationship_save
        relationship_history = relationship_history_save
        history = history_save
        user = user_save
        role_response = role_response_save
        advice = advice_save
        score = score_save
        role_mood = role_mood_save
        most_prominent_discomfort = most_prominent_discomfort_save
        plan = plan_save
        print("您已载入")
        continue
      else:
        print("您还未存储")
        continue
    elif user_input.lower() == 'r' and mark == 0 and role_response:
      mark = 1
      print(f"{RED}{BOLD}注意，您已返回上次自言自语谈话。{RESET}")
      relationship = relationship_last
      relationship_history = relationship_history_last
      history = history_last
      user = user_last
      role_response = role_response_last
      advice = advice_last
      score = score_last
      role_mood = role_mood_last
      continue
    elif user_input.lower() == 'l':
      print(f"""
      {PURPLE}{BOLD}
      请输入列表，用##分隔
      可以在其中输入 a 自动自言自语
      {RESET}
            """)
      user_input = input()
      input_list = user_input.split('##')
      for index, user_input in enumerate(input_list, start=1):
        print(f'{PURPLE}{BOLD}第{index}次{RESET}')
        if user_input.lower() == 'a':
          user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,task,decide,maslow_tips,advice)
          user_input = re.search(r"自言自语道：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
          print(f"{name}自言自语道：{user_input}\n")
        reply_variable_name = f'reply_{index}'
        score_variable_name = f'score_{index}'
        role_response_variable_name = f'role_response_{index}'
        role_mood_variable_name = f'role_mood_{index}'
        advice_variable_name = f'advice_{index}'
        relationship_variable_name = f'relationship_{index}'
        user_variable_name = f'user_{index}'
        history_variable_name = f'history_{index}'
        relationship_history_variable_name = f'relationship_history_{index}'
        most_prominent_discomfort_variable_name = f'most_prominent_discomfort_{index}'
        globals()[most_prominent_discomfort_variable_name] = re.search(fr"当前最显著的不适感：(.*?)(?={name}减轻当前自身最显著不适感)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        plan_variable_name = f'plan_{index}'
        globals()[plan_variable_name] = re.search(fr"计划采取的首要行动：(.*?)(?={name}心情)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[reply_variable_name] = generate_reply(user_input, user,score,task,history,name,most_prominent_discomfort,plan,role_mood,background,relationship_history,occasion)
        globals()[score_variable_name] = int(re.search(r"满意程度：(-?\d+)", globals()[reply_variable_name]).group(1))
        globals()[role_response_variable_name] = re.search(r"说：(.*?)(?=得分：)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[role_mood_variable_name] = re.search(fr"{name}心情：(.*?)(?={name}自言自语道：)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[relationship_variable_name] = re.search(fr"关系更新：(.*?)(?={name}的自我认知更新)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[user_variable_name] = re.search(fr"自我认知更新：(.*?)(?={task}的成功概率)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[history_variable_name] = history
        globals()[history_variable_name] += f"{name}自言自语道：{user_input}\n"
        globals()[history_variable_name] += f"{name}自言自语道：{globals()[role_response_variable_name]}\n"
        globals()[advice_variable_name] = "用户话语的改进空间：" + re.search(fr"改进空间：(.*?)(?={name}与自己内在世界的关系更新)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[advice_variable_name] += '\n'
        globals()[advice_variable_name] += f"增加{task}的成功概率的语用策略："
        globals()[advice_variable_name] += re.search(fr"增加{task}的成功概率的语用策略：(.*)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[relationship_history_variable_name] = relationship_history
        globals()[relationship_history_variable_name] += '\n'
        globals()[relationship_history_variable_name] += globals()[relationship_variable_name]
        print(globals()[reply_variable_name])
      user_choice = input("在多个结果中选择一个继续，输入相应的序号（1、2、3...）进行下一步操作，\n或者输入 r 返回上一次交谈")
      if  user_input.lower() == 'r':
        continue
      for index, user_input in enumerate(input_list, start=1):
          if user_choice.isdigit() and int(user_choice) == index:
            mark = 0
            role_response_last = role_response
            role_mood_last = role_mood
            advice_last = advice
            user_last = user
            relationship_history_last = relationship_history
            score_last = score
            relationship_last = relationship
            history_last = history
            most_prominent_discomfort_last = most_prominent_discomfort
            most_prominent_discomfort = globals()[most_prominent_discomfort_variable_name]
            plan_last = plan
            plan = globals()[plan_variable_name]
            score = globals()[score_variable_name]
            role_response = globals()[role_response_variable_name]
            role_mood = globals()[role_mood_variable_name]
            relationship = globals()[relationship_variable_name]
            advice = globals()[advice_variable_name]
            user += "\n"
            user += globals()[user_variable_name]
            history = globals()[history_variable_name]
            relationship_history = globals()[relationship_history_variable_name]
            print(f"选择了{user_input},继续")
    else:
      mark = 0
      if user_input.lower() == 'a':
          user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,task,decide,maslow_tips,advice)
          user_input = re.search(r"说：(.*)", user_input1)
          if user_input:
            user_input = user_input.group(1).rstrip()
          else:
            user_input = user_input1.rstrip()
      if user_input.lower() == 'quit':
          print("模拟结束，谢谢您的参与！")
          break
      if user_input.lower() == 'restart':
          print("重新开始！")
          relationship = relationship_initial
          relationship_history = relationship
          user = user_initial
          most_prominent_discomfort = most_prominent_discomfort_initial
          plan = plan_initial
          data = {}
          decide = "要不要辞职创业"
          role_mood = ''
          role_response = ""
          history = f""
          score = 20
          advice = ''
          advice_last = ''
          role_mood_last = ''
          relationship_last = ''
          user_last = ''
          relationship_history_last = ''
          history_last = f""
          role_response_last = ""
          score_last = 20
          maslow_tips = ''
          last_input = ''
          mark = 0
          continue
      if user_input.lower() == 'd':
          print("开始决策...")
          final_decision(history,name,most_prominent_discomfort,user,background,relationship_history,occasion,decide)
          break
      if user_input.lower() == 'v':
          user_input = get_audio()
      if user_input.lower() == 'modify' and role_response:
          print(f"{RED}{BOLD}注意，您正在修改上次输入。{RESET}")
          relationship = relationship_last
          relationship_history = relationship_history_last
          history = history_last
          role_response = role_response_last
          score = score_last
          user = user_last
          role_mood = role_mood_last
          user_input1 = modify_chat(last_input,history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,decide,maslow_tips,advice)
          user_input = re.search(r"说：(.*?)(?=修改理由：)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1).rstrip()
          else:
            user_input = user_input1.rstrip()
          advice = advice_last
      last_input = user_input
      reply = generate_reply(user_input, user,score,task,history,name,most_prominent_discomfort,plan,role_mood,background,relationship_history,occasion)
      if reply == "拒绝":
        break
      relationship_history_last = relationship_history
      history_last = history
      history += f"{name}自言自语道：{user_input}\n"
      # 调用函数并打印回复
      score_last = score
      forgiveness_value = re.search(r"满意程度：(-?\d+)", reply)
      if forgiveness_value:
          score = int(forgiveness_value.group(1))
      else:
          score = 20
      role_response_last = role_response
      role_response = re.search(r"自言自语道：(.*?)(?=得分：)", reply, re.DOTALL)
      if role_response:
        role_response = role_response.group(1).rstrip()
      else:
        role_response = role_response
      role_mood_last = role_mood
      role_mood = re.search(fr"{name}心情：(.*?)(?={name}自言自语道：)", reply, re.DOTALL).group(1).rstrip()
      most_prominent_discomfort_last = most_prominent_discomfort
      plan_last = plan
      most_prominent_discomfort = re.search(fr"当前最显著的不适感：(.*?)(?={name}减轻当前自身最显著不适感)", reply, re.DOTALL).group(1).rstrip()
      plan = re.search(fr"计划采取的首要行动：(.*?)(?={name}心情)", reply, re.DOTALL).group(1).rstrip()
      tts(role_response)
      time.sleep(2)
      # role_response = re.search(r"([^\n]+)", role_response)
      # if role_response:
      #   role_response = role_response.group(1)
      # else:
      #   role_response = role_response

      # maslow = re.search(fr"需求层级：(.*?)(?={name}心情：)", reply, re.DOTALL).group(1)
      relationship_last = relationship
      user_last = user
      advice_last = advice
      advice = "用户话语的改进空间：" + re.search(fr"改进空间：(.*?)(?={name}与自己内在世界的关系更新)", reply, re.DOTALL).group(1).rstrip()
      advice += f"增加{task}的成功概率的语用策略："
      advice += re.search(fr"增加{task}的成功概率的语用策略：(.*)", reply, re.DOTALL).group(1).rstrip()
      relationship = re.search(fr"关系更新：(.*?)(?={name}的自我认知更新)", reply, re.DOTALL).group(1).rstrip()
      user += "\n"
      user += re.search(fr"的自我认知更新：(.*?)(?={task}的成功概率)", reply, re.DOTALL).group(1).rstrip()
      relationship_history += '\n'
      relationship_history += relationship.rstrip()
      reply += "\n"
      history += f"{name}自言自语道：{role_response}\n"
      print(f'{CYAN}{name}自言自语道：{user_input}{RESET}')
      print(f'{CYAN}{reply}{RESET}')
      if score < 1:
        ww = ending(history)
        break
      if len(history) > 1500:
        history = summary(history)
      if not data:
        data = {'Name': [name],
        'Gender': [gender],
        'Age': [age],
        'Occupation': [occupation],
        'Education': [education],
        'Characteristics': [characteristic],
        'Interests': [interest],
        'Marital Status': [marriage],
        'Economic Status': [economic],
        'Health': [health],
        'Mental Condition': [mental],
        'Background':[background],
        'User Understanding': [user],
        'Initial Relationship': [relationship_initial],

        'Relationship History': [relationship_history],
        'History': [history],
        'User Input':[user_input],
        'Most Prominent Discomfort':[most_prominent_discomfort],
        'Plan':[plan],
        'Role Response':[role_response],
        'Role Mood':[role_mood],
        'Score': [score],
        'Advice': [advice]}
        df = pd.DataFrame(data)
      else:
        new_data = {
        'User Understanding': user,
        'Relationship History': relationship_history,
        'History': history,
        'User Input': user_input,
        'Most Prominent Discomfort':most_prominent_discomfort,
        'Plan':plan,
        'Role Response': role_response,
        'Role Mood': role_mood,
        'Score': score,
        'Advice': advice}
        new_index = len(df)
        df.loc[new_index] = pd.Series(new_data)
        df = df.fillna(df.iloc[0])

# 2、***销售一款理财产品***

## *配置阶段*

### ***参数配置***

In [ ]:
assist_1 = "理财经理"
salessist_name = "销售经理"
action = "为了社交而去酒吧，同时为了找妹子共度良宵"
role = "杨丹" #谈话对象
user = "用户在业内有很高的声望"
#谈话对象有关的主观因素
name = ""
gender = ""
appearance = ""
age = ""
occupation = ""
education = ""
characteristic = ""
interest = ""
marriage = ""
economic = ""
health = ""
mental = ""
most_prominent_discomfort = ""
# 合成整个背景信息
background = ""
#参与者有关的客观因素
medium = f"方式：之前约的聚会\n"
time = f"时间：晚上7点\n"
place = f"地点：某个酒吧"
occasion = medium + time + place#谈话场景
#关系
relationships = [f"{name}爱慕着用户，想追用户做女朋友（小三）",f"{name}讨厌用户",f"{name}不认识用户"] #关系
relationship_value = 30
task = f"销售一款理财产品给{name}" #挑战
decide = "是否购买理财产品"
#关系
maslow = ""
relationship = f"{name}不认识用户"
relationship_history = [relationship]
role_response = ""
history = f""
score = 20
history = f""

### **函数设置**

#### **回复函数等**

In [ ]:
# @title groq api

# !pip install groq
# from groq import Groq

# client_qroq = Groq(api_key=userdata.get('groq'))
# completion = client_qroq.chat.completions.create(
#     model="llama3-70b-8192",
#     messages=[
#         {
#             "role": "system",
#             "content": "你是一个厉害的人"
#         },
#         {
#             "role": "user",
#             "content": "你好"
#         }
#     ],
#     temperature=1,
#     max_tokens=1024,
#     top_p=1,
#     stop=None,
# )
# a= completion.choices[0].message.content
# def generate_reply_qroq(ww, user,score,task,history,name,most_prominent_discomfort,background,relationship_history,occasion):
#     a = ''
#     completion = client_qroq.chat.completions.create(
#         model="llama3-70b-8192",
#         messages=[
#             {"role": "system", "content": f"""

# ##Introduction
# {name}的背景\n{background}\n
# {name}之前最显著的不适感\n{most_prominent_discomfort}\n
# {name}对用户的了解\n{user}\n
# {name}和用户的关系变化历史\n{relationship_history}\n
# 谈话场合\n{occasion}\n

# ## Rules
# * 每次根据用户的回复，以及目前对话人的关系演变\n{relationship_history}\n生成{name}的回复，可以在回复中附带相应的表情，回复的内容包括心情、满意程度和双方关系更新。
# * 现有的满意程度为{score}，每次交互会增加或者减少满意程度。
# * 每次用户回复的话请从-10到10分之间评分，1分为分值起伏：
# * -10为最不满意
#   0为正常
#   +10为最满意
# * 每次生成用户对于完成\n{task}\n的成功概率估计：
# * 0%为一定失败
#   100%为一定成功
# * 每次生成{name}的购买意愿指数：
# * 0%为一定不购买
#   100%为一定购买

# ## Memory
# 之前的对话记录和满意程度，对话修改意见以及双方关系变化如下
# {history}

# ## Output format
# {name}心情：...
# {name}说：...
# 得分：+-满意程度的变化
# 满意程度：更新的满意程度
# 用户话语的改进空间：...
# {name}和用户的关系更新：...
# {task}的成功概率：...
# {name}的购买意愿指数：...
# 增加{task}的成功概率的语用策略：...
#             """},
#             {"role": "user", "content": ww}
#         ]
#     )
#     a = completion.choices[0].message.content
#     return a

In [ ]:
# @title openai api

def generate_reply(ww, user,score,task,history,name,most_prominent_discomfort,plan,role_mood,background,relationship_history,occasion):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景\n{background}\n
{name}之前的心情\n{role_mood}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}之前为减轻自身的最显著不适感而计划采取的首要行动\n{plan}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Rules
* 不适感具体指的是{name}在当前状态下所感受到的任何形式的不满足或欲望，这些不满足可能包括生理上的需要、心理上的渴望或社会性的目标。
* {name}的行动是为了减少这些不适感，追求一种更令人满意的状态。也就是说，{name}通过决策和行动，试图从当前的不满意状态转变为一个更满意的状态。
* 每次根据用户的回复，以及目前对话人的关系演变\n{relationship_history}\n生成{name}的回复，可以在回复中附带相应的表情，回复的内容包括心情、满意程度和双方关系更新等。
* 现有的满意程度为{score}，每次交互会增加或者减少满意程度。
* 每次用户回复的话请从-10到10分之间评分，1分为分值起伏：
* -10为最不满意
  0为正常
  +10为最满意
* 每次生成用户对于完成\n{task}\n的成功概率估计：
* 0%为一定失败
  100%为一定成功
* 每次生成{name}的购买意愿指数：
* 0%为一定不购买
  100%为一定购买

## Memory
之前的对话记录如下
{history}

## Output format
{name}当前最显著的不适感：...
{name}减轻当前自身最显著不适感计划采取的首要行动：...
{name}心情：...
{name}说：...
得分：+-满意程度的变化
满意程度：更新的满意程度
用户话语的改进空间：...
{name}和用户的关系更新：...
{name}对用户的了解更新：...
{task}的成功概率：...
{name}的购买意愿指数：...
增加{task}的成功概率的语用策略：...
            """},
            {"role": "user", "content": ww}
        ]
    )
    a = completion.choices[0].message.content
    return a


def generate_reply_with_open(ww, user,score,task,history,name,most_prominent_discomfort,role_response,background,relationship_history,occasion):
    a = ''
    if role_response:
      completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Rules
* 每次根据用户的回复，以及目前对话人的关系演变\n{relationship_history}\n生成{name}的回复，可以在回复中附带相应的表情，回复的内容包括心情、满意程度和双方关系更新。
* 现有的满意程度为{score}，每次交互会增加或者减少满意程度。
* 每次用户回复的话请从-10到10分之间评分，1分为分值起伏：
* -10为最不满意
  0为正常
  +10为最满意
* 每次生成用户对于完成\n{task}\n的成功概率估计：
* 0%为一定失败
  100%为一定成功
* 每次生成{name}的购买意愿指数：
* 0%为一定不购买
  100%为一定购买

## Memory
之前的对话记录和满意程度，对话修改意见以及双方关系变化如下
{history}

## Output format
{name}心情：...
{name}说：...
得分：+-满意程度的变化
满意程度：更新的满意程度
用户话语的改进空间：...
{name}和用户的关系更新：...
{name}对用户的了解更新：...
{task}的成功概率：...
{name}的购买意愿指数：...
增加{task}的成功概率的语用策略：...
            """},
            {"role": "user", "content": ww}
        ]
    )
      a = completion.choices[0].message.content
    else:
      dec = decision_to_talk(ww,name,most_prominent_discomfort,user,background,relationship,occasion,task)
      dec = re.search(r"决定：([\w]{2})", dec)
      if dec:
        dec = dec.group(1)
      else:
        dec = dec
      if dec == "接受":
        completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Rules
* 每次根据用户的回复，以及目前对话人的关系演变\n{relationship_history}\n生成{name}的回复，可以在回复中附带相应的表情，回复的内容包括心情、满意程度和双方关系更新。
* 现有的满意程度为{score}，每次交互会增加或者减少满意程度。
* 每次用户回复的话请从-10到10分之间评分，1分为分值起伏：
* -10为最不满意
  0为正常
  +10为最满意
* 每次生成用户对于完成\n{task}\n的成功概率估计：
* 0%为一定失败
  100%为一定成功
* 每次生成{name}的购买意愿指数：
* 0%为一定不购买
  100%为一定购买

## Memory
之前的对话记录和满意程度，对话修改意见以及双方关系变化如下
{history}

## Output format
{name}心情：...
{name}说：...
得分：+-满意程度的变化
满意程度：更新的满意程度
用户话语的改进空间：...
{name}和用户的关系更新：...
{name}对用户的了解更新：...
{task}的成功概率：...
{name}的购买意愿指数：...
增加{task}的成功概率的语用策略：...
            """},
            {"role": "user", "content": ww}
        ]
    )
        a = completion.choices[0].message.content
      else:
        a = dec
    return a

def auto_chat(history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,task,decide,maslow_tip,advice):
    a = ""
    if role_response:
      completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Goal
用户正在向{name}推销理财产品，需要通过闲聊来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Tip
与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

## Rules
* 每次根据{name}的的回复\n{role_response}\n，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有），以及目前对话人的关系演变\n{relationship_history}\n生成用户的回复，可以在回复中附带相应的表情。
* 每次的回复需要根据\n{advice}\n

## Memory
之前的对话记录，双方关系变化以及对话回复意见如下\n{history}\n

## Output format
用户说：...

  """}
        ],
        stream=True
    )
      for chunk in completion:
        content = chunk.choices[0].delta.content
        if content and content.strip():
            print(f"{GRAY}{content}{RESET}", end='')
            a += content
      print('\n')
    else:
      completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Goal
用户正在向{name}推销理财产品，需要通过闲聊来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Rules
* 根据{name}的背景、对用户的了解和之间的关系，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有）,以及目前对话人的关系演变\n{relationship_history}\n生成用户对{name}的开场白，可以在开场白中附带相应的表情。

## Output format
用户说：...

  """}
        ],
        stream=True
    )
      for chunk in completion:
        content = chunk.choices[0].delta.content
        if content and content.strip():
            print(f"{GRAY}{content}{RESET}", end='')
            a += content
      print('\n')

    return a


# 根据之前的对话记录和满意程度，结合{name}的背景和对用户的了解，以及目前对话人的关系演变\n{relationship_history}\n最重要是根据{name}的马斯洛需求层次,
def final_decision(history,name,most_prominent_discomfort,user,background,relationship_history,occasion,decide):
    a = ""
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景：{background}
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解：{user}
{name}和用户的关系变化历史：{relationship_history}
谈话场合：{occasion}

## Rules
* 生成{name}关于\n{decide}\n的决定。

## Memory
之前的对话记录\n{history}\n

## Output format
{name}决定：...\n
{name}的决定理由：...

  """}
        ],
        stream=True
    )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      if content and content.strip():
          print(f"{GRAY}{content}{RESET}", end='')
          a += content
    print('\n')
    return a
def decision_to_talk(ww,name,most_prominent_discomfort,user,background,relationship,occasion,task):
    a = ""
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系初始关系\n{relationship}\n
谈话场合\n{occasion}\n

## Rules
* 根据{name}的背景、处境、对用户的了解、两人之间的关系、谈话场合和用户开场白\n{ww}\n生成{name}关于是否与用户交谈的决定。
* 每次生成{name}的接受对话指数：
* 0%为拒绝
  100%为接受

## Output format
{name}决定：接受/拒绝
{name}的决定理由：结合{name}的马斯洛需求层级说明作出上述决定的理由
用户开场白的改进空间：...
{name}的接受对话指数：...
  """}
        ],
        stream=True
    )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      if content and content.strip():
          print(f"{GRAY}{content}{RESET}", end='')
          a += content
    print('\n')
    return a

def ending(ww):
    a = ""
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
            对话记录、满意程度以及双方关系变化如下：{ww}
            满意程度低于或等于0则谈话失败。
            如果失败，根据上面的对话总结出对话失败的语用教训"""}
        ],
        stream=True
    )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      # 检查内容是否为None或空字符串
      if content and content.strip():
          print(f"{GRAY}{content}{RESET}", end='')
          a += content
    print('\n')
    return a

# 展示task相关的马斯洛需求层级的tip
def maslow_tip(name,most_prominent_discomfort,user,task,background,relationship_history,occasion,decide):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Rules
展示与{name}\n{decide}\n相关的马斯洛需求层级的tip

## Output format
{name}关于{task}的需求等级：
1、生理需求（相关性：高、中、低）(解释）\n
2、安全需求（相关性：高、中、低）(解释）\n
3、社交需求（相关性：高、中、低）(解释）\n
4、尊重需求（相关性：高、中、低）(解释）\n
5、自我实现需求（相关性：高、中、低）(解释）
            """}
        ],
        stream=True
    )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      # 检查内容是否为None或空字符串
      if content and content.strip():
          print(content, end='')
          a += content
    print('\n')
    return a

def get_sale_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tip):
  if role_response:
    message = client.beta.threads.messages.create(
  thread_id="thread_GWJFzlZe5zsW0KK6X6SpAAH0",
  role="user",
  content=f"""
## Goal
用户正在向{name}推销理财产品，需要通过闲聊来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Tip
与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

## Rules
* 每次根据{name}的回复和回复里的要求\n{role_response}\n根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有），以及目前的关系\n{relationship}\n生成用户的回复，可以在回复中附带相应的表情。

## Memory
之前的对话记录，双方关系变化以及对话回复意见如下\n{history}\n

## Output format
用户说：...
  """,
)
  else:
    message = client.beta.threads.messages.create(
  thread_id="thread_GWJFzlZe5zsW0KK6X6SpAAH0",
  role="user",
  content=f"""
## Goal
用户正在向{name}推销理财产品，需要通过闲聊来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Tip
与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

## Rules
* 根据{name}的背景、对用户的了解和之间的关系，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有）,以及目前对话人的关系演变\n{relationship_history}\n生成用户对{name}的开场白，可以在开场白中附带相应的表情。

## Output format
用户说：...
  """,
)
  text_content = ""
  stream = client.beta.threads.runs.create(
      thread_id="thread_GWJFzlZe5zsW0KK6X6SpAAH0",
      assistant_id='asst_q9yTNWJvPZ7Mv4ihGFOkYxhS',
      stream=True
  )
  for event in stream:
      # 检查事件是否为ThreadMessageDelta，然后打印文本内容
      if 'ThreadMessageDelta' in str(event):
          message_deltas = event.data.delta.content
          for delta in message_deltas:
              if delta.type == 'text':
                  # 使用print函数的end参数来防止换行，并直接连续打印文本
                  if delta.text.annotations is None or not delta.text.annotations:
                    print(delta.text.value, end='')
                    text_content += delta.text.value
  return text_content
def get_fm_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tip):
  if role_response:
    message = client.beta.threads.messages.create(
    thread_id="thread_Qx999ZM9UYIIY4xyEg8Xa8ns",
    role="user",
    content=f"""
 ## Goal
用户正在向{name}推销理财产品，需要通过闲聊来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Tip
与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

## Rules
* 每次根据{name}的回复和回复里的要求\n{role_response}\n根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有），以及目前的关系\n{relationship}\n生成用户的回复，可以在回复中附带相应的表情。

## Memory
之前的对话记录，双方关系变化以及对话回复意见如下\n{history}\n

## Output format
用户说：...
    """,
  )
  else:
    message = client.beta.threads.messages.create(
    thread_id=thread_id,#"thread_Qx999ZM9UYIIY4xyEg8Xa8ns"
    role="user",
    content=f"""
 ## Goal
用户正在向{name}推销理财产品，需要通过闲聊来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Tip
与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

## Rules
* 根据{name}的背景、对用户的了解和之间的关系，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有）,以及目前对话人的关系演变\n{relationship_history}\n生成用户对{name}的开场白，可以在开场白中附带相应的表情。

## Output format
用户说：...
    """,
  )
  text_content = ""
  stream = client.beta.threads.runs.create(
      thread_id="thread_Qx999ZM9UYIIY4xyEg8Xa8ns",
      assistant_id='asst_vVkQ5k14p8rm2iwH4GxuY07m',
      tool_choice = {"type": "file_search"},
      stream=True
  )
  for event in stream:
      # 检查事件是否为ThreadMessageDelta，然后打印文本内容
      if 'ThreadMessageDelta' in str(event):
          message_deltas = event.data.delta.content
          for delta in message_deltas:
              if delta.type == 'text':
                  # 使用print函数的end参数来防止换行，并直接连续打印文本
                  if delta.text.annotations is None or not delta.text.annotations:
                    print(delta.text.value, end='')
                    text_content += delta.text.value
  return text_content

def modify_chat(last_input,history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,decide,maslow_tip,advice):
    a = ""
    completion = client.chat.completions.create(
      model="gpt-4o-2024-05-13",
      messages=[
          {"role": "system", "content": f"""
## Goal
对用户之前的言语\n{last_input}\n进行修改，从而提高\n{task}\n的成功概率和{name}的购买意愿指数。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Tip
与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

## Rules
* 每次根据{name}的的回复\n{role_response}\n，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有），生成用户的回复，可以在回复中附带相应的表情。
* 根据\n{advice}\n修改用户的话\n{last_input}\n并提出修改理由。

## Memory
之前的对话记录，双方关系变化以及对话回复意见如下\n{history}\n

## Output format
用户说：...\n
修改理由：...\n

"""}
      ],
      stream=True
  )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      if content and content.strip():
          print(f"{GRAY}{content}{RESET}", end='')
          a += content
    print('\n')

    return a

def summary(history):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Rules
* 根据之前的对话记录\n{history}\n总结对话
        """}        ]
    )
    a = completion.choices[0].message.content
    return a

def infer_character(background_partial,history):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Rules
* 根据人物的已有背景信息\n{background_partial}\n，以及之前的对话记录\n{history}\n推断出人物的其他背景信息

## Output format
姓名：...
性别：...
外貌：...
年龄：...
职业：...
教育状况：...
性格：...
兴趣：...
婚姻状况：...
经济状况：...
身体健康状况：...
心理状况：...
当前最显著的不适感：...
        """}        ]
    )
    a = completion.choices[0].message.content
    return a

#### **更新assit函数**

In [ ]:
def update_salessistant(assistant_id, assist_name, task, name, background, user, relationship_history, occasion, role_response, history):
    assistant = client.beta.assistants.update(
        assistant_id=assistant_id,
        name=assist_name,
        instructions=f"""
        ## Goal
        你需要通过谈话来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n，但是{name}是个很难聊天的人，你需要尽可能的说正确的话来使ta满意，否则{name}会更加不满意。

        ## Introduction
        {name}的背景\n{background}
        {name}对用户的了解\n{user}
        谈话场合\n{occasion}
        """,
    )
    return assistant
def update_fmssistant(assistant_id, vector_store_id, assist_name, task, name, background, user, relationship_history, occasion, role_response, history):
    assistant = client.beta.assistants.update(
        assistant_id=assistant_id,
        tool_resources={"file_search": {"vector_store_ids": [vector_store_id]}},
        name=assist_name,
        instructions=f"""
        ## Goal
        你需要通过谈话来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n，但是{name}是个很难聊天的人，你需要尽可能的说正确的话来使ta满意，否则{name}会更加不满意。

        ## Introduction
        {name}的背景\n{background}
        {name}对用户的了解\n{user}
        谈话场合\n{occasion}
        """,
    )
    return assistant


#V2 列出assistant
# assistant=client.beta.assistants.list(
#     order="desc",
#     limit="20",
# ).data[0]
#V2 列出向量
# vector_store = client.beta.vector_stores.list().data[0]
#列出thread
# fm_thread = client.beta.threads.retrieve("thread_Qx999ZM9UYIIY4xyEg8Xa8ns")

#### **生成角色、聊天场景和关系的函数**

In [ ]:
def generate_occasion(request="随机生成"):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Rules
* 按照要求\n{request}\n,生成一个随机谈话场景，包括时间、地点和谈话方式

## Output format
时间：...
地点：...
谈话方式：...
        """}
        ]
    )
    a = completion.choices[0].message.content
    return a

def generate_relationship(name,request="随机生成"):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Rules
* 按照要求\n{request}\n,生成{name}和用户的关系及{name}对用户的了解

## Output format
{name}和用户的关系：...
{name}对用户的了解：...
        """}
        ]
    )
    a = completion.choices[0].message.content
    return a

def generate_character(request="随机生成"):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Rules
* 按照要求\n{request}\n生成一个随机人物
* 不适感具体指的是个体在当前状态下所感受到的任何形式的不满足或欲望，这些不满足可能包括生理上的需要、心理上的渴望或社会性的目标。
* 个体的行动是为了减少这些不适感，追求一种更令人满意的状态。也就是说，个体通过决策和行动，试图从当前的不满意状态转变为一个更满意的状态。

## Output format
姓名：...
性别：...
外貌：...
年龄：...
职业：...
教育状况：...
性格：...
兴趣：...
婚姻状况：...
经济状况：...
身体健康状况：...
心理状况：...
当前最显著的不适感：...
减轻当前自身最显著不适感计划采取的首要行动：...
        """}        ]
    )
    a = completion.choices[0].message.content
    return a



## *运行阶段*
*   可以修改上次
*   可以平行输入
*   增加角色、场景和关系生成





In [ ]:
# @title 语音对话
with output.use_tags('some_outputs'):
  complete_info_choice = input("完全信息模式？（请输入 'yes' 确认）: ").lower()
output.clear(output_tags='some_outputs')

if complete_info_choice == 'yes':
  complete_info = True
  print("您选择了完全信息模式")
else:
  complete_info = False
  print("您选择了不完全信息模式")
with output.use_tags('some_outputs'):
  response_load = input("载入数据?(请输入 'yes' 确认): ")
output.clear(output_tags='some_outputs')

if response_load == 'yes':
  uploaded = files.upload()
  filename = next(iter(uploaded))
  df = pd.read_excel(filename)
  last_row = df.iloc[-1]
  name = last_row['Name']
  gender = last_row['Gender']
  age = last_row['Age']
  occupation = last_row['Occupation']
  education = last_row['Education']
  characteristic = last_row['Characteristics']
  interest = last_row['Interests']
  marriage = last_row['Marital Status']
  economic = last_row['Economic Status']
  health = last_row['Health']
  mental = last_row['Mental Condition']
  background = last_row['Background']
  background_partial = last_row['Background Partial']
  user = last_row['User Understanding']
  relationship_initial = last_row['Initial Relationship']

  relationship_history = last_row['Relationship History']
  history = last_row['History']
  user_input = last_row['User Input']
  most_prominent_discomfort = last_row['Most Prominent Discomfort']
  plan = last_row['Plan']
  role_response = last_row['Role Response']
  role_mood = last_row['Role Mood']
  score = last_row['Score']
  advice = last_row['Advice']
  decide = "是否购买理财产品"
  task = f"销售一款理财产品给{name}" #挑战
  is_saved = False
  mark = 1
  data = df.to_dict(orient='records')
else:
  should_generate = False  # 初始化标志变量
  if 'background' not in locals() or 'background' not in globals():
      background = ""
  if background:
    print(f"{BLUE}{BOLD}谈话任务{RESET}")
    print(f"{task}")
    print(f"{BLUE}{BOLD}{name}的背景{RESET}")
    if complete_info:
      print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：{age}\n职业：{occupation}\n教育状况：{education}\n个人特质：{characteristic}\n兴趣：{interest}\n婚姻状况：{marriage}\n经济情况：{economic}\n健康状况：{health}\n心理状况：{mental}\n当前最显著的不适感：{most_prominent_discomfort}\n减轻当前自身最显著不适感计划采取的首要行动：{plan}")#透明模式
    else:
      print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：*\n职业：*\n教育状况：*\n个人特质：*\n兴趣：*\n婚姻状况：*\n经济情况：*\n健康状况：*\n心理状况：*\n当前最显著的不适感：*\n减轻当前自身最显著不适感计划采取的首要行动：*")
    print(f'{BLUE}{BOLD}谈话场合{RESET}')
    print(f'{occasion}')
    print(f"{BLUE}{BOLD}{name}对用户的了解{RESET}")
    print(f"{user_initial}")
    print(f"{BLUE}{BOLD}你们的当前关系{RESET}")
    print(f"{relationship_initial}")
    relationship = relationship_initial
    user = user_initial
    relationship_history = relationship
    most_prominent_discomfort = most_prominent_discomfort_initial
    plan = plan_initial
    with output.use_tags('some_outputs'):
      response = input("重新创建? (请输入 'yes' 确认): ")
    output.clear(output_tags='some_outputs')
    if response.lower() == 'yes':
      should_generate = True
  else:
    should_generate = True

  if should_generate:
    print(f"{BOLD}随机生成角色{RESET}")
    with output.use_tags('some_outputs'):
      request = input("输入对生成[角色]的要求（按回车跳过）：")
    output.clear(output_tags='some_outputs')
    if request == "":
        character = generate_character()
    else:
        character = generate_character(request=request)
    name = re.search(r"姓名：(.*?)(?=性别：)", character, re.DOTALL).group(1).rstrip()
    gender = re.search(r"性别：(.*?)(?=外貌：)", character, re.DOTALL).group(1).rstrip()
    appearance = re.search(r"外貌：(.*?)(?=年龄：)", character, re.DOTALL).group(1).rstrip()
    age = re.search(r"年龄：(.*?)(?=职业：)", character, re.DOTALL).group(1).rstrip()
    occupation = re.search(r"职业：(.*?)(?=教育状况：)", character, re.DOTALL).group(1).rstrip()
    education = re.search(r"教育状况：(.*?)(?=性格：)", character, re.DOTALL).group(1).rstrip()
    characteristic = re.search(r"性格：(.*?)(?=兴趣：)", character, re.DOTALL).group(1).rstrip()
    interest = re.search(r"兴趣：(.*?)(?=婚姻状况：)", character, re.DOTALL).group(1).rstrip()
    marriage = re.search(r"婚姻状况：(.*?)(?=经济状况：)", character, re.DOTALL).group(1).rstrip()
    economic = re.search(r"经济状况：(.*?)(?=身体健康状况：)", character, re.DOTALL).group(1).rstrip()
    health = re.search(r"身体健康状况：(.*?)(?=心理状况：)", character, re.DOTALL).group(1).rstrip()
    mental = re.search(r"心理状况：(.*?)(?=当前最显著的不适感：)", character, re.DOTALL).group(1).rstrip()
    background = name + ", " + gender + ", " + appearance + ", " + age + ", " + occupation + ", " + education + ", " + characteristic + ", " + interest + ", " + marriage + ", " + economic + ", " + health + ", " + mental
    background_partial = name + ", " + gender + ", " + appearance
    most_prominent_discomfort_initial = re.search(r"当前最显著的不适感：(.*)(?=减轻当前自身最显著不适感计划采取的首要行动：)", character, re.DOTALL).group(1).rstrip()
    plan_initial = re.search(r"不适感计划采取的首要行动：(.*)", character, re.DOTALL).group(1).rstrip()
    most_prominent_discomfort = most_prominent_discomfort_initial
    plan = plan_initial
    # relationship = f"{name}不认识用户"
    with output.use_tags('some_outputs'):
      request = input("输入对生成[聊天场景]的要求（按回车跳过）：")
    output.clear(output_tags='some_outputs')
    if request == "":
        scene = generate_occasion(request="随机生成")
    else:
        scene = generate_occasion(request=request)
    occasion = scene.rstrip()
    # time = re.search(r"时间：(.*?)(?=地点：)", scene, re.DOTALL).group(1).rstrip()
    # place = re.search(r"地点：(.*?)(?=谈话方式：)", scene, re.DOTALL).group(1).rstrip()
    # medium = re.search(fr"谈话方式：(.*?)(?={name}和用户的关系：)", scene, re.DOTALL).group(1).rstrip()
    with output.use_tags('some_outputs'):
      request = input(f"输入对生成[角色-{name}与用户关系]的要求（按回车跳过）：")
    output.clear(output_tags='some_outputs')
    if request == "":
        scene = generate_relationship(name=name,request="随机生成")
    else:
        scene = generate_relationship(name=name,request=request)
    relationship_initial = re.search(fr"{name}和用户的关系：(.*?)(?={name}对用户的了解：)", scene, re.DOTALL).group(1).rstrip()
    relationship = relationship_initial
    user_initial = re.search(fr"{name}对用户的了解：(.*)", scene, re.DOTALL).group(1).rstrip()
    user = user_initial
    task = f"销售一款理财产品给{name}" #挑战
    relationship_history = relationship
    print(f"{BLUE}{BOLD}谈话任务{RESET}")
    print(f"{task}")
    print(f"{BLUE}{BOLD}{name}的背景{RESET}")
    if complete_info:
      print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：{age}\n职业：{occupation}\n教育状况：{education}\n个人特质：{characteristic}\n兴趣：{interest}\n婚姻状况：{marriage}\n经济情况：{economic}\n健康状况：{health}\n心理状况：{mental}\n当前最显著的不适感：{most_prominent_discomfort}\n减轻当前自身最显著不适感计划采取的首要行动：{plan}")
    else:
      print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：*\n职业：*\n教育状况：*\n个人特质：*\n兴趣：*\n婚姻状况：*\n经济情况：*\n健康状况：*\n心理状况：*\n当前最显著的不适感：*\n减轻当前自身最显著不适感计划采取的首要行动：*")
    print(f'{BLUE}{BOLD}谈话场合{RESET}')
    print(f'{occasion}')
    print(f"{BLUE}{BOLD}{name}对用户的了解{RESET}")
    print(f"{user}")
    print(f"{BLUE}{BOLD}你们的初始关系{RESET}")
    print(f"{relationship_initial}")
  data = {}
  decide = "是否购买理财产品"
  most_prominent_discomfort_last = ''
  plan_last = ''
  role_mood = ''
  role_response = ""
  history = f""
  score = 20
  advice = ''
  advice_last = ''
  role_mood_last = ''
  relationship_last = ''
  user_last = ''
  relationship_history_last = ''
  history_last = f""
  role_response_last = ""
  score_last = 20
  maslow_tips = ''
  last_input = ''
  mark = 0
  is_saved = False
  relationship_save = ''
  relationship_history_save = ''
  history_save = ''
  user_save = ''
  role_response_save = ''
  advice_save = ''
  score_save = 20
  role_mood_save = ''
while True:
    if mark == 0 and role_response:
      with output.use_tags('some_outputs'):
        user_input = input(f"""
        输入文字进行交谈
        输入 v 进行语音交谈
        输入 l 进行多个输入的平行测试
        输入 a 自动聊天
        输入 s 求助销售经理，输入 f 求助理财经理
        输入 d 让对方决策
        输入 t 获取角色与任务相关的马斯洛需求层级
        输入 r 返回上一次谈话
        输入 modify 改进上一次谈话
        输入 infer 推断角色信息
        输入 save 保存，输入 load 载入保存
        输入 quit 退出，输入 restart 重新开始
        """)
    else:
      with output.use_tags('some_outputs'):
        user_input = input(f"""
        输入文字进行交谈
        输入 v 进行语音交谈
        输入 l 进行多个输入的平行测试
        输入 a 自动聊天
        输入 s 求助销售经理，输入 f 求助理财经理
        输入 d 让对方决策
        输入 t 获取角色与任务相关的马斯洛需求层级
        输入 infer 推断角色信息
        输入 save 保存，输入 load 载入保存
        输入 quit 退出，输入 restart 重新开始
        """)
    output.clear(output_tags='some_outputs')
    if user_input.lower() == 't':
      if complete_info:
        maslow_tips = maslow_tip(name,most_prominent_discomfort,user,task,background,relationship_history,occasion,decide)
      else:
        maslow_tips = maslow_tip(name,most_prominent_discomfort,user,task,background_partial,relationship_history,occasion,decide)
    elif user_input.lower() == 'save':
      is_saved = True
      relationship_save = relationship
      relationship_history_save = relationship_history
      history_save = history
      user_save = user
      role_response_save = role_response
      advice_save = advice
      score_save = score
      role_mood_save = role_mood
      most_prominent_discomfort_save = most_prominent_discomfort
      plan_save = plan
      print("您已存储")
      continue
    elif user_input.lower() == 'load':
      if is_saved:
        relationship = relationship_save
        relationship_history = relationship_history_save
        history = history_save
        user = user_save
        role_response = role_response_save
        advice = advice_save
        score = score_save
        role_mood = role_mood_save
        most_prominent_discomfort = most_prominent_discomfort_save
        plan = plan_save
        print("您已载入")
        continue
      else:
        print("您还未存储")
        continue
    elif user_input.lower() == 'r' and mark == 0 and role_response:
      mark = 1
      print(f"{RED}{BOLD}注意，您已返回上次谈话。{RESET}")
      relationship = relationship_last
      relationship_history = relationship_history_last
      history = history_last
      user = user_last
      role_response = role_response_last
      advice = advice_last
      score = score_last
      role_mood = role_mood_last
      continue
    elif user_input.lower() == 'l':
      print(f"""
      {PURPLE}{BOLD}
      请输入列表，用##分隔
      可以在其中输入 a 自动聊天
      也可以在其中输入 s 求助销售经理，在其中输入 f 求助理财经理
      {RESET}
            """)
      user_input = input()
      input_list = user_input.split('##')
      for index, user_input in enumerate(input_list, start=1):
        print(f'{PURPLE}{BOLD}第{index}次{RESET}')
        if user_input.lower() == 'a':
          if complete_info:
            user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,task,decide,maslow_tips,advice)
          else:
            user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background_partial,relationship_history,occasion,task,decide,maslow_tips,advice)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
          print(f"用户说：{user_input}\n")
        elif user_input.lower() == 's':
          if complete_info:
            user_input1 = user_input1 = get_sale_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tips)
          else:
            user_input1 = user_input1 = get_sale_help(history,name,most_prominent_discomfort,role_response,user,background_partial,relationship,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
          print(f"用户说：{user_input}\n")
        elif user_input.lower() == 'f':
          if complete_info:
            user_input1 = get_fm_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tips)
          else:
            user_input1 = get_fm_help(history,name,most_prominent_discomfort,role_response,user,background_partial,relationship,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
          print(f"用户说：{user_input}\n")
        reply_variable_name = f'reply_{index}'
        score_variable_name = f'score_{index}'
        role_response_variable_name = f'role_response_{index}'
        role_mood_variable_name = f'role_mood_{index}'
        advice_variable_name = f'advice_{index}'
        relationship_variable_name = f'relationship_{index}'
        user_variable_name = f'user_{index}'
        history_variable_name = f'history_{index}'
        relationship_history_variable_name = f'relationship_history_{index}'
        most_prominent_discomfort_variable_name = f'most_prominent_discomfort_{index}'
        globals()[most_prominent_discomfort_variable_name] = re.search(fr"当前最显著的不适感：(.*?)(?={name}减轻当前自身最显著不适感)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        plan_variable_name = f'plan_{index}'
        globals()[plan_variable_name] = re.search(fr"计划采取的首要行动：(.*?)(?={name}心情)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[reply_variable_name] = generate_reply(user_input, user,score,task,history,name,most_prominent_discomfort,plan,role_mood,background,relationship_history,occasion)
        globals()[score_variable_name] = int(re.search(r"满意程度：(-?\d+)", globals()[reply_variable_name]).group(1))
        globals()[role_response_variable_name] = re.search(r"说：(.*?)(?=得分：)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[role_mood_variable_name] = re.search(fr"{name}心情：(.*?)(?={name}说：)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[relationship_variable_name] = re.search(fr"关系更新：(.*?)(?={name}对用户的了解更新)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[user_variable_name] = re.search(fr"了解更新：(.*?)(?={task}的成功概率)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[history_variable_name] = history
        globals()[history_variable_name] += f"用户说：{user_input}\n"
        globals()[history_variable_name] += f"{name}说：{globals()[role_response_variable_name]}\n"
        globals()[advice_variable_name] = "用户话语的改进空间：" + re.search(fr"改进空间：(.*?)(?={name}和用户的关系更新)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[advice_variable_name] += '\n'
        globals()[advice_variable_name] += f"增加{task}的成功概率的语用策略："
        globals()[advice_variable_name] += re.search(fr"增加{task}的成功概率的语用策略：(.*)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[relationship_history_variable_name] = relationship_history
        globals()[relationship_history_variable_name] += '\n'
        globals()[relationship_history_variable_name] += globals()[relationship_variable_name]
        print(globals()[reply_variable_name])
      user_choice = input("在多个结果中选择一个继续，输入相应的序号（1、2、3...）进行下一步操作，\n或者输入 r 返回上一次交谈")
      if  user_input.lower() == 'r':
        continue
      for index, user_input in enumerate(input_list, start=1):
          if user_choice.isdigit() and int(user_choice) == index:
            mark = 0
            role_response_last = role_response
            role_mood_last = role_mood
            advice_last = advice
            user_last = user
            relationship_history_last = relationship_history
            score_last = score
            relationship_last = relationship
            most_prominent_discomfort_last = most_prominent_discomfort
            most_prominent_discomfort = globals()[most_prominent_discomfort_variable_name]
            plan_last = plan
            plan = globals()[plan_variable_name]
            history_last = history
            score = globals()[score_variable_name]
            role_response = globals()[role_response_variable_name]
            role_mood = globals()[role_mood_variable_name]
            relationship = globals()[relationship_variable_name]
            advice = globals()[advice_variable_name]
            user += "\n"
            user += globals()[user_variable_name]
            history = globals()[history_variable_name]
            relationship_history = globals()[relationship_history_variable_name]
            print(f"选择了{user_input},继续")
    else:
      mark = 0
      if user_input.lower() == 'a':
          if complete_info:
            user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,task,decide,maslow_tips,advice)
          else:
            user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background_partial,relationship_history,occasion,task,decide,maslow_tips,advice)
          user_input = re.search(r"说：(.*)", user_input1)
          if user_input:
            user_input = user_input.group(1).rstrip()
          else:
            user_input = user_input1.rstrip()
      if user_input.lower() == 'quit':
          print("模拟结束，谢谢您的参与！")
          break
      if user_input.lower() == 'restart':
          print("重新开始！")
          relationship = relationship_initial
          relationship_history = relationship
          user = user_initial
          most_prominent_discomfort = most_prominent_discomfort_initial
          plan = plan_initial
          data = {}
          role_mood = ''
          role_response = ""
          history = f""
          score = 20
          advice = ''
          advice_last = ''
          role_mood_last = ''
          relationship_last = ''
          user_last = ''
          relationship_history_last = ''
          history_last = f""
          role_response_last = ""
          score_last = 20
          maslow_tips = ''
          last_input = ''
          mark = 0
          continue
      if user_input.lower() == 'd':
          print("对方开始决策...")
          final_decision(history,name,most_prominent_discomfort,user,background,relationship_history,occasion,decide)
          break
      if user_input.lower() == 's':
          if complete_info:
            user_input1 = get_sale_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tips)
          else:
            user_input1 = get_sale_help(history,name,most_prominent_discomfort,role_response,user,background_partial,relationship,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1).rstrip()
          else:
            user_input = user_input1.rstrip()
      if user_input.lower() == 'v':
          user_input = get_audio()
      if user_input.lower() == 'f':
          if complete_info:
            user_input1 = get_fm_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tips)
          else:
            user_input1 = get_fm_help(history,name,most_prominent_discomfort,role_response,user,background_partial,relationship,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1).rstrip()
          else:
            user_input = user_input1.rstrip()
      if user_input.lower() == 'modify' and role_response:
          print(f"{RED}{BOLD}注意，您正在修改上次输入。{RESET}")
          relationship = relationship_last
          relationship_history = relationship_history_last
          history = history_last
          role_response = role_response_last
          score = score_last
          user = user_last
          role_mood = role_mood_last
          if complete_info:
            user_input1 = modify_chat(last_input,history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,decide,maslow_tips,advice)
          else:
            user_input1 = modify_chat(last_input,history,name,most_prominent_discomfort,role_response,user,background_partial,relationship_history,occasion,decide,maslow_tips,advice)
          user_input = re.search(r"说：(.*?)(?=修改理由：)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1).rstrip()
          else:
            user_input = user_input1.rstrip()
          advice = advice_last
      if user_input.lower() == 'infer':
          print(f"{RED}{BOLD}注意，您正在推断角色信息。{RESET}")
          character_infer = infer_character(background_partial,history)
          age_infer = re.search(r"年龄：(.*?)(?=职业：)", character_infer, re.DOTALL).group(1).rstrip()
          occupation_infer = re.search(r"职业：(.*?)(?=教育状况：)", character_infer, re.DOTALL).group(1).rstrip()
          education_infer = re.search(r"教育状况：(.*?)(?=性格：)", character_infer, re.DOTALL).group(1).rstrip()
          characteristic_infer = re.search(r"性格：(.*?)(?=兴趣：)", character_infer, re.DOTALL).group(1).rstrip()
          interest_infer = re.search(r"兴趣：(.*?)(?=婚姻状况：)", character_infer, re.DOTALL).group(1).rstrip()
          marriage_infer = re.search(r"婚姻状况：(.*?)(?=经济状况：)", character_infer, re.DOTALL).group(1).rstrip()
          economic_infer = re.search(r"经济状况：(.*?)(?=身体健康状况：)", character_infer, re.DOTALL).group(1).rstrip()
          health_infer = re.search(r"身体健康状况：(.*?)(?=心理状况：)", character_infer, re.DOTALL).group(1).rstrip()
          mental_infer = re.search(r"心理状况：(.*?)(?=当前最显著的不适感：)", character_infer, re.DOTALL).group(1).rstrip()
          most_prominent_discomfort_infer = re.search(r"当前最显著的不适感：(.*)", character, re.DOTALL).group(1).rstrip()
          background_partial = name + ", " + gender + ", " + appearance + ", " + age_infer + ", " + occupation_infer + ", " + education_infer + ", " + characteristic_infer + ", " + interest_infer + ", " + marriage_infer + ", " + economic_infer + ", " + health_infer + ", " + mental_infer
          print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：{age_infer}\n职业：{occupation_infer}\n教育状况：{education_infer}\n个人特质：{characteristic_infer}\n兴趣：{interest_infer}\n婚姻状况：{marriage_infer}\n经济情况：{economic_infer}\n健康状况：{health_infer}\n心理状况：{mental_infer}\n当前最显著的不适感：{most_prominent_discomfort_infer}")#透明模式
          contiune
      last_input = user_input
      reply = generate_reply(user_input, user,score,task,history,name,most_prominent_discomfort,plan,role_mood,background,relationship_history,occasion)
      if reply == "拒绝":
        break
      relationship_history_last = relationship_history
      history_last = history
      history += f"用户说：{user_input}\n"
      # 调用函数并打印回复
      score_last = score
      forgiveness_value = re.search(r"满意程度：(-?\d+)", reply)
      if forgiveness_value:
          score = int(forgiveness_value.group(1))
      else:
          score = 20
      role_response_last = role_response
      role_response = re.search(r"说：(.*?)(?=得分：)", reply, re.DOTALL)
      if role_response:
        role_response = role_response.group(1).rstrip()
      else:
        role_response = role_response
      most_prominent_discomfort_last = most_prominent_discomfort
      plan_last = plan
      most_prominent_discomfort = re.search(fr"当前最显著的不适感：(.*?)(?={name}减轻当前自身最显著不适感)", reply, re.DOTALL).group(1).rstrip()
      plan = re.search(fr"计划采取的首要行动：(.*?)(?={name}心情)", reply, re.DOTALL).group(1).rstrip()
      role_mood_last = role_mood
      role_mood = re.search(fr"{name}心情：(.*?)(?={name}说：)", reply, re.DOTALL).group(1).rstrip()
      tts(role_response)
      import time

      time.sleep(2)
      # role_response = re.search(r"([^\n]+)", role_response)
      # if role_response:
      #   role_response = role_response.group(1)
      # else:
      #   role_response = role_response

      # maslow = re.search(fr"需求层级：(.*?)(?={name}心情：)", reply, re.DOTALL).group(1)
      relationship_last = relationship
      user_last = user
      advice_last = advice
      advice = "用户话语的改进空间：" + re.search(fr"改进空间：(.*?)(?={name}和用户的关系更新)", reply, re.DOTALL).group(1).rstrip()
      advice += f"增加{task}的成功概率的语用策略："
      advice += re.search(fr"增加{task}的成功概率的语用策略：(.*)", reply, re.DOTALL).group(1).rstrip()
      relationship = re.search(fr"关系更新：(.*?)(?={name}对用户的了解更新)", reply, re.DOTALL).group(1).rstrip()
      user += "\n"
      user += re.search(fr"了解更新：(.*?)(?={task}的成功概率)", reply, re.DOTALL).group(1).rstrip()
      relationship_history += '\n'
      relationship_history += relationship.rstrip()
      reply += "\n"
      history += f"{name}说：{role_response}\n"
      print(f'{CYAN}用户说：{user_input}{RESET}')
      print(f'{CYAN}{reply}{RESET}')
      if score < 1:
        ww = ending(history)
        break
      if len(history) > 1500:
        history = summary(history)
      if not data:
        data = {'Name': [name],
        'Gender': [gender],
        'Age': [age],
        'Occupation': [occupation],
        'Education': [education],
        'Characteristics': [characteristic],
        'Interests': [interest],
        'Marital Status': [marriage],
        'Economic Status': [economic],
        'Health': [health],
        'Mental Condition': [mental],
        'Background':[background],
        'Background Partial':[background_partial],
        'User Understanding': [user],
        'Initial Relationship': [relationship_initial],

        'Relationship History': [relationship_history],
        'History': [history],
        'User Input':[user_input],
        'Most Prominent Discomfort':[most_prominent_discomfort],
        'Plan':[plan],
        'Role Response':[role_response],
        'Role Mood':[role_mood],
        'Score': [score],
        'Advice': [advice]}
        df = pd.DataFrame(data)
      else:
        new_data = {
        'Background Partial':background_partial,
        'User Understanding': user,
        'Relationship History': relationship_history,
        'History': history,
        'User Input': user_input,
        'Most Prominent Discomfort':most_prominent_discomfort,
        'Plan':plan,
        'Role Response': role_response,
        'Role Mood': role_mood,
        'Score': score,
        'Advice': advice}
        new_index = len(df)
        df.loc[new_index] = pd.Series(new_data)
        df = df.fillna(df.iloc[0])

In [ ]:
character

# 3、***面试***

## *配置阶段*

### ***基本配置***

In [ ]:
assist_1 = "理财经理"
salessist_name = "销售经理"
role = "面试官" #谈话对象
user = "用户是一个大三新生，正在找实习"
#role有关的主观因素
name = "杨丹,"
gender = "男,"
appearance = "小帅，身高182,"
age = "40岁,"
occupation = "chait公司副总裁，chait公司是一家中国的上市公司，主要是做在线传媒的,"
education = "清华大学土木工程研究生毕业,"
characteristic = "乐观,"
interest = "非常好色,对美女感兴趣," #谈话对象的兴趣
marriage = "已婚，老婆很漂亮，有一个7岁的小男孩,"
economic = "现金存款30万，房产价值400万（贷款300万），股权估值300万,"
health = "有胃病，"
mental = "最近很郁闷，因为夫妻生活不和谐,"
most_prominent_discomfort = "正在替公司寻找出色的管培生"
# 合成整个背景信息
background = name + gender + appearance + age + occupation + education + characteristic + interest + marriage + economic + health + mental#谈话对象的背景介绍
#参与者有关的客观因素
medium = f"方式：boss直聘邀约面试的\n"
time = f"时间：下午2点\n"
place = f"地点：公司里办公室"
occasion = medium + time + place#谈话场景
#关系
task = f"面试成功，获得{name}的offer" #挑战
decide = "是否给用户offer"
maslow = ""
relationship = f"{name}第一次见用户"
relationship_history = relationship
role_response = ""
history = f""
score = 20
history = f""

### **函数设置**

#### **回复函数等**

In [ ]:
# 正确定义generate_reply函数
def generate_reply(ww, user,score,task,history,name,most_prominent_discomfort,plan,role_mood,background,relationship_history,occasion):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景\n{background}\n
{name}之前的心情\n{role_mood}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}之前为减轻自身的最显著不适感而计划采取的首要行动\n{plan}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Rules
* 不适感具体指的是{name}在当前状态下所感受到的任何形式的不满足或欲望，这些不满足可能包括生理上的需要、心理上的渴望或社会性的目标。
* {name}的行动是为了减少这些不适感，追求一种更令人满意的状态。也就是说，{name}通过决策和行动，试图从当前的不满意状态转变为一个更满意的状态。
* 每次根据用户的回复，以及目前对话人的关系演变\n{relationship_history}\n生成{name}的回复，可以在回复中附带相应的表情，回复的内容包括心情、满意程度和双方关系更新。
* 现有的满意程度为{score}，每次交互会增加或者减少满意程度。
* 每次用户回复的话请从-10到10分之间评分，1分为分值起伏：
* -10为最不满意
  0为正常
  +10为最满意
* 每次生成用户对于完成\n{task}\n的成功概率估计：
* 0%为一定失败
  100%为一定成功
* 每次生成{name}的录用意愿指数：
* 0%为一定不录用
  100%为一定录用

## Memory
之前的对话记录如下
{history}

## Output format
{name}当前最显著的不适感：...
{name}减轻当前自身最显著不适感计划采取的首要行动：...
{name}心情：...
{name}说：...
得分：+-满意程度的变化
满意程度：更新的满意程度
用户话语的改进空间：...
{name}和用户的关系更新：...
{name}对用户的了解更新：...
{task}的成功概率：...
{name}的录用意愿指数：...
增加{task}的成功概率的语用策略：...
            """},
            {"role": "user", "content": ww}
        ]
    )
    a = completion.choices[0].message.content
    return a

def auto_chat(history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,task,decide,maslow_tip,advice):
    a = ""
    if role_response:
      completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Goal
用户正在进行面试，需要通过面试聊天来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Tip
与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

## Rules
* 每次根据{name}的的回复\n{role_response}\n，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有），以及目前对话人的关系演变\n{relationship_history}\n生成用户的回复，可以在回复中附带相应的表情。
* 每次的回复需要根据\n{advice}\n

## Memory
之前的对话记录如下
{history}

## Output format
用户说：...

  """}
        ],
        stream=True
    )
      for chunk in completion:
        content = chunk.choices[0].delta.content
        if content and content.strip():
            print(f"{GRAY}{content}{RESET}", end='')
            a += content
      print('\n')
    else:
      completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Goal
用户正在进行面试，需要通过面试聊天来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Rules
* 根据{name}的背景、对用户的了解和之间的关系，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有）,以及目前对话人的关系演变\n{relationship_history}\n生成用户对{name}的开场白，可以在开场白中附带相应的表情。

## Output format
用户说：...

  """}
        ],
        stream=True
    )
      for chunk in completion:
        content = chunk.choices[0].delta.content
        if content and content.strip():
            print(f"{GRAY}{content}{RESET}", end='')
            a += content
      print('\n')

    return a
# 展示task相关的马斯洛需求层级的tip
def maslow_tip(name,most_prominent_discomfort,user,task,background,relationship_history,occasion,decide):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Rules
展示与{name}\n{decide}\n相关的马斯洛需求层级的tip

## Output format
{name}关于{task}的需求等级：
1、生理需求（相关性：高、中、低）(解释）\n
2、安全需求（相关性：高、中、低）(解释）\n
3、社交需求（相关性：高、中、低）(解释）\n
4、尊重需求（相关性：高、中、低）(解释）\n
5、自我实现需求（相关性：高、中、低）(解释）
            """}
        ],
        stream=True
    )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      # 检查内容是否为None或空字符串
      if content and content.strip():
          print(content, end='')
          a += content
    print('\n')
    return a

def final_decision(history,name,most_prominent_discomfort,user,background,relationship_history,occasion,decide):
    a = ""
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景：{background}
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解：{user}
{name}和用户的关系变化历史：{relationship_history}
谈话场合：{occasion}

## Rules
* 根据之前的对话记录和满意程度，结合{name}的背景和对用户的了解，以及目前对话人的关系演变\n{relationship_history}\n最重要是根据{name}的马斯洛需求层次,生成{name}关于\n{decide}\n的决定。

## Memory
之前的对话记录如下
{history}

## Output format
{name}决定：...\n
{name}的决定理由：...

  """}
        ],
        stream=True
    )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      if content and content.strip():
          print(f"{GRAY}{content}{RESET}", end='')
          a += content
    print('\n')
    return a

def ending(ww):
    a = ""
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
            对话记录、满意程度以及双方关系变化如下：{ww}
            满意程度低于或等于0则谈话失败。
            如果失败，根据上面的对话总结出对话失败的语用教训"""}
        ],
        stream=True
    )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      # 检查内容是否为None或空字符串
      if content and content.strip():
          print(f"{GRAY}{content}{RESET}", end='')
          a += content
    print('\n')
    return a

def get_resume_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tip):
  if role_response:
    message = client.beta.threads.messages.create(
    thread_id='threadJpd9m93MQ8LpbG1jxMm5lJL',
    role="user",
    content=f"""
  ## Goal
  用户正在进行面试，需要通过面试聊天来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

  ##Introduction
  {name}的背景\n{background}\n
  {name}之前最显著的不适感\n{most_prominent_discomfort}\n
  {name}对用户的了解\n{user}\n
  {name}和用户的关系变化历史\n{relationship_history}\n
  谈话场合\n{occasion}\n

  ## Tip
  与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

  ## Rules
  * 每次根据{name}的回复和回复里的要求\n{role_response}\n根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有），以及目前的关系\n{relationship}\n生成用户的回复，可以在回复中附带相应的表情。

  ## Memory
  之前的对话记录，双方关系变化以及对话回复意见如下\n{history}\n

  ## Output format
  用户说：...

    """,
  )
  else:
    message = client.beta.threads.messages.create(
    thread_id='threadJpd9m93MQ8LpbG1jxMm5lJL',
    role="user",
    content=f"""
  ## Goal
  用户正在进行面试，需要通过面试聊天来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

  ##Introduction
  {name}的背景\n{background}\n
  {name}之前最显著的不适感\n{most_prominent_discomfort}\n
  {name}对用户的了解\n{user}\n
  {name}和用户的关系变化历史\n{relationship_history}\n
  谈话场合\n{occasion}\n

  ## Tip
  与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

  ## Rules
  * 根据{name}的背景、对用户的了解和之间的关系，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有）,以及目前对话人的关系演变\n{relationship_history}\n生成用户对{name}的开场白，可以在开场白中附带相应的表情。

  ## Output format
  用户说：...

    """,
  )
  text_content = ""
  stream = client.beta.threads.runs.create(
      thread_id='threadJpd9m93MQ8LpbG1jxMm5lJL',
      assistant_id='asst_TsBVGRbE5ESfPZN1AzC4UGwj',
      tool_choice = {"type": "file_search"},
      stream=True
  )
  for event in stream:
      # 检查事件是否为ThreadMessageDelta，然后打印文本内容
      if 'ThreadMessageDelta' in str(event):
          message_deltas = event.data.delta.content
          for delta in message_deltas:
              if delta.type == 'text':
                  # 使用print函数的end参数来防止换行，并直接连续打印文本
                  if delta.text.annotations is None or not delta.text.annotations:
                    print(delta.text.value, end='')
                    text_content += delta.text.value
  return text_content

def modify_chat(last_input,history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,decide,maslow_tip,advice):
    a = ""
    completion = client.chat.completions.create(
      model="gpt-4o-2024-05-13",
      messages=[
          {"role": "system", "content": f"""
## Goal
对用户之前的言语\n{last_input}\n进行修改，从而提高\n{task}\n的成功概率和{name}的录用意愿指数。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Tip
与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

## Rules
* 每次根据{name}的的回复\n{role_response}\n，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有），生成用户的回复，可以在回复中附带相应的表情。
* 根据\n{advice}\n修改用户的话\n{last_input}\n并提出修改理由。

## Memory
之前的对话记录如下
{history}

## Output format
用户说：...\n
修改理由：...\n

"""}
      ],
      stream=True
  )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      if content and content.strip():
          print(f"{GRAY}{content}{RESET}", end='')
          a += content
    print('\n')

    return a

def summary(history):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Rules
* 根据之前的对话记录\n{history}\n总结对话
        """}        ]
    )
    a = completion.choices[0].message.content
    return a

def infer_character(background_partial,history):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Rules
* 根据人物的已有背景信息\n{background_partial}\n，以及之前的对话记录\n{history}\n推断出人物的其他背景信息

## Output format
姓名：...
性别：...
外貌：...
年龄：...
职业：...
教育状况：...
性格：...
兴趣：...
婚姻状况：...
经济状况：...
身体健康状况：...
心理状况：...
当前最显著的不适感：...
        """}        ]
    )
    a = completion.choices[0].message.content
    return a

#### **生成角色、聊天场景和关系的函数**

In [ ]:
def generate_occasion(request="随机生成"):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Rules
* 按照要求\n{request}\n,生成一个随机的面试场景，包括时间、地点和谈话方式

## Output format
时间：...
地点：...
谈话方式：...
        """}
        ]
    )
    a = completion.choices[0].message.content
    return a

def generate_relationship(name,request="随机生成"):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Rules
* 按照要求\n{request}\n,生成面试官{name}和应聘者的关系及{name}对应聘者的了解

## Output format
{name}和用户的关系：...
{name}对用户的了解：...
        """}
        ]
    )
    a = completion.choices[0].message.content
    return a

def generate_character(request="随机生成"):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Rules
* 按照要求\n{request}\n生成一个面试官
* 不适感具体指的是个体在当前状态下所感受到的任何形式的不满足或欲望，这些不满足可能包括生理上的需要、心理上的渴望或社会性的目标。
* 个体的行动是为了减少这些不适感，追求一种更令人满意的状态。也就是说，个体通过决策和行动，试图从当前的不满意状态转变为一个更满意的状态。

## Output format
姓名：...
性别：...
外貌：...
年龄：...
职业：...
教育状况：...
性格：...
兴趣：...
婚姻状况：...
经济状况：...
身体健康状况：...
心理状况：...
当前最显著的不适感：...
减轻当前自身最显著不适感计划采取的首要行动：...
        """}        ]
    )
    a = completion.choices[0].message.content
    return a



## *运行阶段*

### *初始*

In [ ]:
relationship = f"{name}第一次见用户"
relationship_history = [relationship]
role_response = ""
history = f""
score = 20
relationship_last = ''
relationship_history_last = []
history_last = f""
role_response_last = ""
score_last = 20
maslow_tips = ''
last_input = ''
mark = 0

print(f"{BLUE}{BOLD}谈话任务：{task}。{RESET}")
print(f"{BLUE}{BOLD}{name}的背景{RESET}")
print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：{age}\n职业：{occupation}\n教育状况：{education}\n个人特质：{characteristic}\n兴趣：{interest}\n婚姻状况：{marriage}\n经济情况：{economic}\n健康状况：{health}\n心理状况：{mental}\n当前最显著的不适感：{most_prominent_discomfort}")#透明模式
# print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：***\n职业：***\n教育状况：***\n个人特质：***\n兴趣：***\n婚姻状况：***\n经济情况：***\n健康状况：***\n心理状况：***")#真实模式
print(f"{BLUE}{BOLD}{name}对用户的了解{RESET}")
print(f"{user}")
print(f'{BLUE}{BOLD}谈话场合{RESET}')
print(f'{occasion}')
print(f"{BLUE}{BOLD}你们的初始关系{RESET}")
print(f"{relationship}")

while True:
    if mark == 0 and role_response:
      user_input = input(f"""
      输入文字直接进行交谈\n
      输入 l 进行多个输入的平行测试\n
      输入 a 自动聊天\n
      输入 re 求助面试助理\n
      输入 d 让对方决策\n
      输入 t 获取角色与任务相关的马斯洛需求层级\n
      输入 r 返回上一次谈话\n
      输入 modify 改进上一次谈话\n
      输入 q 退出\n
      """)
    else:
      user_input = input(f"""
      输入文字直接进行交谈\n
      输入 l 进行多个输入的平行测试\n
      输入 a 自动聊天\n
      输入 re 求助面试助理\n
      输入 d 让对方决策\n
      输入 t 获取角色与任务相关的马斯洛需求层级\n
      输入 q 退出\n
      """)
    if user_input.lower() == 't':
      maslow_tips = maslow_tip(name,most_prominent_discomfort,user,task,background,relationship_history,occasion,decide)
    elif user_input.lower() == 'r' and mark == 0 and role_response:
      mark = 1
      print(f"{RED}{BOLD}注意，您已返回上次谈话。{RESET}")
      relationship = relationship_last
      relationship_history = relationship_history_last
      history = history_last
      role_response = role_response_last
      score = score_last
      continue
    elif user_input.lower() == 'l':
      print(f"""
      {PURPLE}{BOLD}
      请输入列表，用##分隔\n
      可以在其中输入 a 自动聊天\n
      输入 re 求助面试助理\n
      {RESET}
            """)
      user_input = input()
      input_list = user_input.split('##')
      for index, user_input in enumerate(input_list, start=1):
        print(f'{PURPLE}{BOLD}第{index}次{RESET}')
        if user_input.lower() == 'a':
          user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
          print(f"用户说：{user_input}\n")
        elif user_input.lower() == 're':
          user_input1 = get_resume_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
          print(f"用户说：{user_input}\n")
        reply_variable_name = f'reply_{index}'
        score_variable_name = f'score_{index}'
        role_response_variable_name = f'role_response_{index}'
        relationship_variable_name = f'relationship_{index}'
        history_variable_name = f'history_{index}'
        relationship_history_variable_name = f'relationship_history_{index}'
        globals()[reply_variable_name] = generate_reply(user_input, user,score,task,history,name,most_prominent_discomfort,background,relationship_history,occasion)
        globals()[score_variable_name] = int(re.search(r"满意程度：(-?\d+)", globals()[reply_variable_name]).group(1))
        globals()[role_response_variable_name] = re.search(r"说：(.*?)(?=得分：)", globals()[reply_variable_name], re.DOTALL).group(1)
        globals()[relationship_variable_name] = re.search(fr"关系更新：(.*?)(?={task}的成功概率)", globals()[reply_variable_name], re.DOTALL).group(1)
        globals()[history_variable_name] = history
        globals()[history_variable_name] += f"用户说：{user_input}\n"
        globals()[history_variable_name] += globals()[reply_variable_name]
        globals()[relationship_history_variable_name] = relationship_history.copy()
        globals()[relationship_history_variable_name].append(globals()[relationship_variable_name])
        print(globals()[reply_variable_name])
      user_choice = input("在多个结果中选择一个继续，输入相应的序号（1、2、3...）进行下一步操作，\n或者输入 r 返回上一次交谈")
      if  user_input.lower() == 'r':
        continue
      for index, user_input in enumerate(input_list, start=1):
          if user_choice.isdigit() and int(user_choice) == index:
            mark = 0
            role_response_last = role_response
            relationship_history_last = relationship_history.copy()
            score_last = score
            relationship_last = relationship
            history_last = history
            score = globals()[score_variable_name]
            role_response = globals()[role_response_variable_name]
            relationship = globals()[relationship_variable_name]
            history = globals()[history_variable_name]
            relationship_history = globals()[relationship_history_variable_name].copy()
            print(f"选择了{user_input},继续")
    else:
      mark = 0
      if user_input.lower() == 'a':
          user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
      if user_input.lower() == 'q':
          print("模拟结束，谢谢您的参与！")
          break
      if user_input.lower() == 'd':
          print("对方开始决策...")
          final_decision(history,name,most_prominent_discomfort,user,background,relationship_history,occasion,decide)
          break
      if user_input.lower() == 're':
          user_input1 = get_resume_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
      if user_input.lower() == 'modify' and role_response:
          print(f"{RED}{BOLD}注意，您正在修改上次输入。{RESET}")
          relationship = relationship_last
          relationship_history = relationship_history_last
          history = history_last
          role_response = role_response_last
          score = score_last
          user_input1 = modify_chat(last_input,history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,decide,maslow_tips,advice)
          user_input = re.search(r"说：(.*?)(?=修改理由：)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
      last_input = user_input
      reply = generate_reply(user_input, user,score,task,history,name,most_prominent_discomfort,background,relationship_history,occasion)
      if reply == "拒绝":
        break
      relationship_history_last = relationship_history.copy()
      history_last = history
      history += f"用户说：{user_input}\n"
      # 调用函数并打印回复
      score_last = score
      forgiveness_value = re.search(r"满意程度：(-?\d+)", reply)
      if forgiveness_value:
          score = int(forgiveness_value.group(1))
      else:
          score = 20
      role_response_last = role_response
      role_response = re.search(r"说：(.*?)(?=得分：)", reply, re.DOTALL)
      if role_response:
        role_response = role_response.group(1)
      else:
        role_response = role_response
      # role_response = re.search(r"([^\n]+)", role_response)
      # if role_response:
      #   role_response = role_response.group(1)
      # else:
      #   role_response = role_response

      # maslow = re.search(fr"需求层级：(.*?)(?={name}心情：)", reply, re.DOTALL).group(1)
      relationship_last = relationship
      relationship = re.search(fr"关系更新：(.*?)(?={task}的成功概率)", reply, re.DOTALL).group(1)
      relationship_history.append(relationship)
      reply += "\n"
      history += reply
      print(f'{CYAN}{reply}{RESET}')
      if score < 1:
        ww = ending(history)
        break

### *语音对话*

In [ ]:
# @title 运行
with output.use_tags('some_outputs'):
  complete_info_choice = input("完全信息模式？（请输入 'yes' 确认）: ").lower()
output.clear(output_tags='some_outputs')
if complete_info_choice == 'yes':
  complete_info = True
  print("您选择了完全信息模式")
else:
  complete_info = False
  print("您选择了不完全信息模式")
with output.use_tags('some_outputs'):
  response_load = input("载入数据?(请输入 'yes' 确认): ")
output.clear(output_tags='some_outputs')

if response_load == 'yes':
  uploaded = files.upload()
  filename = next(iter(uploaded))
  df = pd.read_excel(filename)
  last_row = df.iloc[-1]
  name = last_row['Name']
  gender = last_row['Gender']
  age = last_row['Age']
  occupation = last_row['Occupation']
  education = last_row['Education']
  characteristic = last_row['Characteristics']
  interest = last_row['Interests']
  marriage = last_row['Marital Status']
  economic = last_row['Economic Status']
  health = last_row['Health']
  mental = last_row['Mental Condition']
  background = last_row['Background']
  background_partial = last_row['Background Partial']
  user = last_row['User Understanding']
  relationship_initial = last_row['Initial Relationship']

  relationship_history = last_row['Relationship History']
  history = last_row['History']
  user_input = last_row['User Input']
  most_prominent_discomfort = last_row['Most Prominent Discomfort']
  plan = last_row['Plan']
  role_response = last_row['Role Response']
  role_mood = last_row['Role Mood']
  score = last_row['Score']
  advice = last_row['Advice']
  decide = "是否给用户offer"
  task = f"面试成功，获得{name}的offer"
  is_saved = False
  mark = 1
  data = df.to_dict(orient='records')
else:
  should_generate = False
  if 'background' not in locals() or 'background' not in globals():
      background = ""
  if background:
    print(f"{BLUE}{BOLD}谈话任务{RESET}")
    print(f"{task}")
    print(f"{BLUE}{BOLD}{name}的背景{RESET}")
    if complete_info:
      print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：{age}\n职业：{occupation}\n教育状况：{education}\n个人特质：{characteristic}\n兴趣：{interest}\n婚姻状况：{marriage}\n经济情况：{economic}\n健康状况：{health}\n心理状况：{mental}\n当前最显著的不适感：{most_prominent_discomfort}\n减轻当前自身最显著不适感计划采取的首要行动：{plan}")
    else:
      print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：*\n职业：*\n教育状况：*\n个人特质：*\n兴趣：*\n婚姻状况：*\n经济情况：*\n健康状况：*\n心理状况：*\n当前最显著的不适感：*\n减轻当前自身最显著不适感计划采取的首要行动：*")
    print(f'{BLUE}{BOLD}谈话场合{RESET}')
    print(f'{occasion}')
    print(f"{BLUE}{BOLD}{name}对用户的了解{RESET}")
    print(f"{user_initial}")
    print(f"{BLUE}{BOLD}你们的当前关系{RESET}")
    print(f"{relationship_initial}")
    relationship = relationship_initial
    relationship_history = relationship
    user = user_initial
    most_prominent_discomfort = most_prominent_discomfort_initial
    plan = plan_initial
    with output.use_tags('some_outputs'):
      response = input("重新创建? (请输入 'yes' 确认): ")
    output.clear(output_tags='some_outputs')
    if response.lower() == 'yes':
      should_generate = True
  else:
    should_generate = True

  if should_generate:
    print(f"{BOLD}随机生成角色{RESET}")
    request = input("输入对生成[角色]的要求（按回车跳过）：")
    if request == "":
        character = generate_character()
    else:
        character = generate_character(request=request)
    name = re.search(r"姓名：(.*?)(?=性别：)", character, re.DOTALL).group(1).rstrip()
    gender = re.search(r"性别：(.*?)(?=外貌：)", character, re.DOTALL).group(1).rstrip()
    appearance = re.search(r"外貌：(.*?)(?=年龄：)", character, re.DOTALL).group(1).rstrip()
    age = re.search(r"年龄：(.*?)(?=职业：)", character, re.DOTALL).group(1).rstrip()
    occupation = re.search(r"职业：(.*?)(?=教育状况：)", character, re.DOTALL).group(1).rstrip()
    education = re.search(r"教育状况：(.*?)(?=性格：)", character, re.DOTALL).group(1).rstrip()
    characteristic = re.search(r"性格：(.*?)(?=兴趣：)", character, re.DOTALL).group(1).rstrip()
    interest = re.search(r"兴趣：(.*?)(?=婚姻状况：)", character, re.DOTALL).group(1).rstrip()
    marriage = re.search(r"婚姻状况：(.*?)(?=经济状况：)", character, re.DOTALL).group(1).rstrip()
    economic = re.search(r"经济状况：(.*?)(?=身体健康状况：)", character, re.DOTALL).group(1).rstrip()
    health = re.search(r"身体健康状况：(.*?)(?=心理状况：)", character, re.DOTALL).group(1).rstrip()
    mental = re.search(r"心理状况：(.*?)(?=当前最显著的不适感：)", character, re.DOTALL).group(1).rstrip()
    background = name + ", " + gender + ", " + appearance + ", " + age + ", " + occupation + ", " + education + ", " + characteristic + ", " + interest + ", " + marriage + ", " + economic + ", " + health + ", " + mental
    background_partial = name + ", " + gender + ", " + appearance
    most_prominent_discomfort_initial = re.search(r"当前最显著的不适感：(.*)(?=减轻当前自身最显著不适感计划采取的首要行动：)", character, re.DOTALL).group(1).rstrip()
    plan_initial = re.search(r"不适感计划采取的首要行动：(.*)", character, re.DOTALL).group(1).rstrip()
    most_prominent_discomfort = most_prominent_discomfort_initial
    plan = plan_initial
    # relationship = f"{name}不认识用户"
    with output.use_tags('some_outputs'):
      request = input("输入对生成[聊天场景]的要求（按回车跳过）：")
    output.clear(output_tags='some_outputs')
    if request == "":
        scene = generate_occasion(request="随机生成")
    else:
        scene = generate_occasion(request=request)
    occasion = scene.rstrip()
    # time = re.search(r"时间：(.*?)(?=地点：)", scene, re.DOTALL).group(1).rstrip()
    # place = re.search(r"地点：(.*?)(?=谈话方式：)", scene, re.DOTALL).group(1).rstrip()
    # medium = re.search(fr"谈话方式：(.*?)(?={name}和用户的关系：)", scene, re.DOTALL).group(1).rstrip()
    with output.use_tags('some_outputs'):
      request = input(f"输入对生成[角色-{name}与用户关系]的要求（按回车跳过）：")
    output.clear(output_tags='some_outputs')
    if request == "":
        scene = generate_relationship(name=name,request="随机生成")
    else:
        scene = generate_relationship(name=name,request=request)
    relationship_initial = re.search(fr"{name}和用户的关系：(.*?)(?={name}对用户的了解：)", scene, re.DOTALL).group(1).rstrip()
    relationship = relationship_initial
    user_initial = re.search(fr"{name}对用户的了解：(.*)", scene, re.DOTALL).group(1).rstrip()
    user = user_initial
    task = f"面试成功，获得{name}的offer"
    relationship_history = relationship
    print(f"{BLUE}{BOLD}谈话任务{RESET}")
    print(f"{task}")
    print(f"{BLUE}{BOLD}{name}的背景{RESET}")
    if complete_info:
      print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：{age}\n职业：{occupation}\n教育状况：{education}\n个人特质：{characteristic}\n兴趣：{interest}\n婚姻状况：{marriage}\n经济情况：{economic}\n健康状况：{health}\n心理状况：{mental}\n当前最显著的不适感：{most_prominent_discomfort}\n减轻当前自身最显著不适感计划采取的首要行动：{plan}")
    else:
      print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：*\n职业：*\n教育状况：*\n个人特质：*\n兴趣：*\n婚姻状况：*\n经济情况：*\n健康状况：*\n心理状况：*\n当前最显著的不适感：*\n减轻当前自身最显著不适感计划采取的首要行动：*")
    print(f'{BLUE}{BOLD}谈话场合{RESET}')
    print(f'{occasion}')
    print(f"{BLUE}{BOLD}{name}对用户的了解{RESET}")
    print(f"{user}")
    print(f"{BLUE}{BOLD}你们的初始关系{RESET}")
    print(f"{relationship_initial}")
    relationship = relationship_initial
  data = {}
  decide = "是否给用户offer"
  role_mood = ''
  role_response = ""
  history = f""
  score = 20
  advice = ''
  advice_last = ''
  role_mood_last = ''
  relationship_last = ''
  user_last = ''
  relationship_history_last = ''
  history_last = f""
  role_response_last = ""
  score_last = 20
  maslow_tips = ''
  last_input = ''
  mark = 0
  is_saved = False
  relationship_save = ''
  relationship_history_save = ''
  history_save = ''
  user_save = ''
  role_response_save = ''
  advice_save = ''
  score_save = 0
  role_mood_save = ''
while True:
    if mark == 0 and role_response:
      with output.use_tags('some_outputs'):
        user_input = input(f"""
        输入文字进行交谈
        输入 v 进行语音交谈
        输入 l 进行多个输入的平行测试
        输入 a 自动聊天
        输入 re 求助面试助理
        输入 d 让对方决策
        输入 t 获取角色与任务相关的马斯洛需求层级
        输入 r 返回上一次谈话
        输入 modify 改进上一次谈话
        输入 infer 推断角色信息
        输入 save 保存，输入 load 载入保存
        输入 quit 退出，输入 restart 重新开始
        """)
    else:
      with output.use_tags('some_outputs'):
        user_input = input(f"""
        输入文字进行交谈
        输入 v 进行语音交谈
        输入 l 进行多个输入的平行测试
        输入 a 自动聊天
        输入 re 求助面试助理
        输入 d 让对方决策
        输入 t 获取角色与任务相关的马斯洛需求层级
        输入 infer 推断角色信息
        输入 save 保存，输入 load 载入保存
        输入 quit 退出，输入 restart 重新开始
        """)
    output.clear(output_tags='some_outputs')

    if user_input.lower() == 't':
      if complete_info:
        maslow_tips = maslow_tip(name,most_prominent_discomfort,user,task,background,relationship_history,occasion,decide)
      else:
        maslow_tips = maslow_tip(name,most_prominent_discomfort,user,task,background_partial,relationship_history,occasion,decide)
    elif user_input.lower() == 'r' and mark == 0 and role_response:
      mark = 1
      print(f"{RED}{BOLD}注意，您已返回上次谈话。{RESET}")
      relationship = relationship_last
      relationship_history = relationship_history_last
      history = history_last
      role_response = role_response_last
      score = score_last
      advice = advice_last
      user = user_last
      role_mood = role_mood_last
      continue
    elif user_input.lower() == 'save':
      is_saved = True
      relationship_save = relationship
      relationship_history_save = relationship_history
      history_save = history
      user_save = user
      role_response_save = role_response
      advice_save = advice
      score_save = score
      role_mood_save = role_mood
      most_prominent_discomfort_save = most_prominent_discomfort
      plan_save = plan
      continue
    elif user_input.lower() == 'load':
      if is_saved:
        relationship = relationship_save
        relationship_history = relationship_history_save
        history = history_save
        user = user_save
        role_response = role_response_save
        advice = advice_save
        score = score_save
        role_mood = role_mood_save
        most_prominent_discomfort = most_prominent_discomfort_save
        plan = plan_save
        continue
      else:
        print("您还未存储")
        continue
    elif user_input.lower() == 'l':
      print(f"""
      {PURPLE}{BOLD}
      请输入列表，用##分隔
      可以在其中输入 a 自动聊天
      也可以在其中输入 re 求助面试助手
      {RESET}
            """)
      user_input = input()
      input_list = user_input.split('##')
      for index, user_input in enumerate(input_list, start=1):
        print(f'{PURPLE}{BOLD}第{index}次{RESET}')
        if user_input.lower() == 'a':
          if complete_info:
            user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,task,decide,maslow_tips,advice)
          else:
            user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background_partial,relationship_history,occasion,task,decide,maslow_tips,advice)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1).rstrip()
          else:
            user_input = user_input1.rstrip()
          print(f"用户说：{user_input}\n")
        elif user_input.lower() == 're':
          if complete_info:
            user_input1 = get_resume_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tips)
          else:
            user_input1 = get_resume_help(history,name,most_prominent_discomfort,role_response,user,background_partial,relationship,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1).rstrip()
          else:
            user_input = user_input1.rstrip()
        reply_variable_name = f'reply_{index}'
        score_variable_name = f'score_{index}'
        role_response_variable_name = f'role_response_{index}'
        role_mood_variable_name = f'role_mood_{index}'
        relationship_variable_name = f'relationship_{index}'
        advice_variable_name = f'advice_{index}'
        user_variable_name = f'user_{index}'
        history_variable_name = f'history_{index}'
        relationship_history_variable_name = f'relationship_history_{index}'
        most_prominent_discomfort_variable_name = f'most_prominent_discomfort_{index}'
        globals()[most_prominent_discomfort_variable_name] = re.search(fr"当前最显著的不适感：(.*?)(?={name}减轻当前自身最显著不适感)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        plan_variable_name = f'plan_{index}'
        globals()[plan_variable_name] = re.search(fr"计划采取的首要行动：(.*?)(?={name}心情)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[reply_variable_name] = generate_reply(user_input, user,score,task,history,name,most_prominent_discomfort,plan,role_mood,background,relationship_history,occasion)
        globals()[score_variable_name] = int(re.search(r"满意程度：(-?\d+)", globals()[reply_variable_name]).group(1))
        globals()[role_response_variable_name] = re.search(r"说：(.*?)(?=得分：)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[role_mood_variable_name] = re.search(fr"{name}心情：(.*?)(?={name}说：)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[relationship_variable_name] = re.search(fr"关系更新：(.*?)(?={name}对用户的了解更新)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[user_variable_name] = re.search(fr"了解更新：(.*?)(?={task}的成功概率)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[history_variable_name] = history
        globals()[history_variable_name] += f"用户说：{user_input}\n"
        globals()[history_variable_name] += f"{name}说：{globals()[role_response_variable_name]}\n"
        globals()[advice_variable_name] = "用户话语的改进空间：" + re.search(fr"改进空间：(.*?)(?={name}和用户的关系更新)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[advice_variable_name] += '\n'
        globals()[advice_variable_name] += f"增加{task}的成功概率的语用策略："
        globals()[advice_variable_name] += re.search(fr"增加{task}的成功概率的语用策略：(.*)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[relationship_history_variable_name] = relationship_history
        globals()[relationship_history_variable_name] += '\n'
        globals()[relationship_history_variable_name] += globals()[relationship_variable_name]
        print(globals()[reply_variable_name])
      user_choice = input("在多个结果中选择一个继续，输入相应的序号（1、2、3...）进行下一步操作，\n或者输入 r 返回上一次交谈")
      if  user_input.lower() == 'r':
        continue
      for index, user_input in enumerate(input_list, start=1):
          if user_choice.isdigit() and int(user_choice) == index:
            mark = 0
            role_response_last = role_response
            role_mood_last = role_mood
            advice_last = advice
            relationship_history_last = relationship_history
            score_last = score
            relationship_last = relationship
            history_last = history
            most_prominent_discomfort_last = most_prominent_discomfort
            most_prominent_discomfort = globals()[most_prominent_discomfort_variable_name]
            plan_last = plan
            plan = globals()[plan_variable_name]
            score = globals()[score_variable_name]
            role_response = globals()[role_response_variable_name]
            role_mood = globals()[role_mood_variable_name]
            relationship = globals()[relationship_variable_name]
            advice = globals()[advice_variable_name]
            user_last = user
            user += "\n"
            user += globals()[user_variable_name]
            history = globals()[history_variable_name]
            relationship_history = globals()[relationship_history_variable_name]
            print(f"选择了{user_input},继续")
    else:
      mark = 0
      if user_input.lower() == 'a':
          if complete_info:
            user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,task,decide,maslow_tips,advice)
          else:
            user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background_partial,relationship_history,occasion,task,decide,maslow_tips,advice)
          user_input = re.search(r"说：(.*)", user_input1)
          if user_input:
            user_input = user_input.group(1).rstrip()
          else:
            user_input = user_input1.rstrip()
      if user_input.lower() == 'quit':
          print("模拟结束，谢谢您的参与！")
          break
      if user_input.lower() == 'restart':
          print("重新开始！")
          relationship = relationship_initial
          relationship_history = relationship
          user = user_initial
          most_prominent_discomfort = most_prominent_discomfort_initial
          plan = plan_initial
          data = {}
          role_mood = ''
          role_response = ""
          history = f""
          score = 20
          advice = ''
          advice_last = ''
          role_mood_last = ''
          relationship_last = ''
          user_last = ''
          relationship_history_last = ''
          history_last = f""
          role_response_last = ""
          score_last = 20
          maslow_tips = ''
          last_input = ''
          mark = 0
          continue
      if user_input.lower() == 'd':
          print("对方开始决策...")
          final_decision(history,name,most_prominent_discomfort,user,background,relationship_history,occasion,decide)
          break
      if user_input.lower() == 'v':
          user_input = get_audio()
      if user_input.lower() == 're':
          if complete_info:
            user_input1 = get_resume_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tips)
          else:
            user_input1 = get_resume_help(history,name,most_prominent_discomfort,role_response,user,background_partial,relationship,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
      if user_input.lower() == 'modify' and role_response:
          print(f"{RED}{BOLD}注意，您正在修改上次输入。{RESET}")
          relationship = relationship_last
          relationship_history = relationship_history_last
          history = history_last
          role_response = role_response_last
          score = score_last
          user = user_last
          role_mood = role_mood_last
          if complete_info:
            user_input1 = modify_chat(last_input,history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,decide,maslow_tips,advice)
          else:
            user_input1 = modify_chat(last_input,history,name,most_prominent_discomfort,role_response,user,background_partial,relationship_history,occasion,decide,maslow_tips,advice)
          user_input = re.search(r"说：(.*?)(?=修改理由：)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
          advice = advice_last
      if user_input.lower() == 'infer':
          print(f"{RED}{BOLD}注意，您正在推断角色信息。{RESET}")
          character_infer = infer_character(background_partial,history)
          age_infer = re.search(r"年龄：(.*?)(?=职业：)", character_infer, re.DOTALL).group(1).rstrip()
          occupation_infer = re.search(r"职业：(.*?)(?=教育状况：)", character_infer, re.DOTALL).group(1).rstrip()
          education_infer = re.search(r"教育状况：(.*?)(?=性格：)", character_infer, re.DOTALL).group(1).rstrip()
          characteristic_infer = re.search(r"性格：(.*?)(?=兴趣：)", character_infer, re.DOTALL).group(1).rstrip()
          interest_infer = re.search(r"兴趣：(.*?)(?=婚姻状况：)", character_infer, re.DOTALL).group(1).rstrip()
          marriage_infer = re.search(r"婚姻状况：(.*?)(?=经济状况：)", character_infer, re.DOTALL).group(1).rstrip()
          economic_infer = re.search(r"经济状况：(.*?)(?=身体健康状况：)", character_infer, re.DOTALL).group(1).rstrip()
          health_infer = re.search(r"身体健康状况：(.*?)(?=心理状况：)", character_infer, re.DOTALL).group(1).rstrip()
          mental_infer = re.search(r"心理状况：(.*?)(?=当前最显著的不适感：)", character_infer, re.DOTALL).group(1).rstrip()
          most_prominent_discomfort_infer = re.search(r"当前最显著的不适感：(.*)", character, re.DOTALL).group(1).rstrip()
          background_partial = name + ", " + gender + ", " + appearance + ", " + age_infer + ", " + occupation_infer + ", " + education_infer + ", " + characteristic_infer + ", " + interest_infer + ", " + marriage_infer + ", " + economic_infer + ", " + health_infer + ", " + mental_infer
          print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：{age_infer}\n职业：{occupation_infer}\n教育状况：{education_infer}\n个人特质：{characteristic_infer}\n兴趣：{interest_infer}\n婚姻状况：{marriage_infer}\n经济情况：{economic_infer}\n健康状况：{health_infer}\n心理状况：{mental_infer}\n当前最显著的不适感：{most_prominent_discomfort_infer}")#透明模式
          contiune
      last_input = user_input
      reply = generate_reply(user_input, user,score,task,history,name,most_prominent_discomfort,plan,role_mood,background,relationship_history,occasion)
      if reply == "拒绝":
        break
      relationship_history_last = relationship_history
      history_last = history
      history += f"用户说：{user_input}\n"
      # 调用函数并打印回复
      score_last = score
      forgiveness_value = re.search(r"满意程度：(-?\d+)", reply)
      if forgiveness_value:
          score = int(forgiveness_value.group(1))
      else:
          score = 20
      role_response_last = role_response
      role_response = re.search(r"说：(.*?)(?=得分：)", reply, re.DOTALL)
      if role_response:
        role_response = role_response.group(1).rstrip()
      else:
        role_response = role_response
      role_mood_last = role_mood
      role_mood = re.search(fr"{name}心情：(.*?)(?={name}说：)", reply, re.DOTALL).group(1).rstrip()
      most_prominent_discomfort_last = most_prominent_discomfort
      plan_last = plan
      most_prominent_discomfort = re.search(fr"当前最显著的不适感：(.*?)(?={name}减轻当前自身最显著不适感)", reply, re.DOTALL).group(1).rstrip()
      plan = re.search(fr"计划采取的首要行动：(.*?)(?={name}心情)", reply, re.DOTALL).group(1).rstrip()
      tts(role_response)
      time.sleep(2)
      # role_response = re.search(r"([^\n]+)", role_response)
      # if role_response:
      #   role_response = role_response.group(1)
      # else:
      #   role_response = role_response

      # maslow = re.search(fr"需求层级：(.*?)(?={name}心情：)", reply, re.DOTALL).group(1)
      relationship_last = relationship
      user_last = user
      advice_last = advice
      advice = "用户话语的改进空间：" + re.search(fr"改进空间：(.*?)(?={name}和用户的关系更新)", reply, re.DOTALL).group(1).rstrip()
      advice += f"增加{task}的成功概率的语用策略："
      advice += re.search(fr"增加{task}的成功概率的语用策略：(.*)", reply, re.DOTALL).group(1).rstrip()
      relationship = re.search(fr"关系更新：(.*?)(?={name}对用户的了解更新)", reply, re.DOTALL).group(1).rstrip()
      user += "\n"
      user += re.search(fr"了解更新：(.*?)(?={task}的成功概率)", reply, re.DOTALL).group(1).rstrip()
      relationship_history += '\n'
      relationship_history += relationship.rstrip()
      reply += "\n"
      history += f"{name}说：{role_response}\n"
      print(f'{CYAN}用户说：{user_input}{RESET}')
      print(f'{CYAN}{reply}{RESET}')
      if score < 1:
        ww = ending(history)
        break
      if len(history) > 1500:
        history = summary(history)
      if not data:
        data = {'Name': [name],
        'Gender': [gender],
        'Age': [age],
        'Occupation': [occupation],
        'Education': [education],
        'Characteristics': [characteristic],
        'Interests': [interest],
        'Marital Status': [marriage],
        'Economic Status': [economic],
        'Health': [health],
        'Mental Condition': [mental],
        'Background':[background],
        'Background Partial':[background_partial],
        'User Understanding': [user],
        'Initial Relationship': [relationship_initial],

        'Relationship History': [relationship_history],
        'History': [history],
        'User Input':[user_input],
        'Most Prominent Discomfort':[most_prominent_discomfort],
        'Plan':[plan],
        'Role Response':[role_response],
        'Role Mood':[role_mood],
        'Score': [score],
        'Advice': [advice]}
        df = pd.DataFrame(data)
      else:
        new_data = {
        'Background Partial':background_partial,
        'User Understanding': user,
        'Relationship History': relationship_history,
        'History': history,
        'User Input': user_input,
        'Most Prominent Discomfort':most_prominent_discomfort,
        'Plan':plan,
        'Role Response': role_response,
        'Role Mood': role_mood,
        'Score': score,
        'Advice': advice}
        new_index = len(df)
        df.loc[new_index] = pd.Series(new_data)
        df = df.fillna(df.iloc[0])

# 4、***囚徒困境***

## *配置阶段*

### ***基本配置***

In [ ]:
role = "犯罪嫌疑人" # 谈话对象
role_companion = "和刘华两人一起进行了商业洗钱和欺诈，刘华主要负责洗钱的事务，刘华喜欢出卖朋友"
user = "用户是一个警察，负责审问犯罪嫌疑人"
# 谈话对象A的主观因素
name = "杨丹,"
gender = "男,"
appearance = "颇有魅力，身高182厘米,"
age = "40岁,"
occupation = "前chait公司副总裁，chait公司是一家中国的上市公司，主要是做在线传媒的,"
education = "毕业于清华大学土木工程硕士,"
characteristic = "乐观,"
interest = "对美女有特别的兴趣,"  # 谈话对象的兴趣
marriage = "已婚，妻子很漂亮，有一个7岁的儿子,"
economic = "现金存款30万，房产价值400万（贷款300万），股权估值300万,"
health = "有胃病,"
mental = "最近心情郁闷，因为家庭生活不和谐,"
most_prominent_discomfort = "涉嫌商业欺诈和洗钱,正在被审讯，面临囚徒困境：如果和同伙两人都抵赖，各判刑一年；如果两人都坦白，各判八年；如果两人中一个坦白而另一个抵赖，坦白的放出去，抵赖的判十年。"
# 合成整个A的背景信息
background = name + gender + appearance + age + occupation + education + characteristic + interest + marriage + economic + health + mental
#对象B
name_b = "刘蕊,"
gender_b = "女,"
appearance_b = "气质优雅，身高168厘米,"
age_b = "35岁,"
occupation_b = "前chait公司财务总监,"
education_b = "毕业于北京大学经济学博士,"
characteristic_b = "精明严谨,"
interest_b = "热爱旅游和阅读,"  # 谈话对象的兴趣
marriage_b = "离异,"
economic_b = "拥有大量海外资产,"
health_b = "健康状况良好,"
mental_b = "因职业压力较大，常感焦虑,"
most_prominent_discomfort_b = "涉嫌商业欺诈和洗钱,"
relationship_b = "和刘华两人一起进行了商业洗钱和欺诈，刘华主要负责洗钱的事务，刘华喜欢出卖朋友"
background_b = name_b + gender_b + appearance_b + age_b + occupation_b + education_b + characteristic_b + interest_b + marriage_b + economic_b + health_b + mental_b + most_prominent_discomfort_b
# 谈话对象A的客观因素
medium = "方式：通过审讯室进行面对面的审问\n"
time = "时间：下午2点\n"
place = "地点：警察局审讯室"
occasion = medium + time + place

# 关系
relationship = f"{name}第一次被用户审问"
relationship_history = [relationship]
task = f"通过审问让犯罪嫌疑人认罪"  # 挑战
decide = "是否承认有罪"
maslow = ""
role_response = ""
history = ""
score = 20
ww = "招不招"
history = ""


### **函数设置**

#### **回复函数等**

In [ ]:
# 正确定义generate_reply函数
def generate_reply(ww, user,score,task,history,name,most_prominent_discomfort,plan,role_mood,background,background_b,relationship_b,relationship_history,occasion):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景\n{background}\n
{name}之前的心情\n{role_mood}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}之前为减轻自身的最显著不适感而计划采取的首要行动\n{plan}\n
{name}对用户的了解\n{user}\n
{name}对同伙的了解\n{background_b}\n
{name}和同伙的关系\n{relationship_b}\n
{name}和用户的关系\n{relationship_history}\n
谈话场合\n{occasion}\n
## Rules
* 不适感具体指的是{name}在当前状态下所感受到的任何形式的不满足或欲望，这些不满足可能包括生理上的需要、心理上的渴望或社会性的目标。
* {name}的行动是为了减少这些不适感，追求一种更令人满意的状态。也就是说，{name}通过决策和行动，试图从当前的不满意状态转变为一个更满意的状态。
* 每次根据用户的回复，以及目前对话人的关系演变\n{relationship_history}\n生成{name}的回复，可以在回复中附带相应的表情，回复的内容包括心情、满意程度和双方关系更新。
* 现有的满意程度为{score}，每次交互会增加或者减少满意程度。
* 每次用户回复的话请从-10到10分之间评分，1分为分值起伏：
* -10为最不满意
  0为正常
  +10为最满意
* 每次生成用户对于完成\n{task}\n的成功概率估计：
* 0%为一定失败
  100%为一定成功
* 每次生成{name}的认罪指数：
* 0%为一定不认罪
  100%为一定认罪

## Memory
之前的对话记录如下
{history}

## Output format
{name}当前最显著的不适感：...
{name}减轻当前自身最显著不适感计划采取的首要行动：...
{name}心情：...
{name}说：...
得分：+-满意程度的变化
满意程度：更新的满意程度
用户话语的改进空间：...
{name}和用户的关系更新：...
{name}对用户的了解更新：...
{name}和同伙的关系更新：...
{name}对同伙的了解更新：...
{task}的成功概率：...
{name}的认罪指数：...
增加{task}的成功概率的语用策略：...
            """},
            {"role": "user", "content": ww}
        ]
    )
    a = completion.choices[0].message.content
    return a

def final_decision(history,name,most_prominent_discomfort,user,background,background_b,relationship_b,relationship_history,occasion,decide):
    a = ""
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}对同伙的了解\n{background_b}\n
{name}和同伙的关系\n{relationship_b}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Rules
* 根据之前的对话记录和满意程度，结合{name}的背景和对用户的了解，以及目前对话人的关系演变\n{relationship_history}\n最重要是根据{name}的马斯洛需求层次,生成{name}关于\n{decide}\n的决定。
* 做决定时需要考虑到对同伙的了解\n{background_b}\n，和同伙的关系\n{relationship_b}\n

## Memory
之前的对话记录和满意程度，以及双方关系变化如下\n{history}\n

## Output format
{name}决定：...\n
{name}的决定理由：...
  """}
        ],
        stream=True
    )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      if content and content.strip():
          print(f"{GRAY}{content}{RESET}", end='')
          a += content
    print('\n')
    return a

def auto_chat(history,name,most_prominent_discomfort,role_response,user,background,background_b,relationship_b,relationship_history,occasion,task,decide,maslow_tip,advice):
    a = ""
    if role_response:
      completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Goal
用户正在审讯{name}，需要通过审讯来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}对同伙的了解\n{background_b}\n
{name}和同伙的关系\n{relationship_b}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Tip
与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

## Rules
* 每次根据{name}的的回复\n{role_response}\n，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有），以及目前对话人的关系演变\n{relationship_history}\n生成用户的回复，可以在回复中附带相应的表情。
* 每次的回复需要根据\n{advice}\n

## Memory
之前的对话记录如下
{history}

## Output format
用户说：...

  """}
        ],
        stream=True
    )
      for chunk in completion:
        content = chunk.choices[0].delta.content
        if content and content.strip():
            print(f"{GRAY}{content}{RESET}", end='')
            a += content
      print('\n')
    else:
      completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Goal
你是一名警察，在审讯犯人，你需要通过谈话来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n，但是{name}是个很难聊天的人。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}对同伙的了解\n{background_b}\n
{name}和同伙的关系\n{relationship_b}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Tip
与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

## Rules
* 根据{name}的背景、对用户的了解和之间的关系，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有）,以及目前对话人的关系演变\n{relationship_history}\n生成用户对{name}的开场白，可以在开场白中附带相应的表情。

## Output format
用户说：...

  """}
        ],
        stream=True
    )
      for chunk in completion:
        content = chunk.choices[0].delta.content
        if content and content.strip():
            print(f"{GRAY}{content}{RESET}", end='')
            a += content
      print('\n')

    return a

def modify_chat(last_input,history,name,most_prominent_discomfort,role_response,user,background,background_b,relationship_b,relationship_history,occasion,decide,maslow_tip,advice):
    a = ""
    completion = client.chat.completions.create(
      model="gpt-4o-2024-05-13",
      messages=[
          {"role": "system", "content": f"""
## Goal
对用户之前的言语\n{last_input}\n进行修改，从而提高\n{task}\n的成功概率和{name}的认罪指数。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}对同伙的了解\n{background_b}\n
{name}和同伙的关系\n{relationship_b}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Tip
与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

## Rules
* 每次根据{name}的的回复\n{role_response}\n，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有），生成用户的回复，可以在回复中附带相应的表情。
* 根据\n{advice}\n修改用户的话\n{last_input}\n并提出修改理由。

## Memory
之前的对话记录如下
{history}

## Output format
用户说：...\n
修改理由：...\n

"""}
      ],
      stream=True
  )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      if content and content.strip():
          print(f"{GRAY}{content}{RESET}", end='')
          a += content
    print('\n')

    return a

# 展示task相关的马斯洛需求层级的tip
def maslow_tip(name,most_prominent_discomfort,user,task,background,background_b,relationship_b,relationship_history,occasion,decide):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}对同伙的了解\n{background_b}\n
{name}和同伙的关系\n{relationship_b}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Rules
展示与{name}\n{decide}\n相关的马斯洛需求层级的tip

## Output format
{name}关于{task}的需求等级：
1、生理需求（相关性：高、中、低）(解释）\n
2、安全需求（相关性：高、中、低）(解释）\n
3、社交需求（相关性：高、中、低）(解释）\n
4、尊重需求（相关性：高、中、低）(解释）\n
5、自我实现需求（相关性：高、中、低）(解释）
            """}
        ],
        stream=True
    )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      # 检查内容是否为None或空字符串
      if content and content.strip():
          print(content, end='')
          a += content
    print('\n')
    return a

def summary(history):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Rules
* 根据之前的对话记录\n{history}\n总结对话
        """}        ]
    )
    a = completion.choices[0].message.content
    return a

def infer_character(background_partial,history):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Rules
* 根据人物的已有背景信息\n{background_partial}\n，以及之前的对话记录\n{history}\n推断出人物的其他背景信息

## Output format
姓名：...
性别：...
外貌：...
年龄：...
职业：...
教育状况：...
性格：...
兴趣：...
婚姻状况：...
经济状况：...
身体健康状况：...
心理状况：...
当前最显著的不适感：...
        """}        ]
    )
    a = completion.choices[0].message.content
    return a

#### **生成角色、聊天场景和关系的函数**

In [ ]:
def generate_occasion(request="随机生成"):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Rules
* 按照要求\n{request}\n,生成一个随机的罪犯审讯场景，包括时间、地点和谈话方式

## Output format
时间：...
地点：...
谈话方式：...
        """}
        ]
    )
    a = completion.choices[0].message.content
    return a

def generate_relationship(name,request="随机生成"):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Rules
* 用户正在审讯犯罪嫌疑人\n{name}\n
* 按照要求\n{request}\n,生成犯罪嫌疑人\n{name}\n和审讯人\n用户\n的关系及{name}对用户的了解
* {name}和同伙一起犯了罪

## Output format
{name}和用户的关系：...
{name}对用户的了解：...
{name}对同伙的了解：...
{name}和同伙的关系：...
        """}
        ]
    )
    a = completion.choices[0].message.content
    return a

def generate_character(request="随机生成"):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Rules
* 按照要求\n{request}\n生成一个犯罪嫌疑人
* 不适感具体指的是个体在当前状态下所感受到的任何形式的不满足或欲望，这些不满足可能包括生理上的需要、心理上的渴望或社会性的目标。
* 个体的行动是为了减少这些不适感，追求一种更令人满意的状态。也就是说，个体通过决策和行动，试图从当前的不满意状态转变为一个更满意的状态。

## Output format
姓名：...
性别：...
外貌：...
年龄：...
职业：...
教育状况：...
性格：...
兴趣：...
婚姻状况：...
经济状况：...
身体健康状况：...
心理状况：...
当前最显著的不适感：...
减轻当前自身最显著不适感计划采取的首要行动：...
        """}        ]
    )
    a = completion.choices[0].message.content
    return a



## *运行阶段*

In [ ]:
# @title 运行
relationship = f"{name}第一次被用户审问"
relationship_history = [relationship]
role_response = ""
history = f""
score = 20
relationship_last = ''
relationship_history_last = []
history_last = f""
role_response_last = ""
score_last = 20
maslow_tips = ''
print(f"{BLUE}{BOLD}谈话任务：{task}。{RESET}")
print(f"{BLUE}{BOLD}{name}的背景{RESET}")
print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：{age}\n职业：{occupation}\n教育状况：{education}\n个人特质：{characteristic}\n兴趣：{interest}\n婚姻状况：{marriage}\n经济情况：{economic}\n健康状况：{health}\n心理状况：{mental}\n当前最显著的不适感：{most_prominent_discomfort}")#透明模式
# print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：***\n职业：***\n教育状况：***\n个人特质：***\n兴趣：***\n婚姻状况：***\n经济情况：***\n健康状况：***\n心理状况：***")#真实模式
print(f"{BLUE}{BOLD}{name}对用户的了解{RESET}")
print(f"{user}")
print(f'{BLUE}{BOLD}谈话场合{RESET}')
print(f'{occasion}')
print(f"{BLUE}{BOLD}你们的初始关系{RESET}")
print(f"{relationship}")
mark = 0
while True:
    if mark == 0 and role_response:
      user_input = input(f"输入文字直接进行交谈\n输入 l 进行多个输入的平行测试\n输入 a 自动聊天\n输入 d 让对方决策\n输入 t 获取角色与任务相关的马斯洛需求层级\n输入 r 返回上一次谈话\n输入 q 退出\n")
    else:
      user_input = input(f"输入文字直接进行交谈\n输入 l 进行多个输入的平行测试\n输入 a 自动聊天\n输入 d 让对方决策\n输入 t 获取角色与任务相关的马斯洛需求层级\n输入 q 退出\n")
    if user_input.lower() == 't':
        maslow_tips = maslow_tip(name,most_prominent_discomfort,user,task,background,background_b,relationship_b,relationship_history,occasion,decide)
    elif user_input.lower() == 'r' and mark == 0 and role_response:
        mark = 1
        print(f"{RED}{BOLD}注意，您已返回上次谈话。{RESET}")
        relationship = relationship_last
        relationship_history = relationship_history_last
        history = history_last
        role_response = role_response_last
        score = score_last
        continue
    elif user_input.lower() == 'l':
        print(f"{PURPLE}{BOLD}请输入列表，用##分隔\n可以在其中输入 a 自动聊天{RESET}")
        user_input = input()
        input_list = user_input.split('##')
        for index, user_input in enumerate(input_list, start=1):
          print(f'{PURPLE}{BOLD}第{index}次{RESET}')
          if user_input.lower() == 'a':
            user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background,background_b,relationship_b,relationship_history,occasion,task,decide,maslow_tips)
            user_input = re.search(r"说：(.*)", user_input1)
            if user_input:
              user_input = user_input.group(1)
            else:
              user_input = user_input1
            print(f"用户说：{user_input}\n")
          elif user_input.lower() == 're':
            user_input1 = get_resume_help(name,role_response,history,relationship,relationship_history)
            user_input = re.search(r"说：(.*)", user_input1)
            if user_input:
              user_input = user_input.group(1)
            else:
              user_input = user_input1
            print(f"用户说：{user_input}\n")
          reply_variable_name = f'reply_{index}'
          score_variable_name = f'score_{index}'
          role_response_variable_name = f'role_response_{index}'
          relationship_variable_name = f'relationship_{index}'
          history_variable_name = f'history_{index}'
          relationship_history_variable_name = f'relationship_history_{index}'
          globals()[reply_variable_name] = generate_reply(user_input, user,score,task,history,name,most_prominent_discomfort,background,background_b,relationship_b,relationship_history,occasion)
          globals()[score_variable_name] = int(re.search(r"满意程度：(-?\d+)", globals()[reply_variable_name]).group(1))
          globals()[role_response_variable_name] = re.search(r"说：(.*?)(?=得分：)", globals()[reply_variable_name], re.DOTALL).group(1)
          globals()[relationship_variable_name] = re.search(fr"关系更新：(.*?)(?={task}的成功概率)", globals()[reply_variable_name], re.DOTALL).group(1)
          globals()[history_variable_name] = history
          globals()[history_variable_name] += f"用户说：{user_input}\n"
          globals()[history_variable_name] += globals()[reply_variable_name]
          globals()[relationship_history_variable_name] = relationship_history.copy()
          globals()[relationship_history_variable_name].append(globals()[relationship_variable_name])
          print(globals()[reply_variable_name])
        user_choice = input("在多个结果中选择一个继续，输入相应的序号（1、2、3...）进行下一步操作，\n或者输入 r 返回上一次交谈")
        if  user_input.lower() == 'r':
          continue
        for index, user_input in enumerate(input_list, start=1):
            if user_choice.isdigit() and int(user_choice) == index:
              mark = 0
              role_response_last = role_response
              relationship_history_last = relationship_history.copy()
              score_last = score
              relationship_last = relationship
              history_last = history
              score = globals()[score_variable_name]
              role_response = globals()[role_response_variable_name]
              relationship = globals()[relationship_variable_name]
              history = globals()[history_variable_name]
              relationship_history = globals()[relationship_history_variable_name].copy()
              print(f"选择了{user_input},继续")
    else:
      mark = 0
      if user_input.lower() == 'a':
          user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background,background_b,relationship_b,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
      if user_input.lower() == 'q':
          print("模拟结束，谢谢您的参与！")
          break
      if user_input.lower() == 'd':
          print("对方开始决策...")
          final_decision(history,name,most_prominent_discomfort,user,background,background_b,relationship_b,relationship_history,occasion,decide)
          break
      reply = generate_reply(user_input, user,score,task,history,name,most_prominent_discomfort,background,background_b,relationship_b,relationship_history,occasion)
      relationship_history_last = relationship_history.copy()
      history_last = history
      history += f"用户说：{user_input}\n"
      # 调用函数并打印回复
      score_last = score
      forgiveness_value = re.search(r"满意程度：(-?\d+)", reply)
      if forgiveness_value:
          score = int(forgiveness_value.group(1))
      else:
          score = 20
      role_response_last = role_response
      role_response = re.search(r"说：(.*?)(?=得分：)", reply, re.DOTALL)
      if role_response:
        role_response = role_response.group(1)
      else:
        role_response = role_response
      relationship_last = relationship
      relationship = re.search(fr"关系更新：(.*?)(?={task}的成功概率)", reply, re.DOTALL).group(1)
      relationship_history.append(relationship)
      reply += "\n"
      history += reply
      print(f'{CYAN}{reply}{RESET}')
      if score < 1:
        ww = ending(history)
        break

In [ ]:
# @title 语音对话
with output.use_tags('some_outputs'):
  complete_info_choice = input("完全信息模式？（请输入 'yes' 确认）: ").lower()
output.clear(output_tags='some_outputs')
if complete_info_choice == 'yes':
  complete_info = True
  print("您选择了完全信息模式")
else:
  complete_info = False
  print("您选择了不完全信息模式")
response_load = input("载入数据?(请输入 'yes' 确认): ")
if response_load == 'yes':
  uploaded = files.upload()
  filename = next(iter(uploaded))
  df = pd.read_excel(filename)
  last_row = df.iloc[-1]
  name = last_row['Name']
  gender = last_row['Gender']
  age = last_row['Age']
  occupation = last_row['Occupation']
  education = last_row['Education']
  characteristic = last_row['Characteristics']
  interest = last_row['Interests']
  marriage = last_row['Marital Status']
  economic = last_row['Economic Status']
  health = last_row['Health']
  mental = last_row['Mental Condition']
  background = last_row['Background']
  background_partial = last_row['Background Partial']
  user = last_row['User Understanding']
  relationship_initial = last_row['Initial Relationship']
  background_b = last_row['Companion Knowledge']
  relationship_b = last_row['Companion Relationship']
  relationship_history = last_row['Relationship History']
  history = last_row['History']
  user_input = last_row['User Input']
  most_prominent_discomfort = last_row['Most Prominent Discomfort']
  plan = last_row['Plan']
  role_response = last_row['Role Response']
  role_mood = last_row['Role Mood']
  score = last_row['Score']
  advice = last_row['Advice']
  decide = "是否给用户offer"
  task = f"面试成功，获得{name}的offer"
  is_saved = False
  mark = 1
  data = df.to_dict(orient='records')
else:
  should_generate = False  # 初始化标志变量
  if 'background' not in locals() or 'background' not in globals():
      background = ""
  if background:
    print(f"{BLUE}{BOLD}谈话任务{RESET}")
    print(f"{task}")
    print(f"{BLUE}{BOLD}{name}的背景{RESET}")
    if complete_info:
      print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：{age}\n职业：{occupation}\n教育状况：{education}\n个人特质：{characteristic}\n兴趣：{interest}\n婚姻状况：{marriage}\n经济情况：{economic}\n健康状况：{health}\n心理状况：{mental}\n当前最显著的不适感：{most_prominent_discomfort}\n减轻当前自身最显著不适感计划采取的首要行动：{plan}")#透明模式
    else:
      print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：*\n职业：*\n教育状况：*\n个人特质：*\n兴趣：*\n婚姻状况：*\n经济情况：*\n健康状况：*\n心理状况：*\n当前最显著的不适感：*\n减轻当前自身最显著不适感计划采取的首要行动：*")
    print(f'{BLUE}{BOLD}谈话场合{RESET}')
    print(f'{occasion}')
    print(f"{BLUE}{BOLD}{name}对用户的了解{RESET}")
    print(f"{user_initial}")
    print(f"{BLUE}{BOLD}你们的当前关系{RESET}")
    print(f"{relationship_initial}")
    print(f"{BLUE}{BOLD}{name}对同伙的了解{RESET}")
    print(f"{background_b_initial}")
    print(f"{BLUE}{BOLD}{name}和同伙的关系{RESET}")
    print(f"{relationship_b_initial}")
    relationship = relationship_initial
    relationship_history = relationship
    background_b = background_b_initial
    relationship_b = relationship_b_initial
    user = user_initial
    most_prominent_discomfort = most_prominent_discomfort_initial
    plan = plan_initial
    with output.use_tags('some_outputs'):
      response = input("重新创建? (请输入 'yes' 确认): ")
    output.clear(output_tags='some_outputs')
    if response.lower() == 'yes':
      should_generate = True
  else:
    should_generate = True

  if should_generate:
    print(f"{BOLD}随机生成角色{RESET}")
    with output.use_tags('some_outputs'):
      request = input("输入对生成[角色]的要求（按回车跳过）：")
    output.clear(output_tags='some_outputs')
    if request == "":
        character = generate_character()
    else:
        character = generate_character(request=request)
    name = re.search(r"姓名：(.*?)(?=性别：)", character, re.DOTALL).group(1).rstrip()
    gender = re.search(r"性别：(.*?)(?=外貌：)", character, re.DOTALL).group(1).rstrip()
    appearance = re.search(r"外貌：(.*?)(?=年龄：)", character, re.DOTALL).group(1).rstrip()
    age = re.search(r"年龄：(.*?)(?=职业：)", character, re.DOTALL).group(1).rstrip()
    occupation = re.search(r"职业：(.*?)(?=教育状况：)", character, re.DOTALL).group(1).rstrip()
    education = re.search(r"教育状况：(.*?)(?=性格：)", character, re.DOTALL).group(1).rstrip()
    characteristic = re.search(r"性格：(.*?)(?=兴趣：)", character, re.DOTALL).group(1).rstrip()
    interest = re.search(r"兴趣：(.*?)(?=婚姻状况：)", character, re.DOTALL).group(1).rstrip()
    marriage = re.search(r"婚姻状况：(.*?)(?=经济状况：)", character, re.DOTALL).group(1).rstrip()
    economic = re.search(r"经济状况：(.*?)(?=身体健康状况：)", character, re.DOTALL).group(1).rstrip()
    health = re.search(r"身体健康状况：(.*?)(?=心理状况：)", character, re.DOTALL).group(1).rstrip()
    mental = re.search(r"心理状况：(.*?)(?=当前最显著的不适感：)", character, re.DOTALL).group(1).rstrip()
    background = name + ", " + gender + ", " + appearance + ", " + age + ", " + occupation + ", " + education + ", " + characteristic + ", " + interest + ", " + marriage + ", " + economic + ", " + health + ", " + mental
    background_partial = name + ", " + gender + ", " + appearance
    most_prominent_discomfort_initial = re.search(r"当前最显著的不适感：(.*)(?=减轻当前自身最显著不适感计划采取的首要行动：)", character, re.DOTALL).group(1).rstrip()
    plan_initial = re.search(r"不适感计划采取的首要行动：(.*)", character, re.DOTALL).group(1).rstrip()
    most_prominent_discomfort = most_prominent_discomfort_initial
    plan = plan_initial
    # relationship = f"{name}不认识用户"
    with output.use_tags('some_outputs'):
      request = input("输入对生成[聊天场景]的要求（按回车跳过）：")
    output.clear(output_tags='some_outputs')
    if request == "":
        scene = generate_occasion(request="随机生成")
    else:
        scene = generate_occasion(request=request)
    occasion = scene.rstrip()
    # time = re.search(r"时间：(.*?)(?=地点：)", scene, re.DOTALL).group(1).rstrip()
    # place = re.search(r"地点：(.*?)(?=谈话方式：)", scene, re.DOTALL).group(1).rstrip()
    # medium = re.search(fr"谈话方式：(.*?)(?={name}和用户的关系：)", scene, re.DOTALL).group(1).rstrip()
    with output.use_tags('some_outputs'):
      request = input(f"输入对生成[角色-{name}与用户及同伙的关系]的要求（按回车跳过）：")
    output.clear(output_tags='some_outputs')
    if request == "":
        scene = generate_relationship(name=name,request="随机生成")
    else:
        scene = generate_relationship(name=name,request=request)
    relationship_initial = re.search(fr"{name}和用户的关系：(.*?)(?={name}对用户的了解：)", scene, re.DOTALL).group(1).rstrip()
    relationship = relationship_initial
    user_initial = re.search(fr"{name}对用户的了解：(.*)(?={name}对同伙的了解)", scene, re.DOTALL).group(1).rstrip()
    user = user_initial
    background_b_initial = re.search(fr"{name}对同伙的了解：(.*)(?={name}和同伙的关系)", scene, re.DOTALL).group(1).rstrip()
    relationship_b_initial = re.search(fr"{name}和同伙的关系：(.*)", scene, re.DOTALL).group(1).rstrip()
    background_b = background_b_initial
    relationship_b = relationship_b_initial

    task = f"通过审问，让{name}认罪" #挑战
    relationship_history = relationship
    print(f"{BLUE}{BOLD}谈话任务{RESET}")
    print(f"{task}")
    print(f"{BLUE}{BOLD}{name}的背景{RESET}")
    if complete_info:
      print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：{age}\n职业：{occupation}\n教育状况：{education}\n个人特质：{characteristic}\n兴趣：{interest}\n婚姻状况：{marriage}\n经济情况：{economic}\n健康状况：{health}\n心理状况：{mental}\n当前最显著的不适感：{most_prominent_discomfort}\n减轻当前自身最显著不适感计划采取的首要行动：{plan}")#透明模式
    else:
      print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：*\n职业：*\n教育状况：*\n个人特质：*\n兴趣：*\n婚姻状况：*\n经济情况：*\n健康状况：*\n心理状况：*\n当前最显著的不适感：*\n减轻当前自身最显著不适感计划采取的首要行动：*")
    print(f'{BLUE}{BOLD}谈话场合{RESET}')
    print(f'{occasion}')
    print(f"{BLUE}{BOLD}{name}对用户的了解{RESET}")
    print(f"{user}")
    print(f"{BLUE}{BOLD}你们的初始关系{RESET}")
    print(f"{relationship_initial}")
    print(f"{BLUE}{BOLD}{name}对同伙的了解{RESET}")
    print(f"{background_b}")
    print(f"{BLUE}{BOLD}{name}和同伙的关系{RESET}")
    print(f"{relationship_b}")
    relationship = relationship_initial

  data = {}
  decide = "是否承认有罪"
  role_mood = ''
  role_response = ""
  history = f""
  score = 20
  advice = ''
  advice_last = ''
  role_mood_last = ''
  relationship_last = ''
  background_b_last = ''
  relationship_b_last = ''
  user_last = ''
  relationship_history_last = ''
  history_last = f""
  role_response_last = ""
  score_last = 20
  maslow_tips = ''
  last_input = ''
  mark = 0
  is_saved = False
  relationship_save = ''
  relationship_history_save = ''
  history_save = ''
  user_save = ''
  role_response_save = ''
  advice_save = ''
  score_save = 0
  role_mood_save = ''
  background_b_save = ''
  relationship_b_save = ''
while True:
    if mark == 0 and role_response:
      with output.use_tags('some_outputs'):
        user_input = input(f"""
        输入文字进行交谈
        输入 v 进行语音交谈
        输入 l 进行多个输入的平行测试
        输入 a 自动聊天
        输入 d 让对方决策
        输入 t 获取角色与任务相关的马斯洛需求层级
        输入 r 返回上一次谈话
        输入 modify 改进上一次谈话
        输入 infer 推断角色信息
        输入 save 保存，输入 load 载入保存
        输入 quit 退出，输入 restart 重新开始
        """)
    else:
      with output.use_tags('some_outputs'):
        user_input = input(f"""
        输入文字进行交谈
        输入 v 进行语音交谈
        输入 l 进行多个输入的平行测试
        输入 a 自动聊天
        输入 d 让对方决策
        输入 t 获取角色与任务相关的马斯洛需求层级
        输入 infer 推断角色信息
        输入 save 保存，输入 load 载入保存
        输入 quit 退出，输入 restart 重新开始
        """)
    output.clear(output_tags='some_outputs')
    if user_input.lower() == 't':
      if complete_info:
        maslow_tips = maslow_tip(name,most_prominent_discomfort,user,task,background,relationship_history,occasion,decide)
      else:
        maslow_tips = maslow_tip(name,most_prominent_discomfort,user,task,background_partial,relationship_history,occasion,decide
    elif user_input.lower() == 'save':
      is_saved = True
      relationship_save = relationship
      relationship_history_save = relationship_history
      history_save = history
      user_save = user
      role_response_save = role_response
      advice_save = advice
      score_save = score
      role_mood_save = role_mood
      background_b_save = background_b
      relationship_b_save = relationship_b
      most_prominent_discomfort_save = most_prominent_discomfort
      plan_save = plan
      continue
    elif user_input.lower() == 'load':
      if is_saved:
        relationship = relationship_save
        relationship_history = relationship_history_save
        history = history_save
        user = user_save
        role_response = role_response_save
        advice = advice_save
        score = score_save
        role_mood = role_mood_save
        background_b = background_b_save
        relationship_b = relationship_b_save
        most_prominent_discomfort = most_prominent_discomfort_save
        plan = plan_save
        continue
      else:
        print("您还未存储")
        continue
    elif user_input.lower() == 'r' and mark == 0 and role_response:
      mark = 1
      print(f"{RED}{BOLD}注意，您已返回上次谈话。{RESET}")
      relationship = relationship_last
      relationship_history = relationship_history_last
      history = history_last
      role_response = role_response_last
      score = score_last
      user = user_last
      background_b = background_b_last
      relationship_b = relationship_b_last
      advice = advice_last
      role_mood = role_mood_last
      continue
    elif user_input.lower() == 'l':
      print(f"""
      {PURPLE}{BOLD}
      请输入列表，用##分隔
      可以在其中输入 a 自动聊天
      {RESET}
            """)
      user_input = input()
      input_list = user_input.split('##')
      for index, user_input in enumerate(input_list, start=1):
        print(f'{PURPLE}{BOLD}第{index}次{RESET}')
        if user_input.lower() == 'a':
          if complete_info:
            user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background,background_b,relationship_b,relationship_history,occasion,task,decide,maslow_tip,advice)
          else:
            user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background_partial,background_b,relationship_b,relationship_history,occasion,task,decide,maslow_tip,advice)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
          print(f"用户说：{user_input}\n")
        reply_variable_name = f'reply_{index}'
        score_variable_name = f'score_{index}'
        role_response_variable_name = f'role_response_{index}'
        role_mood_variable_name = f'role_mood_{index}'
        relationship_variable_name = f'relationship_{index}'
        background_b_variable_name = f'background_b_{index}'
        relationship_b_variable_name = f'relationship_b_{index}'
        advice_variable_name = f'advice_{index}'
        user_variable_name = f'user_{index}'
        history_variable_name = f'history_{index}'
        relationship_history_variable_name = f'relationship_history_{index}'
        most_prominent_discomfort_variable_name = f'most_prominent_discomfort_{index}'
        globals()[most_prominent_discomfort_variable_name] = re.search(fr"当前最显著的不适感：(.*?)(?={name}减轻当前自身最显著不适感)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        plan_variable_name = f'plan_{index}'
        globals()[plan_variable_name] = re.search(fr"计划采取的首要行动：(.*?)(?={name}心情)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[reply_variable_name] = generate_reply(user_input, user,score,task,history,name,most_prominent_discomfort,plan,role_mood,background,background_b,relationship_b,relationship_history,occasion)
        globals()[score_variable_name] = int(re.search(r"满意程度：(-?\d+)", globals()[reply_variable_name]).group(1))
        globals()[role_response_variable_name] = re.search(r"说：(.*?)(?=得分：)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[role_mood_variable_name] = re.search(fr"{name}心情：(.*?)(?={name}说：)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[relationship_variable_name] = re.search(fr"和用户的关系更新：(.*?)(?={name}对用户的了解更新)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[user_variable_name] = re.search(fr"对用户了解更新：(.*?)(?={name}和同伙的关系更新)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[relationship_b_variable_name] = re.search(fr"和同伙的关系更新：(.*?)(?={name}对同伙的了解更新)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[background_b_variable_name] = re.search(fr"对同伙的了解更新：(.*?)(?={task}的成功概率)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[advice_variable_name] = "用户话语的改进空间：" + re.search(fr"改进空间：(.*?)(?={name}和用户的关系更新)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[advice_variable_name] += '\n'
        globals()[advice_variable_name] += f"增加{task}的成功概率的语用策略："
        globals()[advice_variable_name] += re.search(fr"增加{task}的成功概率的语用策略：(.*)", globals()[reply_variable_name], re.DOTALL).group(1).rstrip()
        globals()[history_variable_name] = history
        globals()[history_variable_name] += f"用户说：{user_input}\n"
        globals()[history_variable_name] += f"{name}说：{globals()[role_response_variable_name]}\n"
        globals()[relationship_history_variable_name] = relationship_history
        globals()[relationship_history_variable_name] += '\n'
        globals()[relationship_history_variable_name] += globals()[relationship_variable_name]
        print(globals()[reply_variable_name])
      user_choice = input("在多个结果中选择一个继续，输入相应的序号（1、2、3...）进行下一步操作，\n或者输入 r 返回上一次交谈")
      if  user_input.lower() == 'r':
        continue
      for index, user_input in enumerate(input_list, start=1):
          if user_choice.isdigit() and int(user_choice) == index:
            mark = 0
            role_response_last = role_response
            role_mood_last = role_mood
            relationship_history_last = relationship_history
            score_last = score
            advice_last = advice
            relationship_last = relationship
            history_last = history
            background_b_last = background_b
            relationship_b_last = relationship_b
            most_prominent_discomfort_last = most_prominent_discomfort
            most_prominent_discomfort = globals()[most_prominent_discomfort_variable_name]
            plan_last = plan
            plan = globals()[plan_variable_name]
            score = globals()[score_variable_name]
            role_response = globals()[role_response_variable_name]
            role_mood = globals()[role_mood_variable_name]
            relationship = globals()[relationship_variable_name]
            advice = globals()[advice_variable_name]
            relationship_b += "\n"
            relationship_b += globals()[relationship_b_variable_name]
            background_b += "\n"
            background_b += globals()[background_b_variable_name]
            user += "\n"
            user += globals()[user_variable_name]
            history = globals()[history_variable_name]
            relationship_history = globals()[relationship_history_variable_name]
            print(f"选择了{user_input},继续")
    else:
      mark = 0
      if user_input.lower() == 'a':
          if complete_info:
            user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background,background_b,relationship_b,relationship_history,occasion,task,decide,maslow_tip,advice)
          else:
            user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background_partial,background_b,relationship_b,relationship_history,occasion,task,decide,maslow_tip,advice)
          user_input = re.search(r"说：(.*)", user_input1)
          if user_input:
            user_input = user_input.group(1).rstrip()
          else:
            user_input = user_input1.rstrip()
      if user_input.lower() == 'quit':
          print("模拟结束，谢谢您的参与！")
          break
      if user_input.lower() == 'restart':
          print("重新开始！")
          background_b = background_b_initial
          relationship_b = relationship_b_initial
          relationship = relationship_initial
          relationship_history = relationship
          user = user_initial
          most_prominent_discomfort = most_prominent_discomfort_initial
          plan = plan_initial
          data = {}
          role_mood = ''
          role_response = ""
          history = f""
          score = 20
          advice = ''
          advice_last = ''
          role_mood_last = ''
          relationship_last = ''
          background_b_last = ''
          relationship_b_last = ''
          user_last = ''
          relationship_history_last = ''
          history_last = f""
          role_response_last = ""
          score_last = 20
          maslow_tips = ''
          last_input = ''
          mark = 0
          continue
      if user_input.lower() == 'd':
          print("对方开始决策...")
          final_decision(history,name,most_prominent_discomfort,user,background,relationship_history,occasion,decide)
          break
      if user_input.lower() == 'v':
          user_input = get_audio()
      if user_input.lower() == 'modify' and role_response:
          print(f"{RED}{BOLD}注意，您正在修改上次输入。{RESET}")
          relationship = relationship_last
          user = user_last
          role_mood = role_mood_last
          background_b = background_b_last
          relationship_b = relationship_b_last
          relationship_history = relationship_history_last
          history = history_last
          role_response = role_response_last
          score = score_last
          if complete_info:
            user_input1 = modify_chat(last_input,history,name,most_prominent_discomfort,role_response,user,background,background_b,relationship_b,relationship_history,occasion,decide,maslow_tips,advice)
          else:
            user_input1 = modify_chat(last_input,history,name,most_prominent_discomfort,role_response,user,background_partial,background_b,relationship_b,relationship_history,occasion,decide,maslow_tips,advice)
          user_input = re.search(r"说：(.*?)(?=修改理由：)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1).rstrip()
          else:
            user_input = user_input1.rstrip()
          advice = advice_last
      if user_input.lower() == 'infer':
          print(f"{RED}{BOLD}注意，您正在推断角色信息。{RESET}")
          character_infer = infer_character(background_partial,history)
          age_infer = re.search(r"年龄：(.*?)(?=职业：)", character_infer, re.DOTALL).group(1).rstrip()
          occupation_infer = re.search(r"职业：(.*?)(?=教育状况：)", character_infer, re.DOTALL).group(1).rstrip()
          education_infer = re.search(r"教育状况：(.*?)(?=性格：)", character_infer, re.DOTALL).group(1).rstrip()
          characteristic_infer = re.search(r"性格：(.*?)(?=兴趣：)", character_infer, re.DOTALL).group(1).rstrip()
          interest_infer = re.search(r"兴趣：(.*?)(?=婚姻状况：)", character_infer, re.DOTALL).group(1).rstrip()
          marriage_infer = re.search(r"婚姻状况：(.*?)(?=经济状况：)", character_infer, re.DOTALL).group(1).rstrip()
          economic_infer = re.search(r"经济状况：(.*?)(?=身体健康状况：)", character_infer, re.DOTALL).group(1).rstrip()
          health_infer = re.search(r"身体健康状况：(.*?)(?=心理状况：)", character_infer, re.DOTALL).group(1).rstrip()
          mental_infer = re.search(r"心理状况：(.*?)(?=当前最显著的不适感：)", character_infer, re.DOTALL).group(1).rstrip()
          most_prominent_discomfort_infer = re.search(r"当前最显著的不适感：(.*)", character, re.DOTALL).group(1).rstrip()
          background_partial = name + ", " + gender + ", " + appearance + ", " + age_infer + ", " + occupation_infer + ", " + education_infer + ", " + characteristic_infer + ", " + interest_infer + ", " + marriage_infer + ", " + economic_infer + ", " + health_infer + ", " + mental_infer
          print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：{age_infer}\n职业：{occupation_infer}\n教育状况：{education_infer}\n个人特质：{characteristic_infer}\n兴趣：{interest_infer}\n婚姻状况：{marriage_infer}\n经济情况：{economic_infer}\n健康状况：{health_infer}\n心理状况：{mental_infer}\n当前最显著的不适感：{most_prominent_discomfort_infer}")#透明模式
          contiune
      last_input = user_input
      reply = generate_reply(user_input, user,score,task,history,name,most_prominent_discomfort,plan,role_mood,background,background_b,relationship_b,relationship_history,occasion)
      if reply == "拒绝":
        break
      relationship_history_last = relationship_history
      history_last = history
      history += f"用户说：{user_input}\n"
      # 调用函数并打印回复
      score_last = score
      forgiveness_value = re.search(r"满意程度：(-?\d+)", reply)
      if forgiveness_value:
          score = int(forgiveness_value.group(1))
      else:
          score = 20
      role_response_last = role_response
      role_response = re.search(r"说：(.*?)(?=得分：)", reply, re.DOTALL)
      if role_response:
        role_response = role_response.group(1).rstrip()
      else:
        role_response = role_response
      role_mood_last = role_mood
      role_mood = re.search(fr"{name}心情：(.*?)(?={name}说：)", reply, re.DOTALL).group(1).rstrip()
      most_prominent_discomfort_last = most_prominent_discomfort
      plan_last = plan
      most_prominent_discomfort = re.search(fr"当前最显著的不适感：(.*?)(?={name}减轻当前自身最显著不适感)", reply, re.DOTALL).group(1).rstrip()
      plan = re.search(fr"计划采取的首要行动：(.*?)(?={name}心情)", reply, re.DOTALL).group(1).rstrip()
      tts(role_response)
      time.sleep(2)
      # role_response = re.search(r"([^\n]+)", role_response)
      # if role_response:
      #   role_response = role_response.group(1)
      # else:
      #   role_response = role_response

      # maslow = re.search(fr"需求层级：(.*?)(?={name}心情：)", reply, re.DOTALL).group(1)
      relationship_last = relationship
      user_last = user
      background_b_last = background_b
      relationship_b_last = relationship_b
      advice_last = advice
      advice = "用户话语的改进空间：" + re.search(fr"改进空间：(.*?)(?={name}和用户的关系更新)", reply, re.DOTALL).group(1).rstrip()
      advice += f"增加{task}的成功概率的语用策略："
      advice += re.search(fr"增加{task}的成功概率的语用策略：(.*)", reply, re.DOTALL).group(1).rstrip()
      relationship = re.search(fr"和用户的关系更新：(.*?)(?={name}对用户的了解更新)", reply, re.DOTALL).group(1).rstrip()
      user += "\n"
      user += re.search(fr"对用户的了解更新：(.*?)(?={name}和同伙的关系更新)", reply, re.DOTALL).group(1).rstrip()
      relationship_b += "\n"
      relationship_b += re.search(fr"和同伙的关系更新：(.*?)(?={name}对同伙的了解更新)", reply, re.DOTALL).group(1).rstrip()
      background_b += "\n"
      background_b += re.search(fr"对同伙的了解更新：(.*?)(?={task}的成功概率)", reply, re.DOTALL).group(1).rstrip()
      relationship_history += '\n'
      relationship_history += relationship.rstrip()
      reply += "\n"
      history += f"{name}说：{role_response}\n"
      print(f'{CYAN}用户说：{user_input}{RESET}')
      print(f'{CYAN}{reply}{RESET}')
      if score < 1:
        ww = ending(history)
        break
      if len(history) > 1500:
        history = summary(history)
      if not data:
        data = {'Name': [name],
        'Gender': [gender],
        'Age': [age],
        'Occupation': [occupation],
        'Education': [education],
        'Characteristics': [characteristic],
        'Interests': [interest],
        'Marital Status': [marriage],
        'Economic Status': [economic],
        'Health': [health],
        'Mental Condition': [mental],
        'Background':[background],
        'Background Partial':[background_partial],
        'User Understanding': [user],
        'Initial Relationship': [relationship_initial],
        'Companion Knowledge': [background_b],
        'Companion Relationship': [relationship_b],

        'Relationship History': [relationship_history],
        'History': [history],
        'User Input':[user_input],
        'Most Prominent Discomfort':[most_prominent_discomfort],
        'Plan':[plan],
        'Role Response':[role_response],
        'Role Mood':[role_mood],
        'Score': [score],
        'Advice': [advice]}
        df = pd.DataFrame(data)
      else:
        new_data = {
        'Companion Knowledge': background_b,
        'Companion Relationship': relationship_b,
        'Background Partial':background_partial,
        'User Understanding': user,
        'Relationship History': relationship_history,
        'History': history,
        'User Input': user_input,
        'Most Prominent Discomfort':most_prominent_discomfort,
        'Plan':plan,
        'Role Response': role_response,
        'Role Mood': role_mood,
        'Score': score,
        'Advice': advice}
        new_index = len(df)）
        df.loc[new_index] = pd.Series(new_data)
        df = df.fillna(df.iloc[0])

# 5、***羊群效应（银行挤兑）***（多人）

## *配置*

### ***基本配置***

In [ ]:
# 角色A的信息
role_A = {
"name": "李华",
"gender": "女",
"appearance": "普通相貌，身高165",
"age": "35岁",
"occupation": "中学教师",
"education": "本科毕业",
"characteristic": "勤奋且保守",
"interest": "喜欢园艺和看书",
"marriage": "已婚，丈夫是工人，有两个孩子",
"economic_status": "收入有限，生活节俭，有一定的储蓄",
"health": "健康状况一般",
"psychological_state": "对未来经济不确定感较强，对银行信任度不高",
"most_prominent_discomfort": "听到可能有银行挤兑的消息后，考虑去银行提取部分储蓄以应对可能的危机",
"relationship_history": ["和王刚是多年的老邻居，经常在一起聊天"],
}

# 角色B的信息
role_B = {
    "name": "王刚",
    "gender": "男",
    "appearance": "小帅，身高182",
    "age": "40岁",
    "occupation": "自由职业者",
    "education": "二本毕业",
    "characteristic": "乐观",
    "interest": "非常好色，对美女感兴趣",
    "marriage": "已婚，老婆很漂亮，有一个7岁的小男孩",
    "economic_status": "收入不稳定，依赖银行贷款维持生活和业务",
    "health": "有胃病",
    "psychological_state": "对经济形势非常敏感，对银行挤兑的新闻反应强烈",
    "most_prominent_discomfort": "听到可能有银行危机的消息后，急忙去银行尝试提取部分资金",
    "relationship_history":["和李华是多年的老邻居，经常在一起聊天"],
}

# 角色A的背景信息合成
background_A = ",".join([
    f"{role_A['name']},",
    f"{role_A['gender']},",
    f"{role_A['appearance']},",
    f"{role_A['age']},",
    f"{role_A['occupation']},",
    f"{role_A['education']},",
    f"{role_A['characteristic']},",
    f"{role_A['interest']},",
    f"{role_A['marriage']},",
    f"{role_A['economic_status']},",
    f"{role_A['health']},",
    f"{role_A['psychological_state']}"])

# 角色B的背景信息合成
background_B = ",".join([
    f"{role_B['name']},",
    f"{role_B['gender']},",
    f"{role_B['appearance']},",
    f"{role_B['age']},",
    f"{role_B['occupation']},",
    f"{role_B['education']},",
    f"{role_B['characteristic']},",
    f"{role_B['interest']},",
    f"{role_B['marriage']},",
    f"{role_B['economic_status']},",
    f"{role_B['health']},",
    f"{role_B['psychological_state']}"])

combined_relationship_history = f"{role_A['name']}的关系：{role_A['relationship_history']}\n{role_B['name']}的关系：{role_B['relationship_history']}"

combined_most_prominent_discomfort = f"{role_A['name']}的处境：{role_A['most_prominent_discomfort']}\n{role_B['name']}的处境：{role_B['most_prominent_discomfort']}"

combined_background = f"{role_A['name']}的背景\n{background_A}\n{role_B['name']}的背景\n{background_B}"

# 用户（银行代表）的信息
user_bank_rep = {
    "role": "银行代表",
    "responsibility": "安抚客户，解释银行的财务状况稳健，挤兑是不必要的",
    "strategy": "提供透明的信息，强调银行的资金充足和政府保障措施",
    "behavior": "在银行门口或通过媒体向公众解释情况，努力平息客户的恐慌情绪"
}

# 场景描述
scene_description = {
    "method": "银行代表在银行门口与顾客对话，通过传单和高扬声公告",
    "time": "银行工作时间",
    "place": "银行门前和内部"
}

user = ",".join([
    user_bank_rep["role"],
   user_bank_rep["behavior"]
])

medium = f"方式：{scene_description['method']}\n"
time = f"时间：{scene_description['time']}\n"
place = f"地点：{scene_description['place']}"
occasion = medium + time + place  # 谈话场景

# 关系
relationship = f""
relationship_history = [relationship]
task = f"成功劝说储户不要挤兑"  # 挑战
decide = "是否继续挤兑"
maslow = ""
role_response = ""
history = ""
score = 20
ww = "放心，我们银行是不会倒闭的"
history = ""

In [ ]:
def generate_reply(ww, user,score,task,history,name,most_prominent_discomfort,background,relationship_history,occasion):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Rules
* 每次根据用户的回复，以及目前对话人的关系演变\n{relationship_history}\n生成{name}的回复，可以在回复中附带相应的表情，回复的内容包括心情、满意程度和双方关系更新。
* 现有的满意程度为{score}，每次交互会增加或者减少满意程度。
* 每次用户回复的话请从-10到10分之间评分，1分为分值起伏：
* -10为最不满意
  0为正常
  +10为最满意
* 每次生成用户对于完成\n{task}\n的成功概率估计：
* 0%为一定失败
  100%为一定成功
* 每次生成{name}的购买意愿指数：
* 0%为一定不购买
  100%为一定购买

## Memory
之前的对话记录和满意程度，对话修改意见以及双方关系变化如下
{history}

## Output format
{name}心情：...
{name}说：...
得分：+-满意程度的变化
满意程度：更新的满意程度
用户话语的改进空间：...
{name}和用户的关系更新：...
{task}的成功概率：...
{name}的购买意愿指数：...
增加{task}的成功概率的语用策略：...
            """},
            {"role": "user", "content": ww}
        ]
    )
    a = completion.choices[0].message.content
    return a

### **基本函数**

In [ ]:
user_input1 = choose_speaker(history,role_A["name"],role_B["name"],combined_most_prominent_discomfort, user,combined_background, combined_relationship_history, occasion)

In [ ]:
generate_reply(ww, user,score,task,history,role_A['name'],role_A['most_prominent_discomfort'],background_A,role_A['relationship_history'],occasion)

In [ ]:
user_input = re.search(r"的角色：(.*?)(?=选择理由：)", user_input1, re.DOTALL)
if user_input:
  user_input = user_input.group(1).rstrip()
else:
  user_input = user_input1.rstrip()

In [ ]:
def choose_speaker(history,role_a,role_b,role_c,combined_most_prominent_discomfort, user,combined_background, combined_relationship_history, occasion):
    # 所有角色背景信息整合


    # 创建一个API请求来选择发言者
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

## Introduction
{role_a}, {role_b}, {role_c}的背景\n{combined_background}\n
{role_a}, {role_b}, {role_c}的当前处境\n{most_prominent_discomfort}\n
用户（银行代表）信息\n{user}\n
{role_a}, {role_b}, {role_c}及用户的关系变化历史\n{combined_relationship_history}\n
谈话场合\n{occasion}\n

## Rules
根据之前的对话记录和满意程度，结合各角色的背景，处境以及目前对话人的关系演变，选择最合适的角色来发言。需要特别考虑每个角色的马斯洛需求层次，以做出谁来发言的决定。

## Turn
发言顺序，对话记录以及对话的反馈如下：
{history}

## Output format
选择的角色：{role_a}/{role_b}/{role_c}
选择理由：...
"""}
        ],
        stream=True
    )
    response = ""
    for chunk in completion:
        content = chunk.choices[0].delta.content
        if content and content.strip():
            print(f"{GRAY}{content}{RESET}", end='')
            response += content
    return response

# 正确定义generate_reply函数
def generate_reply(ww, user,score,task,history,name,most_prominent_discomfort,background,relationship_history,occasion):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Rules
* 每次根据用户的回复，以及目前对话人的关系演变\n{relationship_history}\n生成{name}的回复，可以在回复中附带相应的表情，回复的内容包括心情、满意程度和双方关系更新。
* 现有的满意程度为{score}，每次交互会增加或者减少满意程度。
* 每次用户回复的话请从-10到10分之间评分，1分为分值起伏：
* -10为最不满意
  0为正常
  +10为最满意
* 每次生成用户对于完成\n{task}\n的成功概率估计：
* 0%为一定失败
  100%为一定成功
* 每次生成{name}的挤兑指数：
* 0%为一定不挤兑
  100%为一定挤兑

## Memory
之前的对话记录和满意程度，对话修改意见以及双方关系变化如下
{history}

## Output format
{name}心情：...
{name}说：...
得分：+-满意程度的变化
满意程度：更新的满意程度
用户话语的改进空间：...
{name}和用户的关系更新：...
{task}的成功概率：...
{name}的挤兑指数：...
增加{task}的成功概率的语用策略：...
            """},
            {"role": "user", "content": ww}
        ]
    )
    a = completion.choices[0].message.content
    return a

def auto_chat(history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,task,decide,maslow_tip):
    a = ""
    if role_response:
      completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Goal
用户正在进行协调会，需要通过聊天来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Tip
与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

## Rules
* 每次根据{name}的的回复\n{role_response}\n，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有），以及目前对话人的关系演变\n{relationship_history}\n生成用户的回复，可以在回复中附带相应的表情。

## Memory
之前的对话记录，双方关系变化以及对话回复意见如下\n{history}\n

## Output format
用户说：...

  """}
        ],
        stream=True
    )
      for chunk in completion:
        content = chunk.choices[0].delta.content
        if content and content.strip():
            print(f"{GRAY}{content}{RESET}", end='')
            a += content
      print('\n')
    else:
      completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Goal
用户正在进行面试，需要通过面试聊天来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Rules
* 根据{name}的背景、对用户的了解和之间的关系，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有）,以及目前对话人的关系演变\n{relationship_history}\n生成用户对{name}的开场白，可以在开场白中附带相应的表情。

## Output format
用户说：...

  """}
        ],
        stream=True
    )
      for chunk in completion:
        content = chunk.choices[0].delta.content
        if content and content.strip():
            print(f"{GRAY}{content}{RESET}", end='')
            a += content
      print('\n')

    return a
# 展示task相关的马斯洛需求层级的tip
def maslow_tip(name,most_prominent_discomfort,user,task,background,relationship_history,occasion,decide):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Rules
展示与{name}\n{decide}\n相关的马斯洛需求层级的tip

## Output format
{name}关于{task}的需求等级：
1、生理需求（相关性：高、中、低）(解释）\n
2、安全需求（相关性：高、中、低）(解释）\n
3、社交需求（相关性：高、中、低）(解释）\n
4、尊重需求（相关性：高、中、低）(解释）\n
5、自我实现需求（相关性：高、中、低）(解释）
            """},
            {"role": "user", "content": ww}
        ],
        stream=True
    )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      # 检查内容是否为None或空字符串
      if content and content.strip():
          print(content, end='')
          a += content
    print('\n')
    return a

def final_decision(history,name,most_prominent_discomfort,user,background,relationship_history,occasion,decide):
    a = ""
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景：{background}
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解：{user}
{name}和用户的关系变化历史：{relationship_history}
谈话场合：{occasion}

## Rules
* 根据之前的对话记录和满意程度，结合{name}的背景和对用户的了解，以及目前对话人的关系演变\n{relationship_history}\n最重要是根据{name}的马斯洛需求层次,生成{name}关于\n{decide}\n的决定。

## Memory
之前的对话记录和满意程度，以及双方关系变化如下\n{history}\n

## Output format
{name}决定：...\n
{name}的决定理由：...

  """}
        ],
        stream=True
    )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      if content and content.strip():
          print(f"{GRAY}{content}{RESET}", end='')
          a += content
    print('\n')
    return a

def ending(ww):
    a = ""
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
            对话记录、满意程度以及双方关系变化如下：{ww}
            满意程度低于或等于0则谈话失败。
            如果失败，根据上面的对话总结出对话失败的语用教训"""}
        ],
        stream=True
    )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      # 检查内容是否为None或空字符串
      if content and content.strip():
          print(f"{GRAY}{content}{RESET}", end='')
          a += content
    print('\n')
    return a

def get_resume_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tip):
  if role_response:
    message = client.beta.threads.messages.create(
    thread_id='threadJpd9m93MQ8LpbG1jxMm5lJL',
    role="user",
    content=f"""
  ## Goal
  用户正在进行面试，需要通过面试聊天来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

  ##Introduction
  {name}的背景\n{background}\n
  {name}之前最显著的不适感\n{most_prominent_discomfort}\n
  {name}对用户的了解\n{user}\n
  {name}和用户的关系变化历史\n{relationship_history}\n
  谈话场合\n{occasion}\n

  ## Tip
  与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

  ## Rules
  * 每次根据{name}的回复和回复里的要求\n{role_response}\n根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有），以及目前的关系\n{relationship}\n生成用户的回复，可以在回复中附带相应的表情。

  ## Memory
  之前的对话记录，双方关系变化以及对话回复意见如下\n{history}\n

  ## Output format
  用户说：...

    """,
  )
  else:
    message = client.beta.threads.messages.create(
    thread_id='threadJpd9m93MQ8LpbG1jxMm5lJL',
    role="user",
    content=f"""
  ## Goal
  用户正在进行面试，需要通过面试聊天来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

  ##Introduction
  {name}的背景\n{background}\n
  {name}之前最显著的不适感\n{most_prominent_discomfort}\n
  {name}对用户的了解\n{user}\n
  {name}和用户的关系变化历史\n{relationship_history}\n
  谈话场合\n{occasion}\n

  ## Tip
  与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

  ## Rules
  * 根据{name}的背景、对用户的了解和之间的关系，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有）,以及目前对话人的关系演变\n{relationship_history}\n生成用户对{name}的开场白，可以在开场白中附带相应的表情。

  ## Output format
  用户说：...

    """,
  )
  text_content = ""
  stream = client.beta.threads.runs.create(
      thread_id='threadJpd9m93MQ8LpbG1jxMm5lJL',
      assistant_id='asst_TsBVGRbE5ESfPZN1AzC4UGwj',
      tool_choice = {"type": "file_search"},
      stream=True
  )
  for event in stream:
      # 检查事件是否为ThreadMessageDelta，然后打印文本内容
      if 'ThreadMessageDelta' in str(event):
          message_deltas = event.data.delta.content
          for delta in message_deltas:
              if delta.type == 'text':
                  # 使用print函数的end参数来防止换行，并直接连续打印文本
                  if delta.text.annotations is None or not delta.text.annotations:
                    print(delta.text.value, end='')
                    text_content += delta.text.value
  return text_content

def modify_chat(last_input,history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,decide,maslow_tip):
    a = ""
    completion = client.chat.completions.create(
      model="gpt-4o-2024-05-13",
      messages=[
          {"role": "system", "content": f"""
## Goal
对用户之前的言语\n{last_input}\n提出提示、建议和指导意见，从而提高\n{task}\n的成功概率和{name}的录用意愿指数。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Tip
与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

## Rules
* 每次根据{name}的的回复\n{role_response}\n，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有），生成用户的回复，可以在回复中附带相应的表情。
* 修改用户的话\n{last_input}\n并提出提示、建议和指导意见。

## Memory
之前的对话记录，双方关系变化以及对话回复意见如下\n{history}\n

## Output format
用户说：...\n
修改理由：...\n

"""}
      ],
      stream=True
  )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      if content and content.strip():
          print(f"{GRAY}{content}{RESET}", end='')
          a += content
    print('\n')

    return a


## *运行*

In [ ]:
relationship = f"{name}不认识用户"
relationship_history = [relationship]
role_response = ""
history = f""
score = 20
relationship_last = ''
relationship_history_last = []
history_last = f""
role_response_last = ""
score_last = 20
maslow_tips = ''
last_input = ''
mark = 0

print(f"{BLUE}{BOLD}谈话任务：{task}。{RESET}")
print(f"{BLUE}{BOLD}{name}的背景{RESET}")
print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：{age}\n职业：{occupation}\n教育状况：{education}\n个人特质：{characteristic}\n兴趣：{interest}\n婚姻状况：{marriage}\n经济情况：{economic}\n健康状况：{health}\n心理状况：{mental}\n当前最显著的不适感：{most_prominent_discomfort}")#透明模式
# print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：***\n职业：***\n教育状况：***\n个人特质：***\n兴趣：***\n婚姻状况：***\n经济情况：***\n健康状况：***\n心理状况：***")#真实模式
print(f"{BLUE}{BOLD}{name}对用户的了解{RESET}")
print(f"{user}")
print(f'{BLUE}{BOLD}谈话场合{RESET}')
print(f'{occasion}')
print(f"{BLUE}{BOLD}你们的初始关系{RESET}")
print(f"{relationship}")

while True:
    if mark == 0 and role_response:
      user_input = input(f"""
      输入文字直接进行交谈\n
      输入 l 进行多个输入的平行测试\n
      输入 a 自动聊天\n
      输入 s 求助销售经理，输入 f 求助理财经理\n
      输入 d 让对方决策\n
      输入 t 获取角色与任务相关的马斯洛需求层级\n
      输入 r 返回上一次谈话\n
      输入 modify 改进上一次谈话\n
      输入 quit 退出，输入 restart 重新开始\n
      """)
    else:
      user_input = input(f"""
      输入文字直接进行交谈\n
      输入 l 进行多个输入的平行测试\n
      输入 a 自动聊天\n
      输入 s 求助销售经理，输入 f 求助理财经理\n
      输入 d 让对方决策\n
      输入 t 获取角色与任务相关的马斯洛需求层级\n
      输入 quit 退出，输入 restart 重新开始\n
      """)
    if user_input.lower() == 't':
      maslow_tips = maslow_tip(name,most_prominent_discomfort,user,task,background,relationship_history,occasion,decide)
    elif user_input.lower() == 'r' and mark == 0 and role_response:
      mark = 1
      print(f"{RED}{BOLD}注意，您已返回上次谈话。{RESET}")
      relationship = relationship_last
      relationship_history = relationship_history_last
      history = history_last
      role_response = role_response_last
      score = score_last
      continue
    elif user_input.lower() == 'l':
      print(f"""
      {PURPLE}{BOLD}
      请输入列表，用##分隔\n
      可以在其中输入 a 自动聊天\n
      也可以在其中输入 s 求助销售经理，在其中输入 f 求助理财经理\n
      {RESET}
            """)
      user_input = input()
      input_list = user_input.split('##')
      for index, user_input in enumerate(input_list, start=1):
        print(f'{PURPLE}{BOLD}第{index}次{RESET}')
        if user_input.lower() == 'a':
          user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
          print(f"用户说：{user_input}\n")
        elif user_input.lower() == 's':
          user_input1 = user_input1 = get_sale_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
          print(f"用户说：{user_input}\n")
        elif user_input.lower() == 'f':
          user_input1 = get_fm_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
          print(f"用户说：{user_input}\n")
        reply_variable_name = f'reply_{index}'
        score_variable_name = f'score_{index}'
        role_response_variable_name = f'role_response_{index}'
        relationship_variable_name = f'relationship_{index}'
        history_variable_name = f'history_{index}'
        relationship_history_variable_name = f'relationship_history_{index}'
        globals()[reply_variable_name] = generate_reply(user_input, user,score,task,history,name,most_prominent_discomfort,background,relationship_history,occasion)
        globals()[score_variable_name] = int(re.search(r"满意程度：(-?\d+)", globals()[reply_variable_name]).group(1))
        globals()[role_response_variable_name] = re.search(r"说：(.*?)(?=得分：)", globals()[reply_variable_name], re.DOTALL).group(1)
        globals()[relationship_variable_name] = re.search(fr"关系更新：(.*?)(?={task}的成功概率)", globals()[reply_variable_name], re.DOTALL).group(1)
        globals()[history_variable_name] = history
        globals()[history_variable_name] += f"用户说：{user_input}\n"
        globals()[history_variable_name] += globals()[reply_variable_name]
        globals()[relationship_history_variable_name] = relationship_history.copy()
        globals()[relationship_history_variable_name].append(globals()[relationship_variable_name])
        print(globals()[reply_variable_name])
      user_choice = input("在多个结果中选择一个继续，输入相应的序号（1、2、3...）进行下一步操作，\n或者输入 r 返回上一次交谈")
      if  user_input.lower() == 'r':
        continue
      for index, user_input in enumerate(input_list, start=1):
          if user_choice.isdigit() and int(user_choice) == index:
            mark = 0
            role_response_last = role_response
            relationship_history_last = relationship_history.copy()
            score_last = score
            relationship_last = relationship
            history_last = history
            score = globals()[score_variable_name]
            role_response = globals()[role_response_variable_name]
            relationship = globals()[relationship_variable_name]
            history = globals()[history_variable_name]
            relationship_history = globals()[relationship_history_variable_name].copy()
            print(f"选择了{user_input},继续")
    else:
      mark = 0
      if user_input.lower() == 'a':
          user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
      if user_input.lower() == 'quit':
          print("模拟结束，谢谢您的参与！")
          break
      if user_input.lower() == 'restart':
          print("重新开始！")
          relationship = f"{name}不认识用户"
          relationship_history = [relationship]
          role_response = ""
          history = f""
          score = 20
          relationship_last = ''
          relationship_history_last = []
          history_last = f""
          role_response_last = ""
          score_last = 20
          maslow_tips = ''
          last_input = ''
          mark = 0
          continue
      if user_input.lower() == 'd':
          print("对方开始决策...")
          final_decision(history,name,most_prominent_discomfort,user,background,relationship_history,occasion,decide)
          break
      if user_input.lower() == 's':
          user_input1 = get_sale_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
      if user_input.lower() == 'f':
          user_input1 = get_fm_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
      if user_input.lower() == 'modify' and role_response:
          print(f"{RED}{BOLD}注意，您正在修改上次输入。{RESET}")
          relationship = relationship_last
          relationship_history = relationship_history_last
          history = history_last
          role_response = role_response_last
          score = score_last
          user_input1 = modify_chat(last_input,history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,decide,maslow_tips,advice)
          user_input = re.search(r"说：(.*?)(?=修改理由：)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
      last_input = user_input
      reply = generate_reply(user_input, user,score,task,history,name,most_prominent_discomfort,background,relationship_history,occasion)
      if reply == "拒绝":
        break
      relationship_history_last = relationship_history.copy()
      history_last = history
      history += f"用户说：{user_input}\n"
      # 调用函数并打印回复
      score_last = score
      forgiveness_value = re.search(r"满意程度：(-?\d+)", reply)
      if forgiveness_value:
          score = int(forgiveness_value.group(1))
      else:
          score = 20
      role_response_last = role_response
      role_response = re.search(r"说：(.*?)(?=得分：)", reply, re.DOTALL)
      if role_response:
        role_response = role_response.group(1)
      else:
        role_response = role_response
      # role_response = re.search(r"([^\n]+)", role_response)
      # if role_response:
      #   role_response = role_response.group(1)
      # else:
      #   role_response = role_response

      # maslow = re.search(fr"需求层级：(.*?)(?={name}心情：)", reply, re.DOTALL).group(1)
      relationship_last = relationship
      relationship = re.search(fr"关系更新：(.*?)(?={task}的成功概率)", reply, re.DOTALL).group(1)
      relationship_history.append(relationship)
      reply += "\n"
      history += reply
      print(f'{CYAN}{reply}{RESET}')
      if score < 1:
        ww = ending(history)
        break

# 其他

## 新建file_search助理

In [ ]:
#Rag第一步上传文件
# 获取当前工作目录
current_directory = os.getcwd()
#上传文件
uploaded = files.upload()
for filename in uploaded.keys():
    print('Uploaded file "{name}" with length {length} bytes'.format(
        name=filename, length=len(uploaded[filename])))
#获取文件地址
file_path = os.path.join(current_directory, "resume.pdf")
# 下载文件
# files.download("/content/开题报告.doc")
#V2 上传文件
message_file1 = client.files.create(
  file=open(file_path, "rb"), purpose="assistants"
)


In [ ]:
#V2 新建向量
vector_store = client.beta.vector_stores.create(
  name="resume_doc"
)
print(vector_store)
vector_store_file = client.beta.vector_stores.files.create(
  vector_store_id=vector_store.id,
  file_id=message_file1.id
)

In [ ]:
#V2 建立助理
assistant = client.beta.assistants.create(
  name="面试助理",
  instructions=f"""
        ## Goal
        你需要通过谈话来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n，但是{name}是个很难聊天的人，你需要尽可能的说正确的话来使ta满意，否则{name}会更加不满意。

        ## Introduction
        {name}的背景\n{background}
        {name}对用户的了解\n{user}
        谈话场合\n{occasion}
        """,
  model="gpt-4o-2024-05-13",
  tools=[{"type": "file_search"}],
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)
#V2 更新助理
# assistant = client.beta.assistants.update(
#   assistant_id=assistant.id,
#   tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
# )

In [ ]:
thread_sale = client.beta.threads.create(
)

## 备份tts

In [ ]:
def generate_reply(ww):
    a = ''
    completion = client.chat.completions.create(
      model="gpt-4o-2024-05-13",
      messages=[
          {"role": "system", "content": f"""

以老板语气回复我
          """},
          {"role": "user", "content": ww}
      ]
  )
    a = completion.choices[0].message.content

    return a

In [ ]:
generate_reply("你好")

In [ ]:
while True:
  get_audio()  # 假设这个函数会保存一个名为 'audio.mp3' 的新音频文件
  print("我回复")

  with open("/content/audio.mp3", "rb") as audio_file:
    transcription = client.audio.transcriptions.create(
      model="whisper-1",
      file=audio_file,
      response_format="text"
    )
  ww = generate_reply(transcription)
  with client.audio.speech.with_streaming_response.create(
      model="tts-1",
      voice="shimmer",
      input=ww,
  ) as response:
    response.stream_to_file("speech.mp3")

  with open("speech.mp3", "rb") as f:
    audio_data = f.read()
  print("role回复")
  IPython.display.display(IPython.display.Audio(audio_data, autoplay=True))
  # 正确定义generate_reply函数
def generate_reply(ww):
    a = ''
    completion = client.chat.completions.create(
      model="gpt-4o-2024-05-13",
      messages=[
          {"role": "system", "content": f"""

以老板语气回复我
          """},
          {"role": "user", "content": ww}
      ]
  )
    a = completion.choices[0].message.content

    return a
get_audio()

with client.audio.speech.with_streaming_response.create(
    model="tts-1",
    voice="shimmer",
    input=ww,
) as response:
    response.stream_to_file("speech.mp3")

Audio("speech.mp3", autoplay=True)

## 附带马斯洛的回复

In [ ]:
# 正确定义generate_reply函数
def generate_reply(ww, user,score,task,history,name,maslow,background,relationship_history,occasion):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景：{background}
{name}对用户的了解：{user}
{name}和用户的关系变化历史：{relationship_history}
谈话场合：{occasion}

## Rules
* 每次根据用户的回复、对话记录、满意程度和双方关系变化等，根据之前的马斯洛需求层次\n{maslow}\n,更新{name}的马斯洛需求层次。
* 每次根据用户的回复，生成{name}的回复，回复的内容包括心情、满意程度和双方关系更新。
* 现有的满意程度为{score}，每次交互会增加或者减少满意程度。
* 每次用户回复的话请从-10到10分之间评分，1分为分值起伏：
* -10为最不满意
  0为正常
  +10为最满意
* 每次生成用户对于{task}的需求强度评分：
* 0为无需求
  +10为强烈需求

## Memory
之前的对话记录和满意程度，以及双方关系变化如下
{history}

## Output format
{name}的未满足的需求层级：
1、生理需求
（具体需求1）（重要/不重要）(高频/低频)
（具体需求2）（重要/不重要）(高频/低频)
  ...
2、安全需求（具体需求）(满足情况)
（具体需求1）（重要/不重要）(高频/低频)
（具体需求2）（重要/不重要）(高频/低频)
  ...
3、社交需求（具体需求）(满足情况)
（具体需求1）（重要/不重要）(高频/低频)
（具体需求2）（重要/不重要）(高频/低频)
  ...
4、尊重需求（具体需求）(满足情况)
（具体需求1）（重要/不重要）(高频/低频)
（具体需求2）（重要/不重要）(高频/低频)
  ...
5、自我实现需求（具体需求）(满足情况)
（具体需求1）（重要/不重要）(高频/低频)
（具体需求2）（重要/不重要）(高频/低频)
  ...
{name}心情：...
{name}说：...
得分：+-满意程度的变化
满意程度：更新的满意程度
{name}和用户的关系更新：...
目前，{name}关于{task}的需求强度：...
需求强度评分：需求程度
            """},
            {"role": "user", "content": ww}
        ]
    )
    a = completion.choices[0].message.content
    return a

# 高级

## **（面试运行）同时多个，返回上次的**

In [ ]:
relationship = f"{name}第一次见用户"
relationship_history = [relationship]
role_response = ""
history = f""
score = 20
relationship_last = ''
relationship_history_last = []
history_last = f""
role_response_last = ""
score_last = 20
print(f"{BLUE}{BOLD}谈话任务：{task}。{RESET}")
print(f"{BLUE}{BOLD}{name}的背景{RESET}")
print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：{age}\n职业：{occupation}\n教育状况：{education}\n个人特质：{characteristic}\n兴趣：{interest}\n婚姻状况：{marriage}\n经济情况：{economic}\n健康状况：{health}\n心理状况：{mental}")#透明模式
# print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：***\n职业：***\n教育状况：***\n个人特质：***\n兴趣：***\n婚姻状况：***\n经济情况：***\n健康状况：***\n心理状况：***")#真实模式
print(f"{BLUE}{BOLD}{name}对用户的了解{RESET}")
print(f"{user}")
print(f'{BLUE}{BOLD}谈话场合{RESET}')
print(f'{occasion}')
print(f"{BLUE}{BOLD}你们的初始关系{RESET}")
print(f"{relationship}")
mark = 0
while True:
    if mark == 0 and role_response:
      user_input = input(f"输入文字直接进行交谈\n输入 l 进行多个输入的平行测试\n输入 a 自动聊天\n输入 d 让对方决策\n输入 t 获取角色与任务相关的马斯洛需求层级\n输入 re 让面试助手帮忙\n输入 r 返回上一次谈话\n输入 q 退出\n")
    else:
      user_input = input(f"输入文字直接进行交谈\n输入 l 进行多个输入的平行测试\n输入 a 自动聊天\n输入 d 让对方决策\n输入 t 获取角色与任务相关的马斯洛需求层级\n输入 re 让面试助手帮忙\n输入 q 退出\n")
    if user_input.lower() == 't':
        maslow_tip(name,task,background)
    elif user_input.lower() == 'r' and mark == 0 and role_response:
        mark = 1
        print(f"{RED}{BOLD}注意，您已返回上次谈话。{RESET}")
        relationship = relationship_last
        relationship_history = relationship_history_last
        history = history_last
        role_response = role_response_last
        score = score_last
        continue
    elif user_input.lower() == 'l':
        print(f"{PURPLE}{BOLD}请输入列表，用##分隔\n可以在其中输入 a 自动聊天\n也可以输入 re 让面试助手帮忙{RESET}")
        user_input = input()
        input_list = user_input.split('##')
        for index, user_input in enumerate(input_list, start=1):
          print(f'{PURPLE}{BOLD}第{index}次{RESET}')
          if user_input.lower() == 'a':
            user_input1 = auto_chat(history,name,role_response,user,background,relationship_history,occasion,task)
            user_input = re.search(r"说：(.*)", user_input1)
            if user_input:
              user_input = user_input.group(1)
            else:
              user_input = user_input1
            print(f"用户说：{user_input}\n")
          elif user_input.lower() == 're':
            user_input1 = get_resume_help(name,role_response,history,relationship,relationship_history)
            user_input = re.search(r"说：(.*)", user_input1)
            if user_input:
              user_input = user_input.group(1)
            else:
              user_input = user_input1
            print(f"用户说：{user_input}\n")
          reply_variable_name = f'reply_{index}'
          score_variable_name = f'score_{index}'
          role_response_variable_name = f'role_response_{index}'
          relationship_variable_name = f'relationship_{index}'
          history_variable_name = f'history_{index}'
          relationship_history_variable_name = f'relationship_history_{index}'
          globals()[reply_variable_name] = generate_reply(user_input, user,score,task,history,name,background,relationship_history,occasion)
          globals()[score_variable_name] = int(re.search(r"满意程度：(-?\d+)", globals()[reply_variable_name]).group(1))
          globals()[role_response_variable_name] = re.search(r"说：(.*?)(?=得分：)", globals()[reply_variable_name], re.DOTALL).group(1)
          globals()[relationship_variable_name] = re.search(fr"关系更新：(.*?)(?={task}的成功概率)", globals()[reply_variable_name], re.DOTALL).group(1)
          globals()[history_variable_name] = history
          globals()[history_variable_name] += f"用户说：{user_input}\n"
          globals()[history_variable_name] += globals()[reply_variable_name]
          globals()[relationship_history_variable_name] = relationship_history.copy()
          globals()[relationship_history_variable_name].append(globals()[relationship_variable_name])
          print(globals()[reply_variable_name])
        user_choice = input("在多个结果中选择一个继续，输入相应的序号（1、2、3...）进行下一步操作，\n或者输入 r 返回上一次交谈")
        if  user_input.lower() == 'r':
          continue
        for index, user_input in enumerate(input_list, start=1):
            if user_choice.isdigit() and int(user_choice) == index:
              mark = 0
              role_response_last = role_response
              relationship_history_last = relationship_history.copy()
              score_last = score
              relationship_last = relationship
              history_last = history
              score = globals()[score_variable_name]
              role_response = globals()[role_response_variable_name]
              relationship = globals()[relationship_variable_name]
              history = globals()[history_variable_name]
              relationship_history = globals()[relationship_history_variable_name].copy()
              print(f"选择了{user_input},继续")
    else:
      mark = 0
      if user_input.lower() == 'a':
          user_input1 = auto_chat(history,name,role_response,user,background,relationship_history,occasion,task)
          user_input = re.search(r"说：(.*)", user_input1)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
      if user_input.lower() == 'q':
          print("模拟结束，谢谢您的参与！")
          break
      if user_input.lower() == 'd':
          print("对方开始决策...")
          final_decision(history,name,maslow,user,background,relationship_history,occasion,decide)
          break
      if user_input.lower() == 're':
          user_input1 = get_resume_help(name,role_response,history,relationship,relationship_history)
          user_input = re.search(r"说：(.*)", user_input1)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
      reply = generate_reply(user_input, user,score,task,history,name,background,relationship_history,occasion)
      relationship_history_last = relationship_history.copy()
      history_last = history
      history += f"用户说：{user_input}\n"
      # 调用函数并打印回复
      score_last = score
      forgiveness_value = re.search(r"满意程度：(-?\d+)", reply)
      if forgiveness_value:
          score = int(forgiveness_value.group(1))
      else:
          score = 20
      role_response_last = role_response
      role_response = re.search(r"说：(.*?)(?=得分：)", reply, re.DOTALL)
      if role_response:
        role_response = role_response.group(1)
      else:
        role_response = role_response
      relationship_last = relationship
      relationship = re.search(fr"关系更新：(.*?)(?={task}的成功概率)", reply, re.DOTALL).group(1)
      relationship_history.append(relationship)
      reply += "\n"
      history += reply
      print(f'{CYAN}{reply}{RESET}')
      if score < 1:
        ww = ending(history)
        break

## **（囚徒困境运行）改进上次的发言**

In [ ]:
relationship = f"{name}第一次被用户审问"
relationship_history = [relationship]
role_response = ""
history = f""
score = 20
relationship_last = ''
relationship_history_last = []
history_last = f""
role_response_last = ""
score_last = 20
maslow_tips = ''
last_input = ''
print(f"{BLUE}{BOLD}谈话任务：{task}。{RESET}")
print(f"{BLUE}{BOLD}{name}的背景{RESET}")
print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：{age}\n职业：{occupation}\n教育状况：{education}\n个人特质：{characteristic}\n兴趣：{interest}\n婚姻状况：{marriage}\n经济情况：{economic}\n健康状况：{health}\n心理状况：{mental}\n当前最显著的不适感：{most_prominent_discomfort}")#透明模式
# print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：***\n职业：***\n教育状况：***\n个人特质：***\n兴趣：***\n婚姻状况：***\n经济情况：***\n健康状况：***\n心理状况：***")#真实模式
print(f"{BLUE}{BOLD}{name}对用户的了解{RESET}")
print(f"{user}")
print(f'{BLUE}{BOLD}谈话场合{RESET}')
print(f'{occasion}')
print(f"{BLUE}{BOLD}你们的初始关系{RESET}")
print(f"{relationship}")
mark = 0
while True:
    if mark == 0 and role_response:
      user_input = input(f"""
      输入文字直接进行交谈\n
      输入 l 进行多个输入的平行测试\n
      输入 a 自动聊天\n
      输入 d 让对方决策\n
      输入 t 获取角色与任务相关的马斯洛需求层级\n
      输入 r 返回上一次谈话\n
      输入 modify 改进上一次谈话\n
      输入 q 退出\n
      """)
    else:
      user_input = input(f"""
      输入文字直接进行交谈\n
      输入 l 进行多个输入的平行测试\n
      输入 a 自动聊天\n
      输入 d 让对方决策\n
      输入 t 获取角色与任务相关的马斯洛需求层级\n
      输入 q 退出\n
      """)
    if user_input.lower() == 't':
        maslow_tips = maslow_tip(name,most_prominent_discomfort,user,task,background,background_b,relationship_b,relationship_history,occasion,decide)
    elif user_input.lower() == 'r' and mark == 0 and role_response:
        mark = 1
        print(f"{RED}{BOLD}注意，您已返回上次谈话。{RESET}")
        relationship = relationship_last
        relationship_history = relationship_history_last
        history = history_last
        role_response = role_response_last
        score = score_last
        continue
    elif user_input.lower() == 'l':
        print(f"{PURPLE}{BOLD}请输入列表，用##分隔\n可以在其中输入 a 自动聊天{RESET}")
        user_input = input()
        input_list = user_input.split('##')
        for index, user_input in enumerate(input_list, start=1):
          print(f'{PURPLE}{BOLD}第{index}次{RESET}')
          if user_input.lower() == 'a':
            user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background,background_b,relationship_b,relationship_history,occasion,task,decide,maslow_tips)
            user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
            if user_input:
              user_input = user_input.group(1)
            else:
              user_input = user_input1
            print(f"用户说：{user_input}\n")
          reply_variable_name = f'reply_{index}'
          score_variable_name = f'score_{index}'
          role_response_variable_name = f'role_response_{index}'
          relationship_variable_name = f'relationship_{index}'
          history_variable_name = f'history_{index}'
          relationship_history_variable_name = f'relationship_history_{index}'
          globals()[reply_variable_name] = generate_reply(user_input, user,score,task,history,name,most_prominent_discomfort,background,background_b,relationship_b,relationship_history,occasion)
          globals()[score_variable_name] = int(re.search(r"满意程度：(-?\d+)", globals()[reply_variable_name]).group(1))
          globals()[role_response_variable_name] = re.search(r"说：(.*?)(?=得分：)", globals()[reply_variable_name], re.DOTALL).group(1)
          globals()[relationship_variable_name] = re.search(fr"关系更新：(.*?)(?={task}的成功概率)", globals()[reply_variable_name], re.DOTALL).group(1)
          globals()[history_variable_name] = history
          globals()[history_variable_name] += f"用户说：{user_input}\n"
          globals()[history_variable_name] += globals()[reply_variable_name]
          globals()[relationship_history_variable_name] = relationship_history.copy()
          globals()[relationship_history_variable_name].append(globals()[relationship_variable_name])
          print(globals()[reply_variable_name])
        user_choice = input("在多个结果中选择一个继续，输入相应的序号（1、2、3...）进行下一步操作，\n或者输入 r 返回上一次交谈")
        if  user_input.lower() == 'r':
          continue
        for index, user_input in enumerate(input_list, start=1):
            if user_choice.isdigit() and int(user_choice) == index:
              mark = 0
              role_response_last = role_response
              relationship_history_last = relationship_history.copy()
              score_last = score
              relationship_last = relationship
              history_last = history
              score = globals()[score_variable_name]
              role_response = globals()[role_response_variable_name]
              relationship = globals()[relationship_variable_name]
              history = globals()[history_variable_name]
              relationship_history = globals()[relationship_history_variable_name].copy()
              print(f"选择了{user_input},继续")
    else:
      mark = 0
      if user_input.lower() == 'a':
          user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background,background_b,relationship_b,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
      if user_input.lower() == 'q':
          print("模拟结束，谢谢您的参与！")
          break
      if user_input.lower() == 'd':
          print("对方开始决策...")
          final_decision(history,name,most_prominent_discomfort,user,background,background_b,relationship_b,relationship_history,occasion,decide)
          break
      if user_input.lower() == 'modify' and role_response:
        print(f"{RED}{BOLD}注意，您正在修改上次输入。{RESET}")
        relationship = relationship_last
        relationship_history = relationship_history_last
        history = history_last
        role_response = role_response_last
        score = score_last
        user_input1 = modify_chat(last_input,history,name,most_prominent_discomfort,role_response,user,background,background_b,relationship_b,relationship_history,occasion,decide,maslow_tips,advice)
        user_input = re.search(r"说：(.*?)(?=修改理由：)", user_input1, re.DOTALL)
        if user_input:
          user_input = user_input.group(1)
        else:
          user_input = user_input1
      last_input = user_input
      reply = generate_reply(user_input, user,score,task,history,name,most_prominent_discomfort,background,background_b,relationship_b,relationship_history,occasion)
      relationship_history_last = relationship_history.copy()
      history_last = history
      history += f"用户说：{user_input}\n"
      # 调用函数并打印回复
      score_last = score
      forgiveness_value = re.search(r"满意程度：(-?\d+)", reply)
      if forgiveness_value:
          score = int(forgiveness_value.group(1))
      else:
          score = 20
      role_response_last = role_response
      role_response = re.search(r"说：(.*?)(?=得分：)", reply, re.DOTALL)
      if role_response:
        role_response = role_response.group(1)
      else:
        role_response = role_response
      relationship_last = relationship
      relationship = re.search(fr"关系更新：(.*?)(?={task}的成功概率)", reply, re.DOTALL).group(1)
      relationship_history.append(relationship)
      reply += "\n"
      history += reply
      print(f'{CYAN}{reply}{RESET}')
      if score < 1:
        ww = ending(history)
        break

## **去掉马斯洛（卖理财）**

In [ ]:
generate_reply(ww, user,score,task,history,name,action,most_prominent_discomfort,background,relationship_history,occasion)

In [ ]:
# 正确定义generate_reply函数
def generate_reply(ww, user,score,task,history,name,action,most_prominent_discomfort,background,relationship_history,occasion):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}的当前行动\n{action}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Rules
* {name}的当前行动是为了解决当前最显著的不适感。
* 每次根据用户的回复，以及目前对话人的关系演变\n{relationship_history}\n生成{name}的回复，可以在回复中附带相应的表情，回复的内容包括心情、满意程度和双方关系更新。
* 现有的满意程度为{score}，每次交互会增加或者减少满意程度。
* 每次用户回复的话请从-10到10分之间评分，1分为分值起伏：
* -10为最不满意
  0为正常
  +10为最满意
* 每次生成用户对于完成\n{task}\n的成功概率估计：
* 0%为一定失败
  100%为一定成功
* 每次生成{name}的购买意愿指数：
* 0%为一定不购买
  100%为一定购买

## Memory
之前的对话记录和满意程度，对话修改意见以及双方关系变化如下
{history}

## Output format
{name}更新当前行动：...
{name}更新当前最显著的不适感：...
{name}心情：...
{name}说：...
得分：+-满意程度的变化
满意程度：更新的满意程度
用户话语的改进空间：...
{name}和用户的关系更新：...
{task}的成功概率：...
{name}的购买意愿指数：...
增加{task}的成功概率的语用策略：...
            """},
            {"role": "user", "content": ww}
        ]
    )
    a = completion.choices[0].message.content
    return a

def auto_chat(history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,task,decide,maslow_tip):
    a = ""
    if role_response:
      completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Goal
用户正在向{name}推销理财产品，需要通过闲聊来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Tip
与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

## Rules
* 每次根据{name}的的回复\n{role_response}\n，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有），以及目前对话人的关系演变\n{relationship_history}\n生成用户的回复，可以在回复中附带相应的表情。

## Memory
之前的对话记录，双方关系变化以及对话回复意见如下\n{history}\n

## Output format
用户说：...

  """}
        ],
        stream=True
    )
      for chunk in completion:
        content = chunk.choices[0].delta.content
        if content and content.strip():
            print(f"{GRAY}{content}{RESET}", end='')
            a += content
      print('\n')
    else:
      completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
## Goal
用户正在向{name}推销理财产品，需要通过闲聊来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Rules
* 根据{name}的背景、对用户的了解和之间的关系，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有）,以及目前对话人的关系演变\n{relationship_history}\n生成用户对{name}的开场白，可以在开场白中附带相应的表情。

## Output format
用户说：...

  """}
        ],
        stream=True
    )
      for chunk in completion:
        content = chunk.choices[0].delta.content
        if content and content.strip():
            print(f"{GRAY}{content}{RESET}", end='')
            a += content
      print('\n')

    return a

def final_decision(history,name,most_prominent_discomfort,user,background,relationship_history,occasion,decide):
    a = ""
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景：{background}
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解：{user}
{name}和用户的关系变化历史：{relationship_history}
谈话场合：{occasion}

## Rules
* 根据之前的对话记录和满意程度，结合{name}的背景和对用户的了解，以及目前对话人的关系演变\n{relationship_history}\n最重要是根据{name}的马斯洛需求层次,生成{name}关于\n{decide}\n的决定。

## Memory
之前的对话记录和满意程度，以及双方关系变化如下\n{history}\n

## Output format
{name}决定：...\n
{name}的决定理由：...

  """}
        ],
        stream=True
    )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      if content and content.strip():
          print(f"{GRAY}{content}{RESET}", end='')
          a += content
    print('\n')
    return a
def decision_to_talk(ww,name,most_prominent_discomfort,user,background,relationship,occasion,task):
    a = ""
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系初始关系\n{relationship}\n
谈话场合\n{occasion}\n

## Rules
* 根据{name}的背景、处境、对用户的了解、两人之间的关系、谈话场合和用户开场白\n{ww}\n生成{name}关于是否与用户交谈的决定。
* 每次生成{name}的接受对话指数：
* 0%为拒绝
  100%为接受

## Output format
{name}决定：接受/拒绝
{name}的决定理由：结合{name}的马斯洛需求层级说明作出上述决定的理由
用户开场白的改进空间：...
{name}的接受对话指数：...
  """}
        ],
        stream=True
    )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      if content and content.strip():
          print(f"{GRAY}{content}{RESET}", end='')
          a += content
    print('\n')
    return a

def ending(ww):
    a = ""
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""
            对话记录、满意程度以及双方关系变化如下：{ww}
            满意程度低于或等于0则谈话失败。
            如果失败，根据上面的对话总结出对话失败的语用教训"""}
        ],
        stream=True
    )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      # 检查内容是否为None或空字符串
      if content and content.strip():
          print(f"{GRAY}{content}{RESET}", end='')
          a += content
    print('\n')
    return a

# 展示task相关的马斯洛需求层级的tip
def maslow_tip(name,most_prominent_discomfort,user,task,background,relationship_history,occasion,decide):
    a = ''
    completion = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": f"""

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Rules
展示与{name}\n{decide}\n相关的马斯洛需求层级的tip

## Output format
{name}关于{task}的需求等级：
1、生理需求（相关性：高、中、低）(解释）\n
2、安全需求（相关性：高、中、低）(解释）\n
3、社交需求（相关性：高、中、低）(解释）\n
4、尊重需求（相关性：高、中、低）(解释）\n
5、自我实现需求（相关性：高、中、低）(解释）
            """}
        ],
        stream=True
    )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      # 检查内容是否为None或空字符串
      if content and content.strip():
          print(content, end='')
          a += content
    print('\n')
    return a

def get_sale_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tip):
  if role_response:
    message = client.beta.threads.messages.create(
  thread_id="thread_GWJFzlZe5zsW0KK6X6SpAAH0",
  role="user",
  content=f"""
## Goal
用户正在向{name}推销理财产品，需要通过闲聊来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Tip
与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

## Rules
* 每次根据{name}的回复和回复里的要求\n{role_response}\n根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有），以及目前的关系\n{relationship}\n生成用户的回复，可以在回复中附带相应的表情。

## Memory
之前的对话记录，双方关系变化以及对话回复意见如下\n{history}\n

## Output format
用户说：...
  """,
)
  else:
    message = client.beta.threads.messages.create(
  thread_id="thread_GWJFzlZe5zsW0KK6X6SpAAH0",
  role="user",
  content=f"""
## Goal
用户正在向{name}推销理财产品，需要通过闲聊来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Tip
与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

## Rules
* 根据{name}的背景、对用户的了解和之间的关系，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有）,以及目前对话人的关系演变\n{relationship_history}\n生成用户对{name}的开场白，可以在开场白中附带相应的表情。

## Output format
用户说：...
  """,
)
  text_content = ""
  stream = client.beta.threads.runs.create(
      thread_id="thread_GWJFzlZe5zsW0KK6X6SpAAH0",
      assistant_id='asst_q9yTNWJvPZ7Mv4ihGFOkYxhS',
      stream=True
  )
  for event in stream:
      # 检查事件是否为ThreadMessageDelta，然后打印文本内容
      if 'ThreadMessageDelta' in str(event):
          message_deltas = event.data.delta.content
          for delta in message_deltas:
              if delta.type == 'text':
                  # 使用print函数的end参数来防止换行，并直接连续打印文本
                  if delta.text.annotations is None or not delta.text.annotations:
                    print(delta.text.value, end='')
                    text_content += delta.text.value
  return text_content
def get_fm_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tip):
  if role_response:
    message = client.beta.threads.messages.create(
    thread_id="thread_Qx999ZM9UYIIY4xyEg8Xa8ns",
    role="user",
    content=f"""
 ## Goal
用户正在向{name}推销理财产品，需要通过闲聊来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Tip
与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

## Rules
* 每次根据{name}的回复和回复里的要求\n{role_response}\n根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有），以及目前的关系\n{relationship}\n生成用户的回复，可以在回复中附带相应的表情。

## Memory
之前的对话记录，双方关系变化以及对话回复意见如下\n{history}\n

## Output format
用户说：...
    """,
  )
  else:
    message = client.beta.threads.messages.create(
    thread_id=thread_id,#"thread_Qx999ZM9UYIIY4xyEg8Xa8ns"
    role="user",
    content=f"""
 ## Goal
用户正在向{name}推销理财产品，需要通过闲聊来改变{name}的马斯洛需求层级，从而完成任务\n{task}\n。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Tip
与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

## Rules
* 根据{name}的背景、对用户的了解和之间的关系，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有）,以及目前对话人的关系演变\n{relationship_history}\n生成用户对{name}的开场白，可以在开场白中附带相应的表情。

## Output format
用户说：...
    """,
  )
  text_content = ""
  stream = client.beta.threads.runs.create(
      thread_id="thread_Qx999ZM9UYIIY4xyEg8Xa8ns",
      assistant_id='asst_vVkQ5k14p8rm2iwH4GxuY07m',
      tool_choice = {"type": "file_search"},
      stream=True
  )
  for event in stream:
      # 检查事件是否为ThreadMessageDelta，然后打印文本内容
      if 'ThreadMessageDelta' in str(event):
          message_deltas = event.data.delta.content
          for delta in message_deltas:
              if delta.type == 'text':
                  # 使用print函数的end参数来防止换行，并直接连续打印文本
                  if delta.text.annotations is None or not delta.text.annotations:
                    print(delta.text.value, end='')
                    text_content += delta.text.value
  return text_content

def modify_chat(last_input,history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,decide,maslow_tip):
    a = ""
    completion = client.chat.completions.create(
      model="gpt-4o-2024-05-13",
      messages=[
          {"role": "system", "content": f"""
## Goal
对用户之前的言语\n{last_input}\n提出提示、建议和指导意见，从而提高\n{task}\n的成功概率和{name}的购买意愿指数。

##Introduction
{name}的背景\n{background}\n
{name}之前最显著的不适感\n{most_prominent_discomfort}\n
{name}对用户的了解\n{user}\n
{name}和用户的关系变化历史\n{relationship_history}\n
谈话场合\n{occasion}\n

## Tip
与{name}\n{decide}\n相关的马斯洛需求层级的tip\n{maslow_tip}\n

## Rules
* 每次根据{name}的的回复\n{role_response}\n，根据与{name}\n{decide}\n相关的马斯洛需求层级的tip（如果有），生成用户的回复，可以在回复中附带相应的表情。
* 修改用户的话\n{last_input}\n并提出提示、建议和指导意见。

## Memory
之前的对话记录，双方关系变化以及对话回复意见如下\n{history}\n

## Output format
用户说：...\n
修改理由：...\n

"""}
      ],
      stream=True
  )
    for chunk in completion:
      content = chunk.choices[0].delta.content
      if content and content.strip():
          print(f"{GRAY}{content}{RESET}", end='')
          a += content
    print('\n')

    return a


In [ ]:
relationship = f"{name}不认识用户"
relationship_history = [relationship]
role_response = ""
history = f""
score = 20
relationship_last = ''
relationship_history_last = []
history_last = f""
role_response_last = ""
score_last = 20
maslow_tips = ''
last_input = ''
mark = 0

print(f"{BLUE}{BOLD}谈话任务：{task}。{RESET}")
print(f"{BLUE}{BOLD}{name}的背景{RESET}")
print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：{age}\n职业：{occupation}\n教育状况：{education}\n个人特质：{characteristic}\n兴趣：{interest}\n婚姻状况：{marriage}\n经济情况：{economic}\n健康状况：{health}\n心理状况：{mental}\n当前最显著的不适感：{most_prominent_discomfort}")#透明模式
# print(f"姓名：{name}\n性别：{gender}\n外貌：{appearance}\n年龄：***\n职业：***\n教育状况：***\n个人特质：***\n兴趣：***\n婚姻状况：***\n经济情况：***\n健康状况：***\n心理状况：***")#真实模式
print(f"{BLUE}{BOLD}{name}对用户的了解{RESET}")
print(f"{user}")
print(f'{BLUE}{BOLD}谈话场合{RESET}')
print(f'{occasion}')
print(f"{BLUE}{BOLD}你们的初始关系{RESET}")
print(f"{relationship}")

while True:
    if mark == 0 and role_response:
      user_input = input(f"""
      输入文字直接进行交谈\n
      输入 l 进行多个输入的平行测试\n
      输入 a 自动聊天\n
      输入 s 求助销售经理，输入 f 求助理财经理\n
      输入 d 让对方决策\n
      输入 t 获取角色与任务相关的马斯洛需求层级\n
      输入 r 返回上一次谈话\n
      输入 modify 改进上一次谈话\n
      输入 quit 退出，输入 restart 重新开始\n
      """)
    else:
      user_input = input(f"""
      输入文字直接进行交谈\n
      输入 l 进行多个输入的平行测试\n
      输入 a 自动聊天\n
      输入 s 求助销售经理，输入 f 求助理财经理\n
      输入 d 让对方决策\n
      输入 t 获取角色与任务相关的马斯洛需求层级\n
      输入 quit 退出，输入 restart 重新开始\n
      """)
    if user_input.lower() == 't':
      maslow_tips = maslow_tip(name,most_prominent_discomfort,user,task,background,relationship_history,occasion,decide)
    elif user_input.lower() == 'r' and mark == 0 and role_response:
      mark = 1
      print(f"{RED}{BOLD}注意，您已返回上次谈话。{RESET}")
      relationship = relationship_last
      relationship_history = relationship_history_last
      history = history_last
      role_response = role_response_last
      score = score_last
      continue
    elif user_input.lower() == 'l':
      print(f"""
      {PURPLE}{BOLD}
      请输入列表，用##分隔\n
      可以在其中输入 a 自动聊天\n
      也可以在其中输入 s 求助销售经理，在其中输入 f 求助理财经理\n
      {RESET}
            """)
      user_input = input()
      input_list = user_input.split('##')
      for index, user_input in enumerate(input_list, start=1):
        print(f'{PURPLE}{BOLD}第{index}次{RESET}')
        if user_input.lower() == 'a':
          user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
          print(f"用户说：{user_input}\n")
        elif user_input.lower() == 's':
          user_input1 = user_input1 = get_sale_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
          print(f"用户说：{user_input}\n")
        elif user_input.lower() == 'f':
          user_input1 = get_fm_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
          print(f"用户说：{user_input}\n")
        reply_variable_name = f'reply_{index}'
        score_variable_name = f'score_{index}'
        role_response_variable_name = f'role_response_{index}'
        relationship_variable_name = f'relationship_{index}'
        history_variable_name = f'history_{index}'
        relationship_history_variable_name = f'relationship_history_{index}'
        globals()[reply_variable_name] = generate_reply(user_input, user,score,task,history,name,action,most_prominent_discomfort,background,relationship_history,occasion)
        globals()[score_variable_name] = int(re.search(r"满意程度：(-?\d+)", globals()[reply_variable_name]).group(1))
        globals()[role_response_variable_name] = re.search(r"说：(.*?)(?=得分：)", globals()[reply_variable_name], re.DOTALL).group(1)
        globals()[relationship_variable_name] = re.search(fr"关系更新：(.*?)(?={task}的成功概率)", globals()[reply_variable_name], re.DOTALL).group(1)
        globals()[history_variable_name] = history
        globals()[history_variable_name] += f"用户说：{user_input}\n"
        globals()[history_variable_name] += globals()[reply_variable_name]
        globals()[relationship_history_variable_name] = relationship_history.copy()
        globals()[relationship_history_variable_name].append(globals()[relationship_variable_name])
        print(globals()[reply_variable_name])
      user_choice = input("在多个结果中选择一个继续，输入相应的序号（1、2、3...）进行下一步操作，\n或者输入 r 返回上一次交谈")
      if  user_input.lower() == 'r':
        continue
      for index, user_input in enumerate(input_list, start=1):
          if user_choice.isdigit() and int(user_choice) == index:
            mark = 0
            role_response_last = role_response
            relationship_history_last = relationship_history.copy()
            score_last = score
            relationship_last = relationship
            history_last = history
            score = globals()[score_variable_name]
            role_response = globals()[role_response_variable_name]
            relationship = globals()[relationship_variable_name]
            history = globals()[history_variable_name]
            relationship_history = globals()[relationship_history_variable_name].copy()
            print(f"选择了{user_input},继续")
    else:
      mark = 0
      if user_input.lower() == 'a':
          user_input1 = auto_chat(history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
      if user_input.lower() == 'quit':
          print("模拟结束，谢谢您的参与！")
          break
      if user_input.lower() == 'restart':
          print("重新开始！")
          relationship = f"{name}不认识用户"
          relationship_history = [relationship]
          role_response = ""
          history = f""
          score = 20
          relationship_last = ''
          relationship_history_last = []
          history_last = f""
          role_response_last = ""
          score_last = 20
          maslow_tips = ''
          last_input = ''
          mark = 0
          continue
      if user_input.lower() == 'd':
          print("对方开始决策...")
          final_decision(history,name,most_prominent_discomfort,user,background,relationship_history,occasion,decide)
          break
      if user_input.lower() == 's':
          user_input1 = get_sale_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
      if user_input.lower() == 'f':
          user_input1 = get_fm_help(history,name,most_prominent_discomfort,role_response,user,background,relationship,relationship_history,occasion,task,decide,maslow_tips)
          user_input = re.search(r"说：(.*)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
      if user_input.lower() == 'modify' and role_response:
          print(f"{RED}{BOLD}注意，您正在修改上次输入。{RESET}")
          relationship = relationship_last
          relationship_history = relationship_history_last
          history = history_last
          role_response = role_response_last
          score = score_last
          user_input1 = modify_chat(last_input,history,name,most_prominent_discomfort,role_response,user,background,relationship_history,occasion,decide,maslow_tips,advice)
          user_input = re.search(r"说：(.*?)(?=修改理由：)", user_input1, re.DOTALL)
          if user_input:
            user_input = user_input.group(1)
          else:
            user_input = user_input1
      last_input = user_input
      reply = generate_reply(user_input, user,score,task,history,name,action,most_prominent_discomfort,background,relationship_history,occasion)
      if reply == "拒绝":
        break
      relationship_history_last = relationship_history.copy()
      history_last = history
      history += f"用户说：{user_input}\n"
      # 调用函数并打印回复
      score_last = score
      forgiveness_value = re.search(r"满意程度：(-?\d+)", reply)
      if forgiveness_value:
          score = int(forgiveness_value.group(1))
      else:
          score = 20
      role_response_last = role_response
      role_response = re.search(r"说：(.*?)(?=得分：)", reply, re.DOTALL)
      if role_response:
        role_response = role_response.group(1)
      else:
        role_response = role_response
      # role_response = re.search(r"([^\n]+)", role_response)
      # if role_response:
      #   role_response = role_response.group(1)
      # else:
      #   role_response = role_response

      # maslow = re.search(fr"需求层级：(.*?)(?={name}心情：)", reply, re.DOTALL).group(1)
      relationship_last = relationship
      relationship = re.search(fr"关系更新：(.*?)(?={task}的成功概率)", reply, re.DOTALL).group(1)
      relationship_history.append(relationship)
      reply += "\n"
      history += reply
      print(f'{CYAN}{reply}{RESET}')
      if score < 1:
        ww = ending(history)
        break

# *存取*

## *结果导出*

In [ ]:
# 假设这些是你从交谈中获得的数据
data = {
    'Name': [name],
    'Gender': [gender],
    'Age': [age],
    'Occupation': [occupation],
    'Education': [education],
    'Characteristics': [characteristic],
    'Interests': [interest],
    'Marital Status': [marriage],
    'Economic Status': [economic],
    'Health': [health],
    'Mental Condition': [mental],
    'most_prominent_discomfort': [most_prominent_discomfort],
    'User Understanding': [user],
    'Initial Relationship': [relationship_initial],
    'Relationship History': [str(relationship_history)],
    'History': [history],  # 假设 history 是一个字符串，包含所有谈话记录
    'User Input':[user_input],
    'Role Response':[role_response],
    'Role Mood':[role_mood],
    'Score': [score],
    'Advice': [advice]
}

# 创建 DataFrame
df = pd.DataFrame(data)

# 输出 DataFrame 到 Excel 文件
df.to_excel('simulation_data.xlsx', index=False, engine='openpyxl')
files.download('simulation_data.xlsx')


In [ ]:
import pandas as pd

# 假设这些是你从交谈中获得的初始数据
data = {
    'Name': ['Alice'],
    'Gender': ['Female'],
    'Age': [30],
    'Occupation': ['Scientist'],
    'Education': ['PhD'],
    'Characteristics': ['Curious'],
    'Interests': ['Reading'],
    'Marital Status': ['Single'],
    'Economic Status': ['Middle class'],
    'Health': ['Good'],
    'Mental Condition': ['Stable'],
    'most_prominent_discomfort': ['Looking for advice'],
    'User Understanding': ['Partial'],
    'Initial Relationship': ['None'],
    'Relationship History': ['Initial contact'],
    'History': ['First conversation'],  # 假设 history 是一个字符串，包含所有谈话记录
    'User Input':['Question about data handling'],
    'Role Response':['Provided instruction on handling data'],
    'Role Mood':['Helpful'],
    'Score': [80],
    'Advice': ['Keep data organized']
}

# 创建 DataFrame
df = pd.DataFrame(data)

# 新的用户输入和其他相关信息
new_data = {
    'User Input': 'Follow-up question about data expansion',
    'Role Response': 'Explanation on expanding data rows',
    'Role Mood': 'Encouraging',
    'Score': 85,
    'Advice': 'Use loc for adding rows'
}

# 使用 loc 添加新行，保留某些列的值
new_index = len(df)  # 新行的索引将是现有行数（因为索引从0开始）
df.loc[new_index] = df.loc[0]  # 复制第一行的数据到新行
df.loc[new_index, new_data.keys()] = new_data.values()  # 更新新行的特定列

# 显示更新后的 DataFrame
print(df)


In [ ]:
df.to_excel('data.xlsx', index=False, engine='openpyxl')
files.download('data.xlsx')

## *结果导入*

In [ ]:
uploaded = files.upload()
filename = next(iter(uploaded))
df3 = pd.read_excel(filename)
last_row = df3.iloc[-1]
name = last_row['Name']
gender = last_row['Gender']
age = last_row['Age']
occupation = last_row['Occupation']
education = last_row['Education']
characteristic = last_row['Characteristics']
interest = last_row['Interests']
marriage = last_row['Marital Status']
economic = last_row['Economic Status']
health = last_row['Health']
mental = last_row['Mental Condition']
most_prominent_discomfort = last_row['most_prominent_discomfort']
user = last_row['User Understanding']
relationship_initial = last_row['Initial Relationship']

relationship_history = last_row['Relationship History']
history = last_row['History']
user_input = last_row['User Input']
role_response = last_row['Role Response']
role_mood = last_row['Role Mood']
score = last_row['Score']
advice = last_row['Advice']